In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sentiment Preprocessing

In [3]:
df_comment_opinion=pd.read_csv('/content/drive/MyDrive/5153/index/Copy of df_all_predict_0422.csv')
df_comment_opinion

stock        Date                                               text  \
0      BABA  2023-01-31  China's reopening has revived its economy but ...   
1      BABA  2023-01-26  Investors seem to be positioning for a strong ...   
2      BABA  2023-01-17  E-commerce behemoth Alibaba Group Holdings (NY...   
3      BABA  2023-01-13  Alibaba shares are up 100% from their lows. Ta...   
4      BABA  2023-01-13  The continued easing of US price pressures has...   
...     ...         ...                                                ...   
26136   NIO  2022-07-15  #TSLA Held 20 Day SMA today and closed above 7...   
26137  TSLA  2022-02-24                     750 as I wrote 10 hours ago...   
26138  TSLA  2022-11-10                           Crypto will push it down   
26139  TSLA  2022-03-04  This really doesnt want to move like the other...   
26140  TSLA  2022-04-25                                  Just can't sleep.   

       category  label  
0             2      1  
1             2      1  
2             2      1  
3             2      1  
4             2      0  
...         ...    ...  
26136         1      1  
26137         1      1  
26138         1      2  
26139         1      0  
26140         1      0  

[26141 rows x 5 columns]

In [4]:
# drop rows with stock = "A"
df_comment_opinion = df_comment_opinion[df_comment_opinion['stock'] != 'A']

# modify the 'label' column using loc indexer
df_comment_opinion.loc[df_comment_opinion['label'] == 2, 'label'] = -1

In [5]:
# Filter out the rows with category=1
df_comment = df_comment_opinion[df_comment_opinion['category']== 1]
df_opinion = df_comment_opinion[df_comment_opinion['category']== 2]

In [6]:
df_comment

stock        Date                                               text  \
2545   AAPL  2022-10-08               5 more points in 30 minutes possible   
2546   AAPL  2022-09-09  LMFAO.  Government buying everything not naile...   
2547   AAPL  2022-09-02              Google green. How apple still down 1%   
2548   AAPL  2022-08-26          Thats one beatiful cup and handle forming   
2549   AAPL  2022-08-20                                time to buy the dip   
...     ...         ...                                                ...   
26136   NIO  2022-07-15  #TSLA Held 20 Day SMA today and closed above 7...   
26137  TSLA  2022-02-24                     750 as I wrote 10 hours ago...   
26138  TSLA  2022-11-10                           Crypto will push it down   
26139  TSLA  2022-03-04  This really doesnt want to move like the other...   
26140  TSLA  2022-04-25                                  Just can't sleep.   

       category  label  
2545          1      1  
2546          1      1  
2547          1     -1  
2548          1     -1  
2549          1      1  
...         ...    ...  
26136         1      1  
26137         1      1  
26138         1     -1  
26139         1      0  
26140         1      0  

[23094 rows x 5 columns]

In [7]:
df_opinion

stock        Date                                               text  \
0      BABA  2023-01-31  China's reopening has revived its economy but ...   
1      BABA  2023-01-26  Investors seem to be positioning for a strong ...   
2      BABA  2023-01-17  E-commerce behemoth Alibaba Group Holdings (NY...   
3      BABA  2023-01-13  Alibaba shares are up 100% from their lows. Ta...   
4      BABA  2023-01-13  The continued easing of US price pressures has...   
...     ...         ...                                                ...   
13688   AMD  2022-01-04  Carrier Global (NYSE:CARR) Corp. CARR has comp...   
13689  META  2022-02-01  Stocks extended their rally yesterday, as the ...   
13690  META  2022-09-10  What__ in a name? If you are interested in the...   
13691    ZM  2022-02-02  The lumbering giant known as the US Federal Re...   
13692  MSFT  2022-10-31  When it comes to the stock market, October is ...   

       category  label  
0             2      1  
1             2      1  
2             2      1  
3             2      1  
4             2      0  
...         ...    ...  
13688         2      0  
13689         2      1  
13690         2      0  
13691         2      0  
13692         2     -1  

[3045 rows x 5 columns]

## Weighted Sentiment Index (with penalty_alpha =0.5)

In [8]:
def calculate_weighted_sentiment_index(df, penalty_alpha=0.5):
    weighted_sentiment_indices = (
        df.groupby(["stock", "Date"])["label"]
        .agg(["sum", "count"])
        .reset_index()
    )
    weighted_sentiment_indices["penalty"] = penalty_alpha / weighted_sentiment_indices["count"]
    weighted_sentiment_indices["weighted_sentiment_index"] = (
        weighted_sentiment_indices["sum"] / weighted_sentiment_indices["count"]
    ) * (1 - weighted_sentiment_indices["penalty"])

    return weighted_sentiment_indices[["stock", "Date", "weighted_sentiment_index", "count"]]


In [9]:
weighted_comment_indices = calculate_weighted_sentiment_index(df_comment)
print(weighted_comment_indices)

     stock        Date  weighted_sentiment_index  count
0     AAPL  2022-01-01                  0.000000      4
1     AAPL  2022-01-02                  0.750000      2
2     AAPL  2022-01-04                 -0.152778      6
3     AAPL  2022-01-05                  0.132653      7
4     AAPL  2022-01-06                 -0.375000      2
...    ...         ...                       ...    ...
2758    ZM  2022-11-12                 -0.500000      1
2759    ZM  2022-11-23                 -0.500000      1
2760    ZM  2022-12-28                  0.500000      1
2761    ZM  2023-01-18                  0.500000      1
2762    ZM  2023-02-17                 -0.500000      1

[2763 rows x 4 columns]


In [10]:
weighted_opinion_indices = calculate_weighted_sentiment_index(df_opinion)
print(weighted_opinion_indices)

     stock        Date  weighted_sentiment_index  count
0     AAPL  2022-01-01                  0.000000      1
1     AAPL  2022-01-02                  0.000000      1
2     AAPL  2022-01-03                  0.524691      9
3     AAPL  2022-01-04                  0.360000      5
4     AAPL  2022-01-05                  0.375000      2
...    ...         ...                       ...    ...
1754    ZM  2022-11-22                 -0.375000      2
1755    ZM  2022-11-28                  0.500000      1
1756    ZM  2022-11-29                 -0.750000      2
1757    ZM  2022-12-19                  0.000000      1
1758    ZM  2023-02-10                  0.500000      1

[1759 rows x 4 columns]


## Reform the dataframe

In [11]:
def reformat_sentiment_indices(weighted_sentiment_indices):
    sentiment_time_series = weighted_sentiment_indices.pivot(
        index="Date", columns="stock", values="weighted_sentiment_index"
    )
    count_time_series = weighted_sentiment_indices.pivot(
        index="Date", columns="stock", values="count"
    )
    count_time_series = count_time_series.add_suffix("_count")
    sentiment_time_series = pd.concat([sentiment_time_series, count_time_series], axis=1)
    
    # Reset the index to move the "date" index into a regular column
    sentiment_time_series = sentiment_time_series.reset_index()
    
    return sentiment_time_series


In [12]:
comment_time_series = reformat_sentiment_indices(weighted_comment_indices)
comment_time_series

stock        Date      AAPL  AMC      AMD      AMZN  AVGO  BABA    BB  CLOV  \
0      2021-12-31       NaN  NaN      NaN       NaN   NaN   NaN   NaN   NaN   
1      2022-01-01  0.000000  NaN -0.21875 -0.209877   NaN  -0.5   NaN   NaN   
2      2022-01-02  0.750000  NaN      NaN       NaN   NaN  -0.5   NaN   NaN   
3      2022-01-03       NaN  NaN      NaN       NaN   NaN   NaN   NaN   NaN   
4      2022-01-04 -0.152778  NaN  0.00000  0.500000   NaN   0.5  0.75   0.0   
..            ...       ...  ...      ...       ...   ...   ...   ...   ...   
415    2023-02-20       NaN  NaN  0.00000       NaN   NaN   NaN   NaN   NaN   
416    2023-02-21       NaN  NaN      NaN       NaN   NaN   NaN   NaN   NaN   
417    2023-02-22       NaN  NaN  0.00000       NaN   NaN   NaN   NaN   NaN   
418    2023-02-23       NaN  NaN      NaN       NaN   NaN   NaN   NaN   NaN   
419    2023-02-24       NaN  NaN      NaN       NaN   NaN   NaN   NaN   NaN   

stock  GOOG  ...  PL_count  PLTR_count  RIVN_count  SLA_count  STM_count  \
0       NaN  ...       NaN         NaN         NaN        NaN        NaN   
1       NaN  ...       NaN         NaN         NaN        NaN        NaN   
2       NaN  ...       NaN         NaN         NaN        NaN        NaN   
3       NaN  ...       NaN         1.0         NaN        NaN        NaN   
4       NaN  ...       NaN         NaN         1.0        NaN        NaN   
..      ...  ...       ...         ...         ...        ...        ...   
415     NaN  ...       NaN         NaN         NaN        NaN        NaN   
416     NaN  ...       NaN         NaN         NaN        NaN        NaN   
417   -0.75  ...       NaN         NaN         NaN        NaN        NaN   
418     NaN  ...       NaN         NaN         1.0        NaN        NaN   
419    0.50  ...       NaN         NaN         NaN        NaN        NaN   

stock  TSLA_count  TSM_count  TXN_count  WISH_count  ZM_count  
0             NaN        NaN        NaN         NaN       NaN  
1            40.0        NaN        NaN         NaN       NaN  
2             6.0        NaN        NaN         NaN       NaN  
3            59.0        NaN        NaN         NaN       NaN  
4           280.0        1.0        NaN         1.0       NaN  
..            ...        ...        ...         ...       ...  
415           NaN        NaN        NaN         NaN       NaN  
416           NaN        NaN        NaN         NaN       NaN  
417           NaN        NaN        NaN         NaN       NaN  
418           NaN        NaN        NaN         NaN       NaN  
419           NaN        NaN        NaN         NaN       NaN  

[420 rows x 49 columns]

In [13]:
opinion_time_series = reformat_sentiment_indices(weighted_opinion_indices)
opinion_time_series

stock        Date      AAPL  AMC  AMD      AMZN  AVGO  BABA  BB      GOOG  \
0      2022-01-01  0.000000  NaN  NaN  0.277778  -0.5   NaN NaN  0.375000   
1      2022-01-02  0.000000  0.0  NaN  0.000000   NaN   0.0 NaN       NaN   
2      2022-01-03  0.524691  0.5  0.0  0.458333   0.0   0.5 NaN  0.555556   
3      2022-01-04  0.360000  NaN  0.0  0.277778   NaN   NaN NaN  0.555556   
4      2022-01-05  0.375000  NaN  NaN  0.500000   0.5   NaN NaN  0.750000   
..            ...       ...  ...  ...       ...   ...   ...  ..       ...   
358    2023-02-15       NaN  NaN  NaN       NaN   NaN   NaN NaN       NaN   
359    2023-02-16  0.500000  NaN  NaN       NaN   NaN   NaN NaN  0.000000   
360    2023-02-19  0.500000  NaN  NaN       NaN   NaN   NaN NaN       NaN   
361    2023-02-20       NaN  NaN  NaN       NaN   NaN   NaN NaN  0.000000   
362    2023-02-21  0.000000  NaN  NaN -0.375000   NaN   NaN NaN  0.500000   

stock  LCID  ...  MSFT_count  NIO_count  PLTR_count  RIVN_count  STM_count  \
0       NaN  ...         2.0        NaN         NaN         NaN        NaN   
1       NaN  ...         1.0        NaN         NaN         NaN        NaN   
2       NaN  ...         7.0        NaN         NaN         NaN        NaN   
3       NaN  ...         2.0        NaN         NaN         NaN        NaN   
4       NaN  ...         1.0        NaN         NaN         NaN        NaN   
..      ...  ...         ...        ...         ...         ...        ...   
358     NaN  ...         1.0        NaN         NaN         NaN        NaN   
359     NaN  ...         2.0        NaN         NaN         NaN        NaN   
360     NaN  ...         NaN        NaN         NaN         NaN        NaN   
361     NaN  ...         1.0        NaN         NaN         NaN        NaN   
362     NaN  ...         1.0        NaN         NaN         NaN        NaN   

stock  TSLA_count  TSM_count  TXN_count  WISH_count  ZM_count  
0             1.0        NaN        NaN         NaN       NaN  
1             1.0        NaN        NaN         NaN       NaN  
2             9.0        NaN        NaN         NaN       NaN  
3             7.0        NaN        1.0         NaN       NaN  
4             5.0        NaN        NaN         NaN       NaN  
..            ...        ...        ...         ...       ...  
358           1.0        NaN        NaN         NaN       NaN  
359           2.0        NaN        NaN         NaN       NaN  
360           NaN        NaN        NaN         NaN       NaN  
361           NaN        NaN        NaN         NaN       NaN  
362           NaN        NaN        NaN         NaN       NaN  

[363 rows x 41 columns]

In [14]:
# List of stocks to keep
columns_to_keep = ['Date','TSLA', 'LCID', 'META', 'AMZN', 'MSFT', 'AAPL', 'AMC', 'AMD', 'BABA', 'NIO',
                   'TSLA_count', 'LCID_count', 'META_count', 'AMZN_count', 'MSFT_count', 'AAPL_count', 'AMC_count', 'AMD_count', 'BABA_count', 'NIO_count']

# Filter the DataFrame to keep only the stocks in the list
comments = comment_time_series[columns_to_keep]
opinions = opinion_time_series[columns_to_keep]

# Reset the index of the filtered DataFrame (optional)
#comments.reset_index(drop=True, inplace=True)

# Show the filtered DataFrame
print(comments)
print(opinions)

stock        Date      TSLA  LCID  META      AMZN      MSFT      AAPL  AMC  \
0      2021-12-31       NaN   NaN   NaN       NaN       NaN       NaN  NaN   
1      2022-01-01  0.246875   NaN   0.0 -0.209877  0.277778  0.000000  NaN   
2      2022-01-02 -0.152778   NaN  -0.5       NaN       NaN  0.750000  NaN   
3      2022-01-03  0.252083   NaN   NaN       NaN       NaN       NaN  NaN   
4      2022-01-04  0.228163   0.0   0.0  0.500000  0.000000 -0.152778  NaN   
..            ...       ...   ...   ...       ...       ...       ...  ...   
415    2023-02-20       NaN   NaN   NaN       NaN       NaN       NaN  NaN   
416    2023-02-21       NaN   NaN  -0.5       NaN       NaN       NaN  NaN   
417    2023-02-22       NaN   0.0   NaN       NaN       NaN       NaN  NaN   
418    2023-02-23       NaN   NaN   NaN       NaN       NaN       NaN  NaN   
419    2023-02-24       NaN   NaN   NaN       NaN       NaN       NaN  NaN   

stock      AMD  BABA  ...  TSLA_count  LCID_count  META_count  

In [15]:
null_counts = comments.isnull().sum()
print(null_counts)

stock
Date            0
TSLA           19
LCID          322
META          193
AMZN           96
MSFT          305
AAPL          166
AMC           307
AMD           175
BABA          207
NIO            59
TSLA_count     19
LCID_count    322
META_count    193
AMZN_count     96
MSFT_count    305
AAPL_count    166
AMC_count     307
AMD_count     175
BABA_count    207
NIO_count      59
dtype: int64


In [16]:
null_counts = opinions.isnull().sum()
print(null_counts)

stock
Date            0
TSLA          125
LCID          351
META          195
AMZN           95
MSFT          139
AAPL          118
AMC           331
AMD           279
BABA          294
NIO           342
TSLA_count    125
LCID_count    351
META_count    195
AMZN_count     95
MSFT_count    139
AAPL_count    118
AMC_count     331
AMD_count     279
BABA_count    294
NIO_count     342
dtype: int64


In [17]:
# # drop rows corresponding to weekends
# comments.index = pd.to_datetime(comments.index)

# # # create a list of dates to drop
# # dates_to_drop = ["2022-01-02", "2023-01-02", "2022-01-16", "2023-01-16", 
# #                  "2022-02-20", "2023-02-20", "2022-04-15", "2022-05-29", 
# #                  "2022-07-04", "2022-09-05", "2022-11-24", "2022-12-25"]

# # convert the dates to datetime format
# #dates_to_drop = pd.to_datetime(dates_to_drop)

# # drop rows corresponding to weekends and specific dates
# comments = comments[(comments.index.weekday < 5)]

In [18]:
# Define the specific column name to exclude from renaming
exclude_col = 'Date'

# Loop through each column name in the DataFrame, and modify the name as desired, while excluding the specific column
comments.columns = [f"comments_{col}" if col != exclude_col else col for col in comments.columns]
comments['Date'] = pd.to_datetime(comments['Date'])

# Loop through each column name in the DataFrame, and modify the name as desired, while excluding the specific column
opinions.columns = [f"opinions_{col}" if col != exclude_col else col for col in opinions.columns]
opinions['Date'] = pd.to_datetime(opinions['Date'])

<ipython-input-18-37f8f4bd9f58>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['Date'] = pd.to_datetime(comments['Date'])
<ipython-input-18-37f8f4bd9f58>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions['Date'] = pd.to_datetime(opinions['Date'])


In [19]:
comments

Date  comments_TSLA  comments_LCID  comments_META  comments_AMZN  \
0   2021-12-31            NaN            NaN            NaN            NaN   
1   2022-01-01       0.246875            NaN            0.0      -0.209877   
2   2022-01-02      -0.152778            NaN           -0.5            NaN   
3   2022-01-03       0.252083            NaN            NaN            NaN   
4   2022-01-04       0.228163            0.0            0.0       0.500000   
..         ...            ...            ...            ...            ...   
415 2023-02-20            NaN            NaN            NaN            NaN   
416 2023-02-21            NaN            NaN           -0.5            NaN   
417 2023-02-22            NaN            0.0            NaN            NaN   
418 2023-02-23            NaN            NaN            NaN            NaN   
419 2023-02-24            NaN            NaN            NaN            NaN   

     comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  comments_BABA  \
0              NaN            NaN           NaN           NaN            NaN   
1         0.277778       0.000000           NaN      -0.21875           -0.5   
2              NaN       0.750000           NaN           NaN           -0.5   
3              NaN            NaN           NaN           NaN            NaN   
4         0.000000      -0.152778           NaN       0.00000            0.5   
..             ...            ...           ...           ...            ...   
415            NaN            NaN           NaN       0.00000            NaN   
416            NaN            NaN           NaN           NaN            NaN   
417            NaN            NaN           NaN       0.00000            NaN   
418            NaN            NaN           NaN           NaN            NaN   
419            NaN            NaN           NaN           NaN            NaN   

     ...  comments_TSLA_count  comments_LCID_count  comments_META_count  \
0    ...                  NaN                  NaN                  NaN   
1    ...                 40.0                  NaN                  2.0   
2    ...                  6.0                  NaN                  1.0   
3    ...                 59.0                  NaN                  NaN   
4    ...                280.0                  6.0                  1.0   
..   ...                  ...                  ...                  ...   
415  ...                  NaN                  NaN                  NaN   
416  ...                  NaN                  NaN                  1.0   
417  ...                  NaN                  2.0                  NaN   
418  ...                  NaN                  NaN                  NaN   
419  ...                  NaN                  NaN                  NaN   

     comments_AMZN_count  comments_MSFT_count  comments_AAPL_count  \
0                    NaN                  NaN                  NaN   
1                    9.0                  3.0                  4.0   
2                    NaN                  NaN                  2.0   
3                    NaN                  NaN                  NaN   
4                    1.0                  1.0                  6.0   
..                   ...                  ...                  ...   
415                  NaN                  NaN                  NaN   
416                  NaN                  NaN                  NaN   
417                  NaN                  NaN                  NaN   
418                  NaN                  NaN                  NaN   
419                  NaN                  NaN                  NaN   

     comments_AMC_count  comments_AMD_count  comments_BABA_count  \
0                   NaN                 NaN                  NaN   
1                   NaN                 4.0                  1.0   
2                   NaN                 NaN                  1.0   
3                   NaN                 NaN                  NaN   
4                   NaN       

# Add market index & stock data

In [20]:
#index
YrBond10=pd.read_csv('/content/drive/MyDrive/5153/index/10YrBond.csv')
MonBond3=pd.read_csv('/content/drive/MyDrive/5153/index/13WBond.csv')
dow30=pd.read_csv('/content/drive/MyDrive/5153/index/dow30.csv')
nasdaq=pd.read_csv('/content/drive/MyDrive/5153/index/nasdaq.csv')
sp500=pd.read_csv('/content/drive/MyDrive/5153/index/sp500.csv')

In [21]:
sp500

Date     Open     High      Low   Close*  Adj Close**      Volume
0    2021-12-31  4775.21  4786.83  4765.75  4766.18      4766.18  2677820000
1    2022-01-03  4778.14  4796.64  4758.17  4796.56      4796.56  3831020000
2    2022-01-04  4804.51  4818.62  4774.27  4793.54      4793.54  4683170000
3    2022-01-05  4787.99  4797.70  4699.44  4700.58      4700.58  4887960000
4    2022-01-06  4693.39  4725.01  4671.26  4696.05      4696.05  4295280000
..          ...      ...      ...      ...      ...          ...         ...
284  2023-02-17  4077.39  4081.51  4047.95  4079.09      4079.09  4045480000
285  2023-02-21  4052.35  4052.35  3995.19  3997.34      3997.34  4121590000
286  2023-02-22  4001.83  4017.37  3976.90  3991.05      3991.05  4079320000
287  2023-02-23  4018.60  4028.30  3969.19  4012.32      4012.32  3952940000
288  2023-02-24  3973.24  3978.25  3943.08  3970.04      3970.04  3877700000

[289 rows x 7 columns]

In [22]:
YrBond10['percentage']= (YrBond10['Close'] - YrBond10['Close'].shift(1))/YrBond10['Close'].shift(1)
MonBond3['percentage']= (MonBond3['Close'] - MonBond3['Close'].shift(1))/MonBond3['Close'].shift(1)
dow30['percentage']= (dow30['Close*'] - dow30['Close*'].shift(1))/dow30['Close*'].shift(1)
nasdaq['percentage']= (nasdaq['Close'] - nasdaq['Close'].shift(1))/nasdaq['Close'].shift(1)
sp500['percentage']= (sp500['Close*'] - sp500['Close*'].shift(1))/sp500['Close*'].shift(1)

In [23]:
#stock
aapl=pd.read_csv('/content/drive/MyDrive/5153/stock/AAPL.csv')
amc=pd.read_csv('/content/drive/MyDrive/5153/stock/AMC.csv')
amd=pd.read_csv('/content/drive/MyDrive/5153/stock/AMD.csv')
amzn=pd.read_csv('/content/drive/MyDrive/5153/stock/AMZN.csv')
baba=pd.read_csv('/content/drive/MyDrive/5153/stock/BABA.csv')
lcid=pd.read_csv('/content/drive/MyDrive/5153/stock/LCID.csv')
meta=pd.read_csv('/content/drive/MyDrive/5153/stock/META.csv')
msft=pd.read_csv('/content/drive/MyDrive/5153/stock/MSFT.csv')
nio=pd.read_csv('/content/drive/MyDrive/5153/stock/NIO.csv')
tsla=pd.read_csv('/content/drive/MyDrive/5153/stock/TSLA.csv')

In [24]:
aapl['percentage']= (aapl['Close'] - aapl['Close'].shift(1))/aapl['Close'].shift(1)
amc['percentage']= (amc['Close'] - amc['Close'].shift(1))/amc['Close'].shift(1)
amd['percentage']= (amd['Close'] - amd['Close'].shift(1))/amd['Close'].shift(1)
amzn['percentage']= (amzn['Close'] - amzn['Close'].shift(1))/amzn['Close'].shift(1)
baba['percentage']= (baba['Close'] - baba['Close'].shift(1))/baba['Close'].shift(1)
lcid['percentage']= (lcid['Close'] - lcid['Close'].shift(1))/lcid['Close'].shift(1)
meta['percentage']= (meta['Close'] - meta['Close'].shift(1))/meta['Close'].shift(1)
msft['percentage']= (msft['Close'] - msft['Close'].shift(1))/msft['Close'].shift(1)
nio['percentage']= (nio['Close'] - nio['Close'].shift(1))/nio['Close'].shift(1)
tsla['percentage']= (tsla['Close'] - tsla['Close'].shift(1))/tsla['Close'].shift(1)

In [25]:
#rename each colomn

# Define the specific column name to exclude from renaming
exclude_col = 'Date'

# Loop through each column name in the DataFrame, and modify the name as desired, while excluding the specific column
YrBond10.columns = [f"YrBond10_{col}" if col != exclude_col else col for col in YrBond10.columns]
YrBond10['Date'] = pd.to_datetime(YrBond10['Date'])

MonBond3.columns = [f"MonBond3_{col}" if col != exclude_col else col for col in MonBond3.columns]
MonBond3['Date'] = pd.to_datetime(MonBond3['Date'])

dow30.columns = [f"dow30_{col}" if col != exclude_col else col for col in dow30.columns]
dow30['Date'] = pd.to_datetime(dow30['Date'])

nasdaq.columns = [f"nasdaq_{col}" if col != exclude_col else col for col in nasdaq.columns]
nasdaq['Date'] = pd.to_datetime(nasdaq['Date'])

sp500.columns = [f"sp500_{col}" if col != exclude_col else col for col in sp500.columns]
sp500['Date'] = pd.to_datetime(sp500['Date'])

In [26]:
MonBond3

Date  MonBond3_Open  MonBond3_High  MonBond3_Low  MonBond3_Close  \
0   2021-12-31          0.028          0.033         0.025           0.033   
1   2022-01-02            NaN            NaN           NaN             NaN   
2   2022-01-03          0.050          0.058         0.050           0.053   
3   2022-01-04          0.085          0.085         0.078           0.080   
4   2022-01-05          0.080          0.093         0.080           0.085   
..         ...            ...            ...           ...             ...   
351 2023-02-19            NaN            NaN           NaN             NaN   
352 2023-02-21          4.680          4.695         4.680           4.688   
353 2023-02-22          4.705          4.708         4.690           4.695   
354 2023-02-23          4.690          4.698         4.680           4.690   
355 2023-02-24          4.833          4.851         4.867           4.810   

     MonBond3_Adj Close  MonBond3_Volume  MonBond3_percentage  
0                 0.033              0.0                  NaN  
1                   NaN              NaN                  NaN  
2                 0.053              0.0                  NaN  
3                 0.080              0.0             0.509434  
4                 0.085              0.0             0.062500  
..                  ...              ...                  ...  
351                 NaN              NaN                  NaN  
352               4.688              0.0                  NaN  
353               4.695              0.0             0.001493  
354               4.690              0.0            -0.001065  
355               4.810              0.0             0.025586  

[356 rows x 8 columns]

In [27]:
nasdaq

Date  nasdaq_Open  nasdaq_High  nasdaq_Low  nasdaq_Close  \
0   2021-12-31     15722.91     15777.43    15643.94      15644.97   
1   2022-01-03     15732.50     15832.80    15644.09      15832.80   
2   2022-01-04     15852.14     15852.14    15512.41      15622.72   
3   2022-01-05     15547.16     15586.30    15095.18      15100.17   
4   2022-01-06     15024.15     15198.45    14914.87      15080.86   
..         ...          ...          ...         ...           ...   
284 2023-02-17     11777.50     11803.22    11673.21      11787.27   
285 2023-02-21     11640.37     11684.14    11491.18      11492.30   
286 2023-02-22     11517.20     11582.52    11445.17      11507.07   
287 2023-02-23     11636.93     11638.98    11432.58      11590.40   
288 2023-02-24     11404.18     11434.36    11334.47      11394.94   

     nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  
0            15644.97   3.392580e+09                NaN  
1            15832.80   4.429960e+09           0.012006  
2            15622.72   5.131110e+09          -0.013269  
3            15100.17   5.031850e+09          -0.033448  
4            15080.86   4.790820e+09          -0.001279  
..                ...            ...                ...  
284          11787.27   4.925230e+09          -0.005783  
285          11492.30   5.002540e+09          -0.025024  
286          11507.07   4.671270e+09           0.001285  
287          11590.40   4.714580e+09           0.007242  
288          11394.94   4.432650e+09          -0.016864  

[289 rows x 8 columns]

In [28]:
# stocks=[aapl, amc, amd, amzn, baba, lcid, meta, msft, nio, tsla]
# for stock in stocks:
#   # Convert the date column to a datetime format
#   stock['Date'] = pd.to_datetime(stock['Date'])
#   stock.sort_values('Date', ascending=True, inplace=True)
#   stock.reset_index(drop=True, inplace=True)
#   date_obj = pd.to_datetime(stock['Date'], format="%Y-%m-%d")
#   stock['Date'] = stock['Date'].dt.strftime("%d/%m/%Y")

In [29]:
aapl.columns = [f"aapl_{col}" if col != exclude_col else col for col in aapl.columns]
amc.columns = [f"amc_{col}" if col != exclude_col else col for col in amc.columns]
amd.columns = [f"amd_{col}" if col != exclude_col else col for col in amd.columns]
amzn.columns = [f"amzn_{col}" if col != exclude_col else col for col in amzn.columns]
baba.columns = [f"baba_{col}" if col != exclude_col else col for col in baba.columns]
lcid.columns = [f"lcid_{col}" if col != exclude_col else col for col in lcid.columns]
meta.columns = [f"meta_{col}" if col != exclude_col else col for col in meta.columns]
msft.columns = [f"msft_{col}" if col != exclude_col else col for col in msft.columns]
nio.columns = [f"nio_{col}" if col != exclude_col else col for col in nio.columns]
tsla.columns = [f"tsla_{col}" if col != exclude_col else col for col in tsla.columns]

In [30]:
dates_to_keep=tsla['Date']
dates_to_keep

0      2021-12-31
1      2022-01-03
2      2022-01-04
3      2022-01-05
4      2022-01-06
          ...    
284    2023-02-17
285    2023-02-21
286    2023-02-22
287    2023-02-23
288    2023-02-24
Name: Date, Length: 289, dtype: object

In [31]:
# # Convert the "date" column to a date object
# date_obj = pd.to_datetime(comments['Date'], format="%Y-%m-%d")

# # Format the date object as a string in the desired format
# comments['Date'] = date_obj.dt.strftime("%d/%m/%Y")

In [32]:
#  # Assume YrBond10, MonBond3, dow30, nasdaq, sp500, comments, aapl, amc, amd, amzn, baba, lcid, meta, msft, nio, and tsla are DataFrames with a "Date" column
# dfs = [comments,YrBond10, MonBond3, dow30, nasdaq, sp500]

# # Concatenate the DataFrames horizontally based on the "Date" column
# df = pd.concat(dfs, axis=1, join='outer')

# # Reset the index to make "Date" a regular column again
# df = df.reset_index()

# # Drop the duplicate "Date" columns
# df = df.loc[:,~df.columns.duplicated()]


In [33]:
comments

Date  comments_TSLA  comments_LCID  comments_META  comments_AMZN  \
0   2021-12-31            NaN            NaN            NaN            NaN   
1   2022-01-01       0.246875            NaN            0.0      -0.209877   
2   2022-01-02      -0.152778            NaN           -0.5            NaN   
3   2022-01-03       0.252083            NaN            NaN            NaN   
4   2022-01-04       0.228163            0.0            0.0       0.500000   
..         ...            ...            ...            ...            ...   
415 2023-02-20            NaN            NaN            NaN            NaN   
416 2023-02-21            NaN            NaN           -0.5            NaN   
417 2023-02-22            NaN            0.0            NaN            NaN   
418 2023-02-23            NaN            NaN            NaN            NaN   
419 2023-02-24            NaN            NaN            NaN            NaN   

     comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  comments_BABA  \
0              NaN            NaN           NaN           NaN            NaN   
1         0.277778       0.000000           NaN      -0.21875           -0.5   
2              NaN       0.750000           NaN           NaN           -0.5   
3              NaN            NaN           NaN           NaN            NaN   
4         0.000000      -0.152778           NaN       0.00000            0.5   
..             ...            ...           ...           ...            ...   
415            NaN            NaN           NaN       0.00000            NaN   
416            NaN            NaN           NaN           NaN            NaN   
417            NaN            NaN           NaN       0.00000            NaN   
418            NaN            NaN           NaN           NaN            NaN   
419            NaN            NaN           NaN           NaN            NaN   

     ...  comments_TSLA_count  comments_LCID_count  comments_META_count  \
0    ...                  NaN                  NaN                  NaN   
1    ...                 40.0                  NaN                  2.0   
2    ...                  6.0                  NaN                  1.0   
3    ...                 59.0                  NaN                  NaN   
4    ...                280.0                  6.0                  1.0   
..   ...                  ...                  ...                  ...   
415  ...                  NaN                  NaN                  NaN   
416  ...                  NaN                  NaN                  1.0   
417  ...                  NaN                  2.0                  NaN   
418  ...                  NaN                  NaN                  NaN   
419  ...                  NaN                  NaN                  NaN   

     comments_AMZN_count  comments_MSFT_count  comments_AAPL_count  \
0                    NaN                  NaN                  NaN   
1                    9.0                  3.0                  4.0   
2                    NaN                  NaN                  2.0   
3                    NaN                  NaN                  NaN   
4                    1.0                  1.0                  6.0   
..                   ...                  ...                  ...   
415                  NaN                  NaN                  NaN   
416                  NaN                  NaN                  NaN   
417                  NaN                  NaN                  NaN   
418                  NaN                  NaN                  NaN   
419                  NaN                  NaN                  NaN   

     comments_AMC_count  comments_AMD_count  comments_BABA_count  \
0                   NaN                 NaN                  NaN   
1                   NaN                 4.0                  1.0   
2                   NaN                 NaN                  1.0   
3                   NaN                 NaN                  NaN   
4                   NaN       

In [34]:
comments['Date'] = pd.to_datetime(comments['Date'])
comments.dtypes

<ipython-input-34-e7fbf2f69cd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['Date'] = pd.to_datetime(comments['Date'])


Date                   datetime64[ns]
comments_TSLA                 float64
comments_LCID                 float64
comments_META                 float64
comments_AMZN                 float64
comments_MSFT                 float64
comments_AAPL                 float64
comments_AMC                  float64
comments_AMD                  float64
comments_BABA                 float64
comments_NIO                  float64
comments_TSLA_count           float64
comments_LCID_count           float64
comments_META_count           float64
comments_AMZN_count           float64
comments_MSFT_count           float64
comments_AAPL_count           float64
comments_AMC_count            float64
comments_AMD_count            float64
comments_BABA_count           float64
comments_NIO_count            float64
dtype: object

In [35]:
opinions['Date'] = pd.to_datetime(opinions['Date'])
opinions.dtypes

<ipython-input-35-b701ba31e621>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opinions['Date'] = pd.to_datetime(opinions['Date'])


Date                   datetime64[ns]
opinions_TSLA                 float64
opinions_LCID                 float64
opinions_META                 float64
opinions_AMZN                 float64
opinions_MSFT                 float64
opinions_AAPL                 float64
opinions_AMC                  float64
opinions_AMD                  float64
opinions_BABA                 float64
opinions_NIO                  float64
opinions_TSLA_count           float64
opinions_LCID_count           float64
opinions_META_count           float64
opinions_AMZN_count           float64
opinions_MSFT_count           float64
opinions_AAPL_count           float64
opinions_AMC_count            float64
opinions_AMD_count            float64
opinions_BABA_count           float64
opinions_NIO_count            float64
dtype: object

In [36]:
YrBond10.dtypes

Date                   datetime64[ns]
YrBond10_Open                 float64
YrBond10_High                 float64
YrBond10_Low                  float64
YrBond10_Close                float64
YrBond10_Adj Close            float64
YrBond10_Volume               float64
YrBond10_percentage           float64
dtype: object

In [37]:
# Merge the DataFrames based on the "date" column
merged_df = pd.merge(comments, opinions, on='Date', how='outer')
merged_df = pd.merge(merged_df, YrBond10, on='Date', how='outer')
merged_df = pd.merge(merged_df, MonBond3, on='Date', how='outer')
merged_df = pd.merge(merged_df, dow30, on='Date', how='outer')
merged_df = pd.merge(merged_df, nasdaq, on='Date', how='outer')
merged_df = pd.merge(merged_df, sp500, on='Date', how='outer')
# Print the resulting DataFrame
print(merged_df)


          Date  comments_TSLA  comments_LCID  comments_META  comments_AMZN  \
0   2021-12-31            NaN            NaN            NaN            NaN   
1   2022-01-01       0.246875            NaN            0.0      -0.209877   
2   2022-01-02      -0.152778            NaN           -0.5            NaN   
3   2022-01-03       0.252083            NaN            NaN            NaN   
4   2022-01-04       0.228163            0.0            0.0       0.500000   
..         ...            ...            ...            ...            ...   
416 2023-02-21            NaN            NaN           -0.5            NaN   
417 2023-02-22            NaN            0.0            NaN            NaN   
418 2023-02-23            NaN            NaN            NaN            NaN   
419 2023-02-24            NaN            NaN            NaN            NaN   
420 2023-02-19            NaN            NaN            NaN            NaN   

     comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  

In [38]:
# Create a boolean mask for the rows with dates in "dates_to_keep"
mask = merged_df['Date'].isin(dates_to_keep)

# Filter the DataFrame using the mask
factors = merged_df[mask]

# Print the resulting DataFrame
print(factors)

          Date  comments_TSLA  comments_LCID  comments_META  comments_AMZN  \
0   2021-12-31            NaN            NaN            NaN            NaN   
3   2022-01-03       0.252083            NaN            NaN            NaN   
4   2022-01-04       0.228163           0.00          0.000       0.500000   
5   2022-01-05       0.168773           0.00         -0.375      -0.277778   
6   2022-01-06       0.160335           0.36            NaN            NaN   
..         ...            ...            ...            ...            ...   
413 2023-02-17            NaN            NaN            NaN            NaN   
416 2023-02-21            NaN            NaN         -0.500            NaN   
417 2023-02-22            NaN           0.00            NaN            NaN   
418 2023-02-23            NaN            NaN            NaN            NaN   
419 2023-02-24            NaN            NaN            NaN            NaN   

     comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  

In [39]:
# # Alternatively, you can filter the DataFrame to keep only rows with dates in the list
# merged_df.loc[merged_df['Date'].isin(dates_to_keep)]
# merged_df.reset_index(drop=True, inplace=True)

In [40]:
factors = factors.reset_index()
factors

index       Date  comments_TSLA  comments_LCID  comments_META  \
0        0 2021-12-31            NaN            NaN            NaN   
1        3 2022-01-03       0.252083            NaN            NaN   
2        4 2022-01-04       0.228163           0.00          0.000   
3        5 2022-01-05       0.168773           0.00         -0.375   
4        6 2022-01-06       0.160335           0.36            NaN   
..     ...        ...            ...            ...            ...   
284    413 2023-02-17            NaN            NaN            NaN   
285    416 2023-02-21            NaN            NaN         -0.500   
286    417 2023-02-22            NaN           0.00            NaN   
287    418 2023-02-23            NaN            NaN            NaN   
288    419 2023-02-24            NaN            NaN            NaN   

     comments_AMZN  comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  \
0              NaN            NaN            NaN           NaN           NaN   
1              NaN            NaN            NaN           NaN           NaN   
2         0.500000          0.000      -0.152778           NaN      0.000000   
3        -0.277778          0.375       0.132653           NaN      0.375000   
4              NaN            NaN      -0.375000           NaN      0.000000   
..             ...            ...            ...           ...           ...   
284            NaN            NaN            NaN           NaN     -0.277778   
285            NaN            NaN            NaN           NaN           NaN   
286            NaN            NaN            NaN           NaN      0.000000   
287            NaN            NaN            NaN           NaN           NaN   
288            NaN            NaN            NaN           NaN           NaN   

     ...  nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  sp500_Open  \
0    ...          15644.97   3.392580e+09                NaN     4775.21   
1    ...          15832.80   4.429960e+09           0.012006     4778.14   
2    ...          15622.72   5.131110e+09          -0.013269     4804.51   
3    ...          15100.17   5.031850e+09          -0.033448     4787.99   
4    ...          15080.86   4.790820e+09          -0.001279     4693.39   
..   ...               ...            ...                ...         ...   
284  ...          11787.27   4.925230e+09          -0.005783     4077.39   
285  ...          11492.30   5.002540e+09          -0.025024     4052.35   
286  ...          11507.07   4.671270e+09           0.001285     4001.83   
287  ...          11590.40   4.714580e+09           0.007242     4018.60   
288  ...          11394.94   4.432650e+09          -0.016864     3973.24   

     sp500_High  sp500_Low  sp500_Close*  sp500_Adj Close**  sp500_Volume  \
0       4786.83    4765.75       4766.18            4766.18  2.677820e+09   
1       4796.64    4758.17       4796.56            4796.56  3.831020e+09   
2       4818.62    4774.27       4793.54            4793.54  4.683170e+09   
3       4797.70    4699.44       4700.58            4700.58  4.887960e+09   
4       4725.01    4671.26       4696.05            4696.05  4.295280e+09   
..          ...        ...           ...                ...           ...   
284     4081.51    4047.95       4079.09            4079.09  4.045480e+09   
285     4052.35    3995.19       3997.34            3997.34  4.121590e+09   
286     4017.37    3976.90       3991.05            3991.05  4.079320e+09   
287     4028.30    3969.19       4012.32            4012.32  3.952940e+09   
288     3978.25    3943.08       3970.04            3970.04  3.877700e+09   

     sp500_percentage  
0                 NaN  
1            0.006374  
2           -0.000630  
3           -0.019393  
4           -0.000964  
..                ...  
284         -0.002767  
285         -0.020041  
286         -0.001574  
287          0.005329  
288         -0.010538  

[289 rows x 77 columns]

In [41]:
# # Assume df1, df2, df3, df4, and df5 are DataFrames with a "date" column
# dfs = [YrBond10, MonBond3, dow30, nasdaq, sp500, comments, aapl, amc, amd, amzn, baba, lcid, meta, msft, nio, tsla]

# # Concatenate the DataFrames horizontally based on the "date" column
# df = pd.concat(dfs, axis=1, join='outer')

# # Reset the index to make "date" a regular column again
# df = df.reset_index()

# # Drop the duplicate "date" columns
# df = df.loc[:,~df.columns.duplicated()]

In [42]:
# from pandas.core.describe import DataFrameDescriber
# # Create an empty dataframe to store the concatenated data
# dataset = pd.DataFrame(columns=tsla.columns)
# dfs = [YrBond10, MonBond3, dow30, nasdaq, sp500, comments, aapl, amc, amd, amzn, baba, lcid, meta, msft, nio]
# # Loop through each dataset
# for df in dfs:
#     # Merge the dataset with the TSLA dataset on the date column
#     merged_df = pd.concat(tsla[["Date"]], df, how="outer")
    
#     # Append the merged dataframe to the concatenated dataframe
#     dataset = dataset.append(merged_df, ignore_index=True)

# dataset

In [43]:
# Fill null values with the last row's data from the same column
factors = factors.fillna(method='ffill')

In [44]:
factors

index       Date  comments_TSLA  comments_LCID  comments_META  \
0        0 2021-12-31            NaN            NaN            NaN   
1        3 2022-01-03       0.252083            NaN            NaN   
2        4 2022-01-04       0.228163           0.00          0.000   
3        5 2022-01-05       0.168773           0.00         -0.375   
4        6 2022-01-06       0.160335           0.36         -0.375   
..     ...        ...            ...            ...            ...   
284    413 2023-02-17       0.270062           0.75          0.500   
285    416 2023-02-21       0.270062           0.75         -0.500   
286    417 2023-02-22       0.270062           0.00         -0.500   
287    418 2023-02-23       0.270062           0.00         -0.500   
288    419 2023-02-24       0.270062           0.00         -0.500   

     comments_AMZN  comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  \
0              NaN            NaN            NaN           NaN           NaN   
1              NaN            NaN            NaN           NaN           NaN   
2         0.500000          0.000      -0.152778           NaN      0.000000   
3        -0.277778          0.375       0.132653           NaN      0.375000   
4        -0.277778          0.375      -0.375000           NaN      0.000000   
..             ...            ...            ...           ...           ...   
284       0.750000         -0.500       0.000000       0.21875     -0.277778   
285       0.750000         -0.500       0.000000       0.21875     -0.277778   
286       0.750000         -0.500       0.000000       0.21875      0.000000   
287       0.750000         -0.500       0.000000       0.21875      0.000000   
288       0.750000         -0.500       0.000000       0.21875      0.000000   

     ...  nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  sp500_Open  \
0    ...          15644.97   3.392580e+09                NaN     4775.21   
1    ...          15832.80   4.429960e+09           0.012006     4778.14   
2    ...          15622.72   5.131110e+09          -0.013269     4804.51   
3    ...          15100.17   5.031850e+09          -0.033448     4787.99   
4    ...          15080.86   4.790820e+09          -0.001279     4693.39   
..   ...               ...            ...                ...         ...   
284  ...          11787.27   4.925230e+09          -0.005783     4077.39   
285  ...          11492.30   5.002540e+09          -0.025024     4052.35   
286  ...          11507.07   4.671270e+09           0.001285     4001.83   
287  ...          11590.40   4.714580e+09           0.007242     4018.60   
288  ...          11394.94   4.432650e+09          -0.016864     3973.24   

     sp500_High  sp500_Low  sp500_Close*  sp500_Adj Close**  sp500_Volume  \
0       4786.83    4765.75       4766.18            4766.18  2.677820e+09   
1       4796.64    4758.17       4796.56            4796.56  3.831020e+09   
2       4818.62    4774.27       4793.54            4793.54  4.683170e+09   
3       4797.70    4699.44       4700.58            4700.58  4.887960e+09   
4       4725.01    4671.26       4696.05            4696.05  4.295280e+09   
..          ...        ...           ...                ...           ...   
284     4081.51    4047.95       4079.09            4079.09  4.045480e+09   
285     4052.35    3995.19       3997.34            3997.34  4.121590e+09   
286     4017.37    3976.90       3991.05            3991.05  4.079320e+09   
287     4028.30    3969.19       4012.32            4012.32  3.952940e+09   
288     3978.25    3943.08       3970.04            3970.04  3.877700e+09   

     sp500_percentage  
0                 NaN  
1            0.006374  
2           -0.000630  
3           -0.019393  
4           -0.000964  
..                ...  
284         -0.002767  
285         -0.020041  
286         -0.001574  
287          0.005329  
288         -0.010538  

[289 rows x 77 columns]

In [45]:
# Check for null values
null_values = factors.isnull().sum()
# Fill null values with 0
factors = factors.fillna(0)

In [46]:
factors

index       Date  comments_TSLA  comments_LCID  comments_META  \
0        0 2021-12-31       0.000000           0.00          0.000   
1        3 2022-01-03       0.252083           0.00          0.000   
2        4 2022-01-04       0.228163           0.00          0.000   
3        5 2022-01-05       0.168773           0.00         -0.375   
4        6 2022-01-06       0.160335           0.36         -0.375   
..     ...        ...            ...            ...            ...   
284    413 2023-02-17       0.270062           0.75          0.500   
285    416 2023-02-21       0.270062           0.75         -0.500   
286    417 2023-02-22       0.270062           0.00         -0.500   
287    418 2023-02-23       0.270062           0.00         -0.500   
288    419 2023-02-24       0.270062           0.00         -0.500   

     comments_AMZN  comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  \
0         0.000000          0.000       0.000000       0.00000      0.000000   
1         0.000000          0.000       0.000000       0.00000      0.000000   
2         0.500000          0.000      -0.152778       0.00000      0.000000   
3        -0.277778          0.375       0.132653       0.00000      0.375000   
4        -0.277778          0.375      -0.375000       0.00000      0.000000   
..             ...            ...            ...           ...           ...   
284       0.750000         -0.500       0.000000       0.21875     -0.277778   
285       0.750000         -0.500       0.000000       0.21875     -0.277778   
286       0.750000         -0.500       0.000000       0.21875      0.000000   
287       0.750000         -0.500       0.000000       0.21875      0.000000   
288       0.750000         -0.500       0.000000       0.21875      0.000000   

     ...  nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  sp500_Open  \
0    ...          15644.97   3.392580e+09           0.000000     4775.21   
1    ...          15832.80   4.429960e+09           0.012006     4778.14   
2    ...          15622.72   5.131110e+09          -0.013269     4804.51   
3    ...          15100.17   5.031850e+09          -0.033448     4787.99   
4    ...          15080.86   4.790820e+09          -0.001279     4693.39   
..   ...               ...            ...                ...         ...   
284  ...          11787.27   4.925230e+09          -0.005783     4077.39   
285  ...          11492.30   5.002540e+09          -0.025024     4052.35   
286  ...          11507.07   4.671270e+09           0.001285     4001.83   
287  ...          11590.40   4.714580e+09           0.007242     4018.60   
288  ...          11394.94   4.432650e+09          -0.016864     3973.24   

     sp500_High  sp500_Low  sp500_Close*  sp500_Adj Close**  sp500_Volume  \
0       4786.83    4765.75       4766.18            4766.18  2.677820e+09   
1       4796.64    4758.17       4796.56            4796.56  3.831020e+09   
2       4818.62    4774.27       4793.54            4793.54  4.683170e+09   
3       4797.70    4699.44       4700.58            4700.58  4.887960e+09   
4       4725.01    4671.26       4696.05            4696.05  4.295280e+09   
..          ...        ...           ...                ...           ...   
284     4081.51    4047.95       4079.09            4079.09  4.045480e+09   
285     4052.35    3995.19       3997.34            3997.34  4.121590e+09   
286     4017.37    3976.90       3991.05            3991.05  4.079320e+09   
287     4028.30    3969.19       4012.32            4012.32  3.952940e+09   
288     3978.25    3943.08       3970.04            3970.04  3.877700e+09   

     sp500_percentage  
0            0.000000  
1            0.006374  
2           -0.000630  
3           -0.019393  
4           -0.000964  
..                ...  
284         -0.002767  
285         -0.020041  
286         -0.001574  
287          0.005329  
288         -0.010538  

[289 rows x 77 columns]

In [47]:
factors.iloc[:,2:]

comments_TSLA  comments_LCID  comments_META  comments_AMZN  \
0         0.000000           0.00          0.000       0.000000   
1         0.252083           0.00          0.000       0.000000   
2         0.228163           0.00          0.000       0.500000   
3         0.168773           0.00         -0.375      -0.277778   
4         0.160335           0.36         -0.375      -0.277778   
..             ...            ...            ...            ...   
284       0.270062           0.75          0.500       0.750000   
285       0.270062           0.75         -0.500       0.750000   
286       0.270062           0.00         -0.500       0.750000   
287       0.270062           0.00         -0.500       0.750000   
288       0.270062           0.00         -0.500       0.750000   

     comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  comments_BABA  \
0            0.000       0.000000       0.00000      0.000000       0.000000   
1            0.000       0.000000       0.00000      0.000000       0.000000   
2            0.000      -0.152778       0.00000      0.000000       0.500000   
3            0.375       0.132653       0.00000      0.375000       0.209877   
4            0.375      -0.375000       0.00000      0.000000       0.147929   
..             ...            ...           ...           ...            ...   
284         -0.500       0.000000       0.21875     -0.277778      -0.375000   
285         -0.500       0.000000       0.21875     -0.277778      -0.375000   
286         -0.500       0.000000       0.21875      0.000000      -0.375000   
287         -0.500       0.000000       0.21875      0.000000      -0.375000   
288         -0.500       0.000000       0.21875      0.000000      -0.375000   

     comments_NIO  ...  nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  \
0       -0.285000  ...          15644.97   3.392580e+09           0.000000   
1        0.734568  ...          15832.80   4.429960e+09           0.012006   
2        0.693576  ...          15622.72   5.131110e+09          -0.013269   
3        0.338882  ...          15100.17   5.031850e+09          -0.033448   
4        0.355109  ...          15080.86   4.790820e+09          -0.001279   
..            ...  ...               ...            ...                ...   
284      0.277778  ...          11787.27   4.925230e+09          -0.005783   
285      0.500000  ...          11492.30   5.002540e+09          -0.025024   
286      0.000000  ...          11507.07   4.671270e+09           0.001285   
287      0.209877  ...          11590.40   4.714580e+09           0.007242   
288      0.209877  ...          11394.94   4.432650e+09          -0.016864   

     sp500_Open  sp500_High  sp500_Low  sp500_Close*  sp500_Adj Close**  \
0       4775.21     4786.83    4765.75       4766.18            4766.18   
1       4778.14     4796.64    4758.17       4796.56            4796.56   
2       4804.51     4818.62    4774.27       4793.54            4793.54   
3       4787.99     4797.70    4699.44       4700.58            4700.58   
4       4693.39     4725.01    4671.26       4696.05            4696.05   
..          ...         ...        ...           ...                ...   
284     4077.39     4081.51    4047.95       4079.09            4079.09   
285     4052.35     4052.35    3995.19       3997.34            3997.34   
286     4001.83     4017.37    3976.90       3991.05            3991.05   
287     4018.60     4028.30    3969.19       4012.32            4012.32   
288     3973.24     3978.25    3943.08       3970.04            3970.04   

     sp500_Volume  sp500_percentage  
0    2.677820e+09          0.000000  
1    3.831020e+09          0.006374  
2    4.683170e+09         -0.000630  
3    4.887960e+09         -0.019393  
4    4.295280e+09         -0.000964  
..            ...               ...  
284  4.045480e+09         -0.002767  
285  4.121590e+09         -0.020041  
286  4.079320e+09         -0.001574  
287  3.952940e+09    

## standard scaler

In [48]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(factors.iloc[:,2:])

# Transform the data using the scaler
factors_scaled = scaler.transform(factors.iloc[:,2:])

In [49]:
# tsla
scaler1 = StandardScaler()
scaler1.fit(tsla.iloc[:,1:])
tsla_scaled = scaler1.transform(tsla.iloc[:,1:])

In [50]:
#aapl
scaler2 = StandardScaler()
scaler2.fit(aapl.iloc[:,1:])
aapl_scaled = scaler2.transform(aapl.iloc[:,1:])

In [51]:
#amc
scaler3 = StandardScaler()
scaler3.fit(amc.iloc[:,1:])
amc_scaled = scaler3.transform(amc.iloc[:,1:])

In [52]:
#amd
scaler4 = StandardScaler()
scaler4.fit(amd.iloc[:,1:])
amd_scaled = scaler4.transform(amd.iloc[:,1:])

In [53]:
#amzn
scaler5 = StandardScaler()
scaler5.fit(amzn.iloc[:,1:])
amzn_scaled = scaler5.transform(amzn.iloc[:,1:])

In [54]:
#baba
scaler6 = StandardScaler()
scaler6.fit(baba.iloc[:,1:])
baba_scaled = scaler6.transform(baba.iloc[:,1:])

In [55]:
#lcid
scaler7 = StandardScaler()
scaler7.fit(lcid.iloc[:,1:])
lcid_scaled = scaler7.transform(lcid.iloc[:,1:])

In [56]:
#meta
scaler8 = StandardScaler()
scaler8.fit(meta.iloc[:,1:])
meta_scaled = scaler8.transform(meta.iloc[:,1:])

In [57]:
#msft
scaler9 = StandardScaler()
scaler9.fit(msft.iloc[:,1:])
msft_scaled = scaler9.transform(msft.iloc[:,1:])

In [58]:
#nio
scaler10 = StandardScaler()
scaler10.fit(nio.iloc[:,1:])
nio_scaled = scaler10.transform(nio.iloc[:,1:])

### factors

In [59]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(factors.columns[2:])

# Create a new dataframe using the transformed data and column names
factors_scaled = pd.DataFrame(factors_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
factors_scaled = pd.concat([factors.iloc[:,:2], factors_scaled], axis=1)

### tesla

In [60]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(tsla.columns[1:])

# Create a new dataframe using the transformed data and column names
tsla_scaled = pd.DataFrame(tsla_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
tsla_scaled = pd.concat([tsla.iloc[:,:1], tsla_scaled], axis=1)

### AAPL

In [61]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(aapl.columns[1:])

# Create a new dataframe using the transformed data and column names
aapl_scaled = pd.DataFrame(aapl_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
aapl_scaled = pd.concat([aapl.iloc[:,:1], aapl_scaled], axis=1)

### AMC

In [62]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(amc.columns[1:])

# Create a new dataframe using the transformed data and column names
amc_scaled = pd.DataFrame(amc_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
amc_scaled = pd.concat([amc.iloc[:,:1], amc_scaled], axis=1)

### AMD

In [63]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(amd.columns[1:])

# Create a new dataframe using the transformed data and column names
amd_scaled = pd.DataFrame(amd_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
amd_scaled = pd.concat([amd.iloc[:,:1], amd_scaled], axis=1)

### AMZN

In [64]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(amzn.columns[1:])

# Create a new dataframe using the transformed data and column names
amzn_scaled = pd.DataFrame(amzn_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
amzn_scaled = pd.concat([amzn.iloc[:,:1], amzn_scaled], axis=1)

### BABA

In [65]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(baba.columns[1:])

# Create a new dataframe using the transformed data and column names
baba_scaled = pd.DataFrame(baba_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
baba_scaled = pd.concat([baba.iloc[:,:1], baba_scaled], axis=1)

### LCID

In [66]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(lcid.columns[1:])

# Create a new dataframe using the transformed data and column names
lcid_scaled = pd.DataFrame(lcid_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
lcid_scaled = pd.concat([lcid.iloc[:,:1], lcid_scaled], axis=1)

### META

In [67]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(meta.columns[1:])

# Create a new dataframe using the transformed data and column names
meta_scaled = pd.DataFrame(meta_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
meta_scaled = pd.concat([meta.iloc[:,:1], meta_scaled], axis=1)

### MSFT

In [68]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(msft.columns[1:])

# Create a new dataframe using the transformed data and column names
msft_scaled = pd.DataFrame(msft_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
msft_scaled = pd.concat([msft.iloc[:,:1], msft_scaled], axis=1)

### NIO

In [69]:
# Get the column names of the original dataframe except for the first two columns
col_names = list(nio.columns[1:])

# Create a new dataframe using the transformed data and column names
nio_scaled = pd.DataFrame(nio_scaled, columns=col_names)

# Concatenate the first two columns of the original dataframe with the new scaled dataframe
nio_scaled = pd.concat([nio.iloc[:,:1], nio_scaled], axis=1)

In [70]:
factors_scaled

index       Date  comments_TSLA  comments_LCID  comments_META  \
0        0 2021-12-31       0.021044       0.023804       0.061247   
1        3 2022-01-03       1.017168       0.023804       0.061247   
2        4 2022-01-04       0.922648       0.023804       0.061247   
3        5 2022-01-05       0.687965       0.023804      -0.789824   
4        6 2022-01-06       0.654619       0.932626      -0.789824   
..     ...        ...            ...            ...            ...   
284    413 2023-02-17       1.088213       1.917182       1.196008   
285    416 2023-02-21       1.088213       1.917182      -1.073514   
286    417 2023-02-22       1.088213       0.023804      -1.073514   
287    418 2023-02-23       1.088213       0.023804      -1.073514   
288    419 2023-02-24       1.088213       0.023804      -1.073514   

     comments_AMZN  comments_MSFT  comments_AAPL  comments_AMC  comments_AMD  \
0        -0.156245      -0.447242       0.022247      0.080491     -0.141409   
1        -0.156245      -0.447242       0.022247      0.080491     -0.141409   
2         1.102543      -0.447242      -0.372743      0.080491     -0.141409   
3        -0.855572       0.456069       0.365207      0.080491      0.715227   
4        -0.855572       0.456069      -0.947273      0.080491     -0.141409   
..             ...            ...            ...           ...           ...   
284       1.731937      -1.651656       0.022247      1.083020     -0.775954   
285       1.731937      -1.651656       0.022247      1.083020     -0.775954   
286       1.731937      -1.651656       0.022247      1.083020     -0.141409   
287       1.731937      -1.651656       0.022247      1.083020     -0.141409   
288       1.731937      -1.651656       0.022247      1.083020     -0.141409   

     ...  nasdaq_Adj Close  nasdaq_Volume  nasdaq_percentage  sp500_Open  \
0    ...          2.701753      -2.143037           0.046525    2.443759   
1    ...          2.846329      -0.762989           0.662783    2.454237   
2    ...          2.684627       0.169765          -0.634556    2.548537   
3    ...          2.282412       0.037718          -1.670368    2.489461   
4    ...          2.267549      -0.282929          -0.019115    2.151167   
..   ...               ...            ...                ...         ...   
284  ...         -0.267579      -0.104121          -0.250307   -0.051677   
285  ...         -0.494622      -0.001274          -1.237982   -0.141221   
286  ...         -0.483253      -0.441969           0.112495   -0.321883   
287  ...         -0.419113      -0.384353           0.418239   -0.261913   
288  ...         -0.569562      -0.759410          -0.819103   -0.424122   

     sp500_High  sp500_Low  sp500_Close*  sp500_Adj Close**  sp500_Volume  \
0      2.386453   2.565384      2.431756           2.431756     -2.181684   
1      2.421844   2.538062      2.541157           2.541157     -0.835482   
2      2.501143   2.596093      2.530281           2.530281      0.159287   
3      2.425669   2.326376      2.195526           2.195526      0.398351   
4      2.163422   2.224804      2.179213           2.179213     -0.293522   
..          ...        ...           ...                ...           ...   
284   -0.158161  -0.021854     -0.042504          -0.042504     -0.585129   
285   -0.263363  -0.212022     -0.336892          -0.336892     -0.496281   
286   -0.389562  -0.277947     -0.359542          -0.359542     -0.545625   
287   -0.350129  -0.305737     -0.282948          -0.282948     -0.693157   
288   -0.530697  -0.399847     -0.435201          -0.435201     -0.780989   

     sp500_percentage  
0            0.035819  
1            0.470860  
2           -0.007153  
3           -1.287766  
4           -0.029955  
..                ...  
284         -0.153063  
285         -1.332025  
286         -0.071577  
287          0.399561  
288         -0.683384  

[289 rows x 77 columns]

In [71]:
tsla_scaled

Date  tsla_Open  tsla_High  tsla_Low  tsla_Close  tsla_Adj Close  \
0    2021-12-31   1.630044   1.553009  1.685381    1.568765        1.568765   
1    2022-01-03   2.009476   2.151690  2.111684    2.305123        2.305123   
2    2022-01-04   2.222909   2.186534  2.043696    2.046677        2.046677   
3    2022-01-05   2.003859   1.997180  1.823661    1.730146        1.730146   
4    2022-01-06   1.648222   1.583177  1.506956    1.609548        1.609548   
..          ...        ...        ...       ...         ...             ...   
284  2023-02-17  -0.787520  -0.743166 -0.733164   -0.654984       -0.654984   
285  2023-02-21  -0.710929  -0.724010 -0.737560   -0.823986       -0.823986   
286  2023-02-22  -0.819076  -0.840457 -0.822978   -0.770072       -0.770072   
287  2023-02-23  -0.727473  -0.792943 -0.751535   -0.751380       -0.751380   
288  2023-02-24  -0.843585  -0.905620 -0.806962   -0.831556       -0.831556   

     tsla_Volume  tsla_percentage  
0      -1.278389              NaN  
1       0.071410         3.166390  
2      -0.007255        -0.945887  
3      -0.437167        -1.216059  
4      -0.218953        -0.474434  
..           ...              ...  
284     2.416709         0.745600  
285     1.696508        -1.193929  
286     1.948748         0.435677  
287     0.977615         0.165043  
288     0.889365        -0.571021  

[289 rows x 8 columns]

# Prepare datasets for each stock

## tsla

In [72]:
# Create a new DataFrame from selected columns in the three DataFrames
tsla_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_TSLA'], factors_scaled['comments_TSLA_count'], factors_scaled['opinions_TSLA'], factors_scaled['opinions_TSLA_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], tsla_scaled['tsla_Volume'],tsla_scaled['tsla_percentage']], axis=1)

# Print the resulting DataFrame
tsla_scaled_df


Date  comments_TSLA  comments_TSLA_count  opinions_TSLA  \
0   2021-12-31       0.021044            -0.858083      -0.052801   
1   2022-01-03       1.017168             0.524695       0.965828   
2   2022-01-04       0.922648             5.704254       0.591026   
3   2022-01-05       0.687965             1.907473       0.820823   
4   2022-01-06       0.654619             1.907473       0.820823   
..         ...            ...                  ...            ...   
284 2023-02-17       1.088213            -0.436219      -0.052801   
285 2023-02-21       1.088213            -0.436219      -0.052801   
286 2023-02-22       1.088213            -0.436219      -0.052801   
287 2023-02-23       1.088213            -0.436219      -0.052801   
288 2023-02-24       1.088213            -0.436219      -0.052801   

     opinions_TSLA_count  nasdaq_percentage  YrBond10_Close  tsla_Volume  \
0              -1.627208           0.046525       -2.150586    -1.278389   
1               5.890321           0.662783       -1.987054     0.071410   
2               4.219759          -0.634556       -1.930663    -0.007255   
3               2.549197          -1.670368       -1.878502    -0.437167   
4               2.549197          -0.019115       -1.839028    -0.218953   
..                   ...                ...             ...          ...   
284             0.043354          -0.250307        1.114425     2.416709   
285             0.043354          -1.237982        1.293465     1.696508   
286             0.043354           0.112495        1.248352     1.948748   
287             0.043354           0.418239        1.186323     0.977615   
288             0.043354          -0.819103        1.285006     0.889365   

     tsla_percentage  
0                NaN  
1           3.166390  
2          -0.945887  
3          -1.216059  
4          -0.474434  
..               ...  
284         0.745600  
285        -1.193929  
286         0.435677  
287         0.165043  
288        -0.571021  

[289 rows x 9 columns]

In [73]:
tsla_scaled_df.to_csv('tsla_scaled_df.csv', index=False)

## msft

In [74]:
# Create a new DataFrame from selected columns in the three DataFrames
msft_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_MSFT'], factors_scaled['comments_MSFT_count'], factors_scaled['opinions_MSFT'], factors_scaled['opinions_MSFT_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], msft_scaled['msft_Volume'],msft_scaled['msft_percentage']], axis=1)

# Print the resulting DataFrame
msft_scaled_df

Date  comments_MSFT  comments_MSFT_count  opinions_MSFT  \
0   2021-12-31      -0.447242            -0.640631      -0.313664   
1   2022-01-03      -0.447242            -0.640631       1.463806   
2   2022-01-04      -0.447242            -0.229203      -0.313664   
3   2022-01-05       0.456069             0.182224       1.026275   
4   2022-01-06       0.456069             0.182224       1.026275   
..         ...            ...                  ...            ...   
284 2023-02-17      -1.651656            -0.229203       0.691290   
285 2023-02-21      -1.651656            -0.229203       1.026275   
286 2023-02-22      -1.651656            -0.229203       1.026275   
287 2023-02-23      -1.651656            -0.229203       1.026275   
288 2023-02-24      -1.651656            -0.229203       1.026275   

     opinions_MSFT_count  nasdaq_percentage  YrBond10_Close  msft_Volume  \
0              -1.335072           0.046525       -2.150586    -1.190510   
1               4.327088           0.662783       -1.987054    -0.229552   
2               0.282688          -0.634556       -1.930663     0.107376   
3              -0.526192          -1.670368       -1.878502     0.760144   
4              -0.526192          -0.019115       -1.839028     0.724038   
..                   ...                ...             ...          ...   
284             0.282688          -0.250307        1.114425    -0.129160   
285            -0.526192          -1.237982        1.293465    -0.270920   
286            -0.526192           0.112495        1.248352    -0.793338   
287            -0.526192           0.418239        1.186323    -0.198240   
288            -0.526192          -0.819103        1.285006    -0.572228   

     msft_percentage  
0                NaN  
1          -0.175365  
2          -0.740605  
3          -1.702723  
4          -0.321837  
..               ...  
284        -0.670607  
285        -0.909989  
286        -0.171867  
287         0.623196  
288        -0.950654  

[289 rows x 9 columns]

In [75]:
msft_scaled_df.to_csv('msft_scaled_df.csv', index=False)

## nio

In [76]:
# Create a new DataFrame from selected columns in the three DataFrames
nio_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_NIO'], factors_scaled['comments_NIO_count'], factors_scaled['opinions_NIO'], factors_scaled['opinions_NIO_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], nio_scaled['nio_Volume'],nio_scaled['nio_percentage']], axis=1)

# Print the resulting DataFrame
nio_scaled_df

Date  comments_NIO  comments_NIO_count  opinions_NIO  \
0   2021-12-31     -1.110573            0.100294     -0.156886   
1   2022-01-03      1.633749            0.013252     -0.156886   
2   2022-01-04      1.523414            1.318881     -0.156886   
3   2022-01-05      0.568701            1.754090     -0.156886   
4   2022-01-06      0.612377            3.843096     -0.156886   
..         ...           ...                 ...           ...   
284 2023-02-17      0.404229           -0.508999     -1.350041   
285 2023-02-21      1.002374           -0.683083     -1.350041   
286 2023-02-22     -0.343452           -0.683083     -1.350041   
287 2023-02-23      0.221463            0.013252     -1.350041   
288 2023-02-24      0.221463            0.013252     -1.350041   

     opinions_NIO_count  nasdaq_percentage  YrBond10_Close  nio_Volume  \
0             -6.347103           0.046525       -2.150586    0.370971   
1             -6.347103           0.662783       -1.987054    0.235914   
2             -6.347103          -0.634556       -1.930663   -0.439627   
3             -6.347103          -1.670368       -1.878502   -0.662786   
4             -6.347103          -0.019115       -1.839028   -0.404393   
..                  ...                ...             ...         ...   
284            0.157552          -0.250307        1.114425   -0.867497   
285            0.157552          -1.237982        1.293465   -0.967073   
286            0.157552           0.112495        1.248352   -1.006047   
287            0.157552           0.418239        1.186323   -0.341100   
288            0.157552          -0.819103        1.285006   -0.071730   

     nio_percentage  
0               NaN  
1          1.036983  
2         -0.943359  
3         -0.897134  
4         -0.287887  
..              ...  
284        0.046514  
285       -0.228731  
286        0.308672  
287       -0.625054  
288       -0.830863  

[289 rows x 9 columns]

In [77]:
nio_scaled_df.to_csv('nio_scaled_df.csv', index=False)

## meta

In [78]:
# Create a new DataFrame from selected columns in the three DataFrames
meta_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_META'], factors_scaled['comments_META_count'], factors_scaled['opinions_META'], factors_scaled['opinions_META_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], meta_scaled['meta_Volume'],meta_scaled['meta_percentage']], axis=1)

# Print the resulting DataFrame
meta_scaled_df

Date  comments_META  comments_META_count  opinions_META  \
0   2021-12-31       0.061247            -0.370421       0.137701   
1   2022-01-03       0.061247            -0.370421       0.137701   
2   2022-01-04       0.061247            -0.295560       1.982918   
3   2022-01-05      -0.789824            -0.220699       1.982918   
4   2022-01-06      -0.789824            -0.220699       1.982918   
..         ...            ...                  ...            ...   
284 2023-02-17       1.196008            -0.295560       1.367846   
285 2023-02-21      -1.073514            -0.295560       1.367846   
286 2023-02-22      -1.073514            -0.295560       1.367846   
287 2023-02-23      -1.073514            -0.295560       1.367846   
288 2023-02-24      -1.073514            -0.295560       1.367846   

     opinions_META_count  nasdaq_percentage  YrBond10_Close  meta_Volume  \
0              -1.141170           0.046525       -2.150586    -0.948551   
1              -1.141170           0.662783       -1.987054    -0.877950   
2               0.305313          -0.634556       -1.930663    -0.816125   
3               0.305313          -1.670368       -1.878502    -0.622768   
4               0.305313          -0.019115       -1.839028    -0.309505   
..                   ...                ...             ...          ...   
284            -0.417929          -0.250307        1.114425    -0.470047   
285            -0.417929          -1.237982        1.293465    -0.028782   
286            -0.417929           0.112495        1.248352    -0.543642   
287            -0.417929           0.418239        1.186323    -0.645917   
288            -0.417929          -0.819103        1.285006    -0.655507   

     meta_percentage  
0                NaN  
1           0.195405  
2          -0.108595  
3          -0.860525  
4           0.660912  
..               ...  
284         0.098712  
285        -0.076609  
286        -0.099842  
287         0.167693  
288        -0.197816  

[289 rows x 9 columns]

In [79]:
meta_scaled_df.to_csv('meta_scaled_df.csv', index=False)

## amzn

In [80]:
# Create a new DataFrame from selected columns in the three DataFrames
amzn_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_AMZN'], factors_scaled['comments_AMZN_count'], factors_scaled['opinions_AMZN'], factors_scaled['opinions_AMZN_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], amzn_scaled['amzn_Volume'],amzn_scaled['amzn_percentage']], axis=1)

# Print the resulting DataFrame
amzn_scaled_df

Date  comments_AMZN  comments_AMZN_count  opinions_AMZN  \
0   2021-12-31      -0.156245            -0.370669      -0.051453   
1   2022-01-03      -0.156245            -0.370669       1.101807   
2   2022-01-04       1.102543            -0.312890       0.647493   
3   2022-01-05      -0.855572            -0.197330       1.206649   
4   2022-01-06      -0.855572            -0.197330       2.045383   
..         ...            ...                  ...            ...   
284 2023-02-17       1.731937            -0.255110      -0.051453   
285 2023-02-21       1.731937            -0.255110      -0.995029   
286 2023-02-22       1.731937            -0.255110      -0.995029   
287 2023-02-23       1.731937            -0.255110      -0.995029   
288 2023-02-24       1.731937            -0.255110      -0.995029   

     opinions_AMZN_count  nasdaq_percentage  YrBond10_Close  amzn_Volume  \
0              -1.347562           0.046525       -2.150586    -0.856906   
1               2.653590           0.662783       -1.987054    -0.378252   
2               0.653014          -0.634556       -1.930663    -0.158419   
3              -0.680704          -1.670368       -1.878502    -0.354395   
4               0.653014          -0.019115       -1.839028    -0.730973   
..                   ...                ...             ...          ...   
284            -0.680704          -0.250307        1.114425    -0.484739   
285            -0.013845          -1.237982        1.293465    -0.589958   
286            -0.013845           0.112495        1.248352    -0.499849   
287            -0.013845           0.418239        1.186323    -0.837473   
288            -0.013845          -0.819103        1.285006    -0.575516   

     amzn_percentage  
0                NaN  
1           0.759827  
2          -0.494769  
3          -0.558328  
4          -0.166781  
..               ...  
284        -0.262180  
285        -0.817438  
286         0.460113  
287         0.058984  
288        -0.729287  

[289 rows x 9 columns]

In [81]:
amzn_scaled_df.to_csv('amzn_scaled_df.csv', index=False)

## aapl

In [82]:
# Create a new DataFrame from selected columns in the three DataFrames
aapl_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_AAPL'], factors_scaled['comments_AAPL_count'], factors_scaled['opinions_AAPL'], factors_scaled['opinions_AAPL_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], aapl_scaled['aapl_Volume'],aapl_scaled['aapl_percentage']], axis=1)

# Print the resulting DataFrame
aapl_scaled_df

Date  comments_AAPL  comments_AAPL_count  opinions_AAPL  \
0   2021-12-31       0.022247            -0.734293      -0.131558   
1   2022-01-03       0.022247            -0.734293       1.391846   
2   2022-01-04      -0.372743             0.827994       0.913676   
3   2022-01-05       0.365207             1.088375       0.957228   
4   2022-01-06      -0.947273            -0.213530       0.957228   
..         ...            ...                  ...            ...   
284 2023-02-17       0.022247            -0.473912       1.320156   
285 2023-02-21       0.022247            -0.473912      -0.131558   
286 2023-02-22       0.022247            -0.473912      -0.131558   
287 2023-02-23       0.022247            -0.473912      -0.131558   
288 2023-02-24       0.022247            -0.473912      -0.131558   

     opinions_AAPL_count  nasdaq_percentage  YrBond10_Close  aapl_Volume  \
0              -1.487138           0.046525       -2.150586    -0.911688   
1               5.545674           0.662783       -1.987054     0.786841   
2               2.419980          -0.634556       -1.930663     0.569302   
3               0.075709          -1.670368       -1.878502     0.368767   
4               0.075709          -0.019115       -1.839028     0.468194   
..                   ...                ...             ...          ...   
284            -0.705715          -0.250307        1.114425    -1.118332   
285            -0.705715          -1.237982        1.293465    -1.129966   
286            -0.705715           0.112495        1.248352    -1.460041   
287            -0.705715           0.418239        1.186323    -1.570002   
288            -0.705715          -0.819103        1.285006    -1.272720   

     aapl_percentage  
0                NaN  
1           1.165620  
2          -0.562257  
3          -1.199777  
4          -0.745686  
..               ...  
284        -0.326428  
285        -1.203441  
286         0.152242  
287         0.170322  
288        -0.805822  

[289 rows x 9 columns]

In [83]:
aapl_scaled_df.to_csv('aapl_scaled_df.csv', index=False)

## amc

In [219]:
# Create a new DataFrame from selected columns in the three DataFrames
amc_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_AMC'], factors_scaled['comments_AMC_count'], factors_scaled['opinions_AMC'], factors_scaled['opinions_AMC_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], amc_scaled['amc_Volume'],amc_scaled['amc_percentage']], axis=1)

# Print the resulting DataFrame
amc_scaled_df

Date  comments_AMC  comments_AMC_count  opinions_AMC  \
0   2021-12-31      0.080491           -0.614095      0.729903   
1   2022-01-03      0.080491           -0.614095      2.427624   
2   2022-01-04      0.080491           -0.614095      2.427624   
3   2022-01-05      0.080491           -0.614095      2.427624   
4   2022-01-06      0.080491           -0.614095      2.427624   
..         ...           ...                 ...           ...   
284 2023-02-17      1.083020            1.341534      0.729903   
285 2023-02-21      1.083020            1.341534      0.729903   
286 2023-02-22      1.083020            1.341534      0.729903   
287 2023-02-23      1.083020            1.341534      0.729903   
288 2023-02-24      1.083020            1.341534      0.729903   

     opinions_AMC_count  nasdaq_percentage  YrBond10_Close  amc_Volume  \
0             -8.573670           0.046525       -2.150586   -0.644966   
1             -0.058926           0.662783       -1.987054   -0.521768   
2             -0.058926          -0.634556       -1.930663   -0.277545   
3             -0.058926          -1.670368       -1.878502    0.159529   
4             -0.058926          -0.019115       -1.839028    0.674833   
..                  ...                ...             ...         ...   
284           -0.058926          -0.250307        1.114425   -0.490293   
285           -0.058926          -1.237982        1.293465    1.207130   
286           -0.058926           0.112495        1.248352    1.028009   
287           -0.058926           0.418239        1.186323    0.047608   
288           -0.058926          -0.819103        1.285006   -0.645329   

     amc_percentage  
0               NaN  
1         -0.304307  
2         -0.490297  
3         -1.413008  
4         -0.139631  
..              ...  
284        0.006091  
285        2.237479  
286        0.384214  
287       -0.032717  
288       -0.033028  

[289 rows x 9 columns]

In [220]:
amc_scaled_df.to_csv('amc_scaled_df.csv', index=False)

## amd

In [221]:
# Create a new DataFrame from selected columns in the three DataFrames
amd_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_AMD'], factors_scaled['comments_AMD_count'], factors_scaled['opinions_AMD'], factors_scaled['opinions_AMD_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], amd_scaled['amd_Volume'],amd_scaled['amd_percentage']], axis=1)

# Print the resulting DataFrame
amd_scaled_df

Date  comments_AMD  comments_AMD_count  opinions_AMD  \
0   2021-12-31     -0.141409           -0.921195     -0.113841   
1   2022-01-03     -0.141409           -0.921195     -0.113841   
2   2022-01-04     -0.141409           -0.605013     -0.113841   
3   2022-01-05      0.715227           -0.288831     -0.113841   
4   2022-01-06     -0.141409           -0.288831     -0.113841   
..         ...           ...                 ...           ...   
284 2023-02-17     -0.775954            0.027351      0.799348   
285 2023-02-21     -0.775954            0.027351      0.799348   
286 2023-02-22     -0.141409            0.343533      0.799348   
287 2023-02-23     -0.141409            0.343533      0.799348   
288 2023-02-24     -0.141409            0.343533      0.799348   

     opinions_AMD_count  nasdaq_percentage  YrBond10_Close  amd_Volume  \
0             -2.523278           0.046525       -2.150586   -1.278098   
1             -0.313498           0.662783       -1.987054   -0.939163   
2              1.896282          -0.634556       -1.930663   -0.230396   
3              1.896282          -1.670368       -1.878502   -0.734524   
4             -0.313498          -0.019115       -1.839028   -0.754976   
..                  ...                ...             ...         ...   
284            1.896282          -0.250307        1.114425   -1.333341   
285            1.896282          -1.237982        1.293465   -1.462155   
286            1.896282           0.112495        1.248352   -1.249524   
287            1.896282           0.418239        1.186323   -0.360171   
288            1.896282          -0.819103        1.285006   -1.371723   

     amd_percentage  
0               NaN  
1          1.187982  
2         -0.976302  
3         -1.460555  
4          0.051663  
..              ...  
284       -0.479443  
285       -0.539773  
286       -0.018175  
287        1.107690  
288       -0.507800  

[289 rows x 9 columns]

In [222]:
amd_scaled_df.to_csv('amd_scaled_df.csv', index=False)

## baba

In [223]:
# Create a new DataFrame from selected columns in the three DataFrames
baba_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_BABA'], factors_scaled['comments_BABA_count'], factors_scaled['opinions_BABA'], factors_scaled['opinions_BABA_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], baba_scaled['baba_Volume'],baba_scaled['baba_percentage']], axis=1)

# Print the resulting DataFrame
baba_scaled_df

Date  comments_BABA  comments_BABA_count  opinions_BABA  \
0   2021-12-31       0.051218            -0.867153      -0.029969   
1   2022-01-03       0.051218            -0.867153       1.164653   
2   2022-01-04       1.331331            -0.591760       1.164653   
3   2022-01-05       0.588549             1.611380       1.164653   
4   2022-01-06       0.429950             2.712950       1.164653   
..         ...            ...                  ...            ...   
284 2023-02-17      -0.908866            -0.316368      -1.224591   
285 2023-02-21      -0.908866            -0.316368      -1.224591   
286 2023-02-22      -0.908866            -0.316368      -1.224591   
287 2023-02-23      -0.908866            -0.316368      -1.224591   
288 2023-02-24      -0.908866            -0.316368      -1.224591   

     opinions_BABA_count  nasdaq_percentage  YrBond10_Close  baba_Volume  \
0              -3.757744           0.046525       -2.150586    -0.109618   
1              -0.288134           0.662783       -1.987054    -0.555550   
2              -0.288134          -0.634556       -1.930663    -0.315199   
3              -0.288134          -1.670368       -1.878502     0.624376   
4              -0.288134          -0.019115       -1.839028     0.350982   
..                   ...                ...             ...          ...   
284            -0.288134          -0.250307        1.114425    -0.475627   
285            -0.288134          -1.237982        1.293465     0.061691   
286            -0.288134           0.112495        1.248352    -0.097924   
287            -0.288134           0.418239        1.186323     0.851851   
288            -0.288134          -0.819103        1.285006     0.596083   

     baba_percentage  
0                NaN  
1           0.295992  
2          -0.149734  
3           0.295937  
4           0.996954  
..               ...  
284        -0.662916  
285        -1.082898  
286        -0.073664  
287        -0.143768  
288        -1.208808  

[289 rows x 9 columns]

In [224]:
baba_scaled_df.to_csv('baba_scaled_df.csv', index=False)

## lcid

In [225]:
# Create a new DataFrame from selected columns in the three DataFrames
lcid_scaled_df = pd.concat([factors_scaled['Date'], factors_scaled['comments_LCID'], factors_scaled['comments_LCID_count'], factors_scaled['opinions_LCID'], factors_scaled['opinions_LCID_count'], factors_scaled['nasdaq_percentage'], factors_scaled['YrBond10_Close'], lcid_scaled['lcid_Volume'],lcid_scaled['lcid_percentage']], axis=1)

# Print the resulting DataFrame
lcid_scaled_df

Date  comments_LCID  comments_LCID_count  opinions_LCID  \
0   2021-12-31       0.023804            -1.424335      -0.313914   
1   2022-01-03       0.023804            -1.424335      -0.313914   
2   2022-01-04       0.023804             4.240336      -0.313914   
3   2022-01-05       0.023804            -0.480223      -0.313914   
4   2022-01-06       0.932626             3.296224      -0.313914   
..         ...            ...                  ...            ...   
284 2023-02-17       1.917182             0.463889      -0.313914   
285 2023-02-21       1.917182             0.463889      -0.313914   
286 2023-02-22       0.023804             0.463889      -0.313914   
287 2023-02-23       0.023804             0.463889      -0.313914   
288 2023-02-24       0.023804             0.463889      -0.313914   

     opinions_LCID_count  nasdaq_percentage  YrBond10_Close  lcid_Volume  \
0              -6.347103           0.046525       -2.150586    -0.368346   
1              -6.347103           0.662783       -1.987054     1.166578   
2              -6.347103          -0.634556       -1.930663     0.272984   
3              -6.347103          -1.670368       -1.878502     1.027540   
4              -6.347103          -0.019115       -1.839028     1.912232   
..                   ...                ...             ...          ...   
284             0.157552          -0.250307        1.114425    -0.589485   
285             0.157552          -1.237982        1.293465    -0.139222   
286             0.157552           0.112495        1.248352     0.067604   
287             0.157552           0.418239        1.186323     2.382708   
288             0.157552          -0.819103        1.285006     0.101748   

     lcid_percentage  
0                NaN  
1           1.387018  
2          -0.586443  
3          -1.148523  
4           0.797477  
..               ...  
284         0.556752  
285        -1.585176  
286         0.204459  
287        -2.022497  
288        -0.494052  

[289 rows x 9 columns]

# Lagged Dataset

## tsla

In [84]:
tsla_scaled_df.iloc[:,1:5].columns

Index(['comments_TSLA', 'comments_TSLA_count', 'opinions_TSLA',
       'opinions_TSLA_count'],
      dtype='object')

In [85]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(tsla_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
tsla_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in tsla_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      tsla_scaled_lag['%s_lag%d' % (col, lag)] = tsla_scaled_df[col].shift(lag)

In [86]:
#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(tsla_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in tsla_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    tsla_scaled_lag['%s_lag%d' % (col, lags)] = tsla_scaled_df[col].shift(lags)

In [87]:
# Drop rows with missing data
tsla_scaled_lag = tsla_scaled_lag.dropna()
tsla_scaled_lag

Date  comments_TSLA_lag1  comments_TSLA_lag3  comments_TSLA_lag5  \
5   2022-01-07            0.654619            0.922648            0.021044   
6   2022-01-10            0.061160            0.687965            1.017168   
7   2022-01-11           -0.442031            0.654619            0.922648   
8   2022-01-12            1.202565            0.061160            0.687965   
9   2022-01-13            1.482026           -0.442031            0.654619   
..         ...                 ...                 ...                 ...   
284 2023-02-17            1.088213            0.850387            1.088213   
285 2023-02-21            1.088213            1.088213            2.216364   
286 2023-02-22            1.088213            1.088213            0.850387   
287 2023-02-23            1.088213            1.088213            1.088213   
288 2023-02-24            1.088213            1.088213            1.088213   

     comments_TSLA_count_lag1  comments_TSLA_count_lag3  \
5                    1.907473                  5.704254   
6                    1.438735                  1.907473   
7                   -0.670588                  1.907473   
8                    2.657455                  1.438735   
9                    0.524695                 -0.670588   
..                        ...                       ...   
284                 -0.436219                 -0.647151   
285                 -0.436219                 -0.436219   
286                 -0.436219                 -0.436219   
287                 -0.436219                 -0.436219   
288                 -0.436219                 -0.436219   

     comments_TSLA_count_lag5  opinions_TSLA_lag1  opinions_TSLA_lag3  \
5                   -0.858083            0.820823            0.591026   
6                    0.524695            1.767250            0.820823   
7                    5.704254            1.008895            0.820823   
8                    1.907473            0.478047            1.767250   
9                    1.907473            1.767250            1.008895   
..                        ...                 ...                 ...   
284                 -0.436219           -0.052801           -0.052801   
285                 -0.787772           -0.052801            1.160566   
286                 -0.647151           -0.052801           -0.052801   
287                 -0.436219           -0.052801           -0.052801   
288                 -0.436219           -0.052801           -0.052801   

     opinions_TSLA_lag5  opinions_TSLA_count_lag1  opinions_TSLA_count_lag3  \
5             -0.052801                  2.549197                  4.219759   
6              0.965828                  0.043354                  2.549197   
7              0.591026                  1.713916                  2.549197   
8              0.820823                  1.713916                  0.043354   
9              0.820823                  0.043354                  1.713916   
..                  ...                       ...                       ...   
284           -0.052801                  0.043354                  0.043354   
285           -0.052801                  0.043354                 -0.791927   
286           -0.052801                  0.043354                  0.043354   
287            1.160566                  0.043354                  0.043354   
288           -0.052801                  0.043354                  0.043354   

     opinions_TSLA_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.627208               -0.019115            -1.839028   
6                    5.890321               -0.446869            -1.785457   
7                    4.219759                0.070342            -1.772770   
8                    2.549197                0.770026            -1.820702   
9                    2.549197                0.164879            -1.850307   
..                        ...                     ...                 

In [88]:
tsla_scaled_lag.to_csv('tsla_scaled_lag.csv', index=False)

##msft

In [89]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(msft_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
msft_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in msft_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      msft_scaled_lag['%s_lag%d' % (col, lag)] = msft_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(msft_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in msft_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    msft_scaled_lag['%s_lag%d' % (col, lags)] = msft_scaled_df[col].shift(lags)

# Drop rows with missing data
msft_scaled_lag = msft_scaled_lag.dropna()
msft_scaled_lag


Date  comments_MSFT_lag1  comments_MSFT_lag3  comments_MSFT_lag5  \
5   2022-01-07            0.456069           -0.447242           -0.447242   
6   2022-01-10            0.456069            0.456069           -0.447242   
7   2022-01-11            0.456069            0.456069           -0.447242   
8   2022-01-12           -1.651656            0.456069            0.456069   
9   2022-01-13           -1.651656            0.456069            0.456069   
..         ...                 ...                 ...                 ...   
284 2023-02-17           -1.651656            0.456069            0.456069   
285 2023-02-21           -1.651656           -1.651656            0.456069   
286 2023-02-22           -1.651656           -1.651656            0.456069   
287 2023-02-23           -1.651656           -1.651656           -1.651656   
288 2023-02-24           -1.651656           -1.651656           -1.651656   

     comments_MSFT_count_lag1  comments_MSFT_count_lag3  \
5                    0.182224                 -0.229203   
6                    0.182224                  0.182224   
7                    0.182224                  0.182224   
8                   -0.229203                  0.182224   
9                   -0.229203                  0.182224   
..                        ...                       ...   
284                 -0.229203                  0.182224   
285                 -0.229203                 -0.229203   
286                 -0.229203                 -0.229203   
287                 -0.229203                 -0.229203   
288                 -0.229203                 -0.229203   

     comments_MSFT_count_lag5  opinions_MSFT_lag1  opinions_MSFT_lag3  \
5                   -0.640631            1.026275           -0.313664   
6                   -0.640631            1.026275            1.026275   
7                   -0.229203            1.026275            1.026275   
8                    0.182224           -0.313664            1.026275   
9                    0.182224            0.691290            1.026275   
..                        ...                 ...                 ...   
284                  0.182224            0.691290           -0.313664   
285                  0.182224            0.691290           -0.313664   
286                  0.182224            1.026275            0.691290   
287                 -0.229203            1.026275            0.691290   
288                 -0.229203            1.026275            1.026275   

     opinions_MSFT_lag5  opinions_MSFT_count_lag1  opinions_MSFT_count_lag3  \
5             -0.313664                 -0.526192                  0.282688   
6              1.463806                 -0.526192                 -0.526192   
7             -0.313664                 -0.526192                 -0.526192   
8              1.026275                  1.900448                 -0.526192   
9              1.026275                  0.282688                 -0.526192   
..                  ...                       ...                       ...   
284            0.691290                  0.282688                  0.282688   
285           -0.313664                  0.282688                 -0.526192   
286           -0.313664                 -0.526192                  0.282688   
287           -0.313664                 -0.526192                  0.282688   
288            0.691290                 -0.526192                 -0.526192   

     opinions_MSFT_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.335072               -0.019115            -1.839028   
6                    4.327088               -0.446869            -1.785457   
7                    0.282688                0.070342            -1.772770   
8                   -0.526192                0.770026            -1.820702   
9                   -0.526192                0.164879            -1.850307   
..                        ...                     ...                 

In [90]:
msft_scaled_lag.to_csv('msft_scaled_lag.csv', index=False)

## nio

In [91]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(nio_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
nio_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in nio_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      nio_scaled_lag['%s_lag%d' % (col, lag)] = nio_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(nio_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in nio_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    nio_scaled_lag['%s_lag%d' % (col, lags)] = nio_scaled_df[col].shift(lags)

# Drop rows with missing data
nio_scaled_lag = nio_scaled_lag.dropna()
nio_scaled_lag


Date  comments_NIO_lag1  comments_NIO_lag3  comments_NIO_lag5  \
5   2022-01-07           0.612377           1.523414          -1.110573   
6   2022-01-10           0.305042           0.568701           1.633749   
7   2022-01-11           1.084771           0.612377           1.523414   
8   2022-01-12          -0.457936           0.305042           0.568701   
9   2022-01-13           0.500650           1.084771           0.612377   
..         ...                ...                ...                ...   
284 2023-02-17           0.625543           0.404229           0.067773   
285 2023-02-21           0.404229          -1.091133           0.067773   
286 2023-02-22           1.002374           0.625543           0.404229   
287 2023-02-23          -0.343452           0.404229          -1.091133   
288 2023-02-24           0.221463           1.002374           0.625543   

     comments_NIO_count_lag1  comments_NIO_count_lag3  \
5                   3.843096                 1.318881   
6                   2.798593                 1.754090   
7                  -0.160832                 3.843096   
8                   1.231839                 2.798593   
9                   1.405923                -0.160832   
..                       ...                      ...   
284                -0.334916                -0.508999   
285                -0.508999                -0.508999   
286                -0.683083                -0.334916   
287                -0.683083                -0.508999   
288                 0.013252                -0.683083   

     comments_NIO_count_lag5  opinions_NIO_lag1  opinions_NIO_lag3  \
5                   0.100294          -0.156886          -0.156886   
6                   0.013252          -0.156886          -0.156886   
7                   1.318881          -0.156886          -0.156886   
8                   1.754090          -1.350041          -0.156886   
9                   3.843096          -1.350041          -0.156886   
..                       ...                ...                ...   
284                -0.247874          -1.350041          -1.350041   
285                -0.247874          -1.350041          -1.350041   
286                -0.508999          -1.350041          -1.350041   
287                -0.508999          -1.350041          -1.350041   
288                -0.334916          -1.350041          -1.350041   

     opinions_NIO_lag5  opinions_NIO_count_lag1  opinions_NIO_count_lag3  \
5            -0.156886                -6.347103                -6.347103   
6            -0.156886                -6.347103                -6.347103   
7            -0.156886                -6.347103                -6.347103   
8            -0.156886                 0.157552                -6.347103   
9            -0.156886                 0.157552                -6.347103   
..                 ...                      ...                      ...   
284          -1.350041                 0.157552                 0.157552   
285          -1.350041                 0.157552                 0.157552   
286          -1.350041                 0.157552                 0.157552   
287          -1.350041                 0.157552                 0.157552   
288          -1.350041                 0.157552                 0.157552   

     opinions_NIO_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                  -6.347103               -0.019115            -1.839028   
6                  -6.347103               -0.446869            -1.785457   
7                  -6.347103                0.070342            -1.772770   
8                  -6.347103                0.770026            -1.820702   
9                  -6.347103                0.164879            -1.850307   
..                       ...                     ...                  ...   
284                 0.157552               -0.866740             1.135571   
285                 0.157552               -0.250307  

In [92]:
nio_scaled_lag.to_csv('nio_scaled_lag.csv', index=False)

## meta

In [93]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(meta_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
meta_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in meta_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      meta_scaled_lag['%s_lag%d' % (col, lag)] = meta_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(meta_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in meta_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    meta_scaled_lag['%s_lag%d' % (col, lags)] = meta_scaled_df[col].shift(lags)

# Drop rows with missing data
meta_scaled_lag =meta_scaled_lag.dropna()
meta_scaled_lag


Date  comments_META_lag1  comments_META_lag3  comments_META_lag5  \
5   2022-01-07           -0.789824            0.061247            0.061247   
6   2022-01-10            0.061247           -0.789824            0.061247   
7   2022-01-11            0.061247           -0.789824            0.061247   
8   2022-01-12            1.763389            0.061247           -0.789824   
9   2022-01-13            1.763389            0.061247           -0.789824   
..         ...                 ...                 ...                 ...   
284 2023-02-17            1.196008            0.061247            0.061247   
285 2023-02-21            1.196008            1.196008            0.061247   
286 2023-02-22           -1.073514            1.196008            0.061247   
287 2023-02-23           -1.073514            1.196008            1.196008   
288 2023-02-24           -1.073514           -1.073514            1.196008   

     comments_META_count_lag1  comments_META_count_lag3  \
5                   -0.220699                 -0.295560   
6                   -0.295560                 -0.220699   
7                   -0.295560                 -0.220699   
8                   -0.220699                 -0.295560   
9                   -0.220699                 -0.295560   
..                        ...                       ...   
284                 -0.295560                 -0.295560   
285                 -0.295560                 -0.295560   
286                 -0.295560                 -0.295560   
287                 -0.295560                 -0.295560   
288                 -0.295560                 -0.295560   

     comments_META_count_lag5  opinions_META_lag1  opinions_META_lag3  \
5                   -0.370421            1.982918            1.982918   
6                   -0.370421            1.982918            1.982918   
7                   -0.295560            0.137701            1.982918   
8                   -0.220699            0.137701            1.982918   
9                   -0.220699            0.137701            0.137701   
..                        ...                 ...                 ...   
284                 -0.295560            1.367846            1.060309   
285                 -0.295560            1.367846            1.367846   
286                 -0.295560            1.367846            1.367846   
287                 -0.295560            1.367846            1.367846   
288                 -0.295560            1.367846            1.367846   

     opinions_META_lag5  opinions_META_count_lag1  opinions_META_count_lag3  \
5              0.137701                  0.305313                  0.305313   
6              0.137701                  0.305313                  0.305313   
7              1.982918                 -0.417929                  0.305313   
8              1.982918                 -0.417929                  0.305313   
9              1.982918                 -0.417929                 -0.417929   
..                  ...                       ...                       ...   
284            1.982918                 -0.417929                  0.305313   
285            1.982918                 -0.417929                 -0.417929   
286            1.060309                 -0.417929                 -0.417929   
287            1.367846                 -0.417929                 -0.417929   
288            1.367846                 -0.417929                 -0.417929   

     opinions_META_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.141170               -0.019115            -1.839028   
6                   -1.141170               -0.446869            -1.785457   
7                    0.305313                0.070342            -1.772770   
8                    0.305313                0.770026            -1.820702   
9                    0.305313                0.164879            -1.850307   
..                        ...                     ...                 

In [94]:
meta_scaled_lag.to_csv('meta_scaled_lag.csv', index=False)

## amzn

In [95]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(amzn_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
amzn_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in amzn_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      amzn_scaled_lag['%s_lag%d' % (col, lag)] = amzn_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(amzn_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in amzn_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    amzn_scaled_lag['%s_lag%d' % (col, lags)] = amzn_scaled_df[col].shift(lags)

# Drop rows with missing data
amzn_scaled_lag = amzn_scaled_lag.dropna()
amzn_scaled_lag


Date  comments_AMZN_lag1  comments_AMZN_lag3  comments_AMZN_lag5  \
5   2022-01-07           -0.855572            1.102543           -0.156245   
6   2022-01-10            0.787846           -0.855572           -0.156245   
7   2022-01-11            0.787846           -0.855572            1.102543   
8   2022-01-12            0.543081            0.787846           -0.855572   
9   2022-01-13            0.945194            0.787846           -0.855572   
..         ...                 ...                 ...                 ...   
284 2023-02-17            1.731937            1.731937           -1.362584   
285 2023-02-21            1.731937            1.731937           -1.362584   
286 2023-02-22            1.731937            1.731937            1.731937   
287 2023-02-23            1.731937            1.731937            1.731937   
288 2023-02-24            1.731937            1.731937            1.731937   

     comments_AMZN_count_lag1  comments_AMZN_count_lag3  \
5                   -0.197330                  -0.31289   
6                   -0.255110                  -0.19733   
7                   -0.255110                  -0.19733   
8                   -0.197330                  -0.25511   
9                   -0.139551                  -0.25511   
..                        ...                       ...   
284                 -0.255110                  -0.25511   
285                 -0.255110                  -0.25511   
286                 -0.255110                  -0.25511   
287                 -0.255110                  -0.25511   
288                 -0.255110                  -0.25511   

     comments_AMZN_count_lag5  opinions_AMZN_lag1  opinions_AMZN_lag3  \
5                   -0.370669            2.045383            0.647493   
6                   -0.370669            2.045383            1.206649   
7                   -0.312890            1.206649            2.045383   
8                   -0.197330            0.401464            2.045383   
9                   -0.197330            0.854381            1.206649   
..                        ...                 ...                 ...   
284                  0.322686           -0.051453           -0.051453   
285                  0.322686           -0.051453           -0.051453   
286                 -0.255110           -0.995029           -0.051453   
287                 -0.255110           -0.995029           -0.051453   
288                 -0.255110           -0.995029           -0.995029   

     opinions_AMZN_lag5  opinions_AMZN_count_lag1  opinions_AMZN_count_lag3  \
5             -0.051453                  0.653014                  0.653014   
6              1.101807                  0.653014                 -0.680704   
7              0.647493                 -0.680704                  0.653014   
8              1.206649                  1.986731                  0.653014   
9              2.045383                  1.986731                 -0.680704   
..                  ...                       ...                       ...   
284            0.892124                 -0.680704                 -0.680704   
285           -0.051453                 -0.680704                 -0.680704   
286           -0.051453                 -0.013845                 -0.680704   
287           -0.051453                 -0.013845                 -0.680704   
288           -0.051453                 -0.013845                 -0.013845   

     opinions_AMZN_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.347562               -0.019115            -1.839028   
6                    2.653590               -0.446869            -1.785457   
7                    0.653014                0.070342            -1.772770   
8                   -0.680704                0.770026            -1.820702   
9                    0.653014                0.164879            -1.850307   
..                        ...                     ...                 

In [96]:
amzn_scaled_lag.to_csv('amzn_scaled_lag.csv', index=False)

## aapl

In [97]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(aapl_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
aapl_scaled_lag = pd.DataFrame(tsla_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in aapl_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      aapl_scaled_lag['%s_lag%d' % (col, lag)] = aapl_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(aapl_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in aapl_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    aapl_scaled_lag['%s_lag%d' % (col, lags)] = aapl_scaled_df[col].shift(lags)

# Drop rows with missing data
aapl_scaled_lag = aapl_scaled_lag.dropna()
aapl_scaled_lag


Date  comments_AAPL_lag1  comments_AAPL_lag3  comments_AAPL_lag5  \
5   2022-01-07           -0.947273           -0.372743            0.022247   
6   2022-01-10           -0.372743            0.365207            0.022247   
7   2022-01-11            1.314942           -0.947273           -0.372743   
8   2022-01-12            0.708167           -0.372743            0.365207   
9   2022-01-13            0.708167            1.314942           -0.947273   
..         ...                 ...                 ...                 ...   
284 2023-02-17            0.022247            0.022247           -1.270447   
285 2023-02-21            0.022247            0.022247           -1.270447   
286 2023-02-22            0.022247            0.022247            0.022247   
287 2023-02-23            0.022247            0.022247            0.022247   
288 2023-02-24            0.022247            0.022247            0.022247   

     comments_AAPL_count_lag1  comments_AAPL_count_lag3  \
5                   -0.213530                  0.827994   
6                    0.827994                  1.088375   
7                   -0.473912                 -0.213530   
8                    1.088375                  0.827994   
9                    1.088375                 -0.473912   
..                        ...                       ...   
284                 -0.473912                 -0.473912   
285                 -0.473912                 -0.473912   
286                 -0.473912                 -0.473912   
287                 -0.473912                 -0.473912   
288                 -0.473912                 -0.473912   

     comments_AAPL_count_lag5  opinions_AAPL_lag1  opinions_AAPL_lag3  \
5                   -0.734293            0.957228            0.913676   
6                   -0.734293            0.957228            0.957228   
7                    0.827994           -0.131558            0.957228   
8                    1.088375           -0.131558            0.957228   
9                   -0.213530            0.957228           -0.131558   
..                        ...                 ...                 ...   
284                 -0.473912            1.320156           -0.131558   
285                 -0.473912            1.320156           -0.131558   
286                 -0.473912           -0.131558            1.320156   
287                 -0.473912           -0.131558            1.320156   
288                 -0.473912           -0.131558           -0.131558   

     opinions_AAPL_lag5  opinions_AAPL_count_lag1  opinions_AAPL_count_lag3  \
5             -0.131558                  0.075709                  2.419980   
6              1.391846                  0.075709                  0.075709   
7              0.913676                  1.638556                  0.075709   
8              0.957228                  1.638556                  0.075709   
9              0.957228                  0.075709                  1.638556   
..                  ...                       ...                       ...   
284           -0.131558                 -0.705715                 -0.705715   
285            0.957228                 -0.705715                 -0.705715   
286           -0.131558                 -0.705715                 -0.705715   
287           -0.131558                 -0.705715                 -0.705715   
288            1.320156                 -0.705715                 -0.705715   

     opinions_AAPL_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.487138               -0.019115            -1.839028   
6                    5.545674               -0.446869            -1.785457   
7                    2.419980                0.070342            -1.772770   
8                    0.075709                0.770026            -1.820702   
9                    0.075709                0.164879            -1.850307   
..                        ...                     ...                 

In [98]:
aapl_scaled_lag.to_csv('aapl_scaled_lag.csv', index=False)

## amc

In [226]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(amc_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
amc_scaled_lag = pd.DataFrame(amc_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in amc_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      amc_scaled_lag['%s_lag%d' % (col, lag)] = amc_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(amc_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in amc_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    amc_scaled_lag['%s_lag%d' % (col, lags)] = amc_scaled_df[col].shift(lags)

# Drop rows with missing data
amc_scaled_lag = amc_scaled_lag.dropna()
amc_scaled_lag

Date  comments_AMC_lag1  comments_AMC_lag3  comments_AMC_lag5  \
5   2022-01-07           0.080491           0.080491           0.080491   
6   2022-01-10           0.080491           0.080491           0.080491   
7   2022-01-11           0.080491           0.080491           0.080491   
8   2022-01-12           0.080491           0.080491           0.080491   
9   2022-01-13           0.080491           0.080491           0.080491   
..         ...                ...                ...                ...   
284 2023-02-17           1.083020          -3.356750          -3.356750   
285 2023-02-21           1.083020          -3.738665          -3.356750   
286 2023-02-22           1.083020           1.083020          -3.356750   
287 2023-02-23           1.083020           1.083020          -3.738665   
288 2023-02-24           1.083020           1.083020           1.083020   

     comments_AMC_count_lag1  comments_AMC_count_lag3  \
5                  -0.614095                -0.614095   
6                  -0.614095                -0.614095   
7                  -0.614095                -0.614095   
8                  -0.614095                -0.614095   
9                  -0.614095                -0.614095   
..                       ...                      ...   
284                 1.341534                 0.363720   
285                 1.341534                 0.852627   
286                 1.341534                 1.341534   
287                 1.341534                 1.341534   
288                 1.341534                 1.341534   

     comments_AMC_count_lag5  opinions_AMC_lag1  opinions_AMC_lag3  \
5                  -0.614095           2.427624           2.427624   
6                  -0.614095           2.427624           2.427624   
7                  -0.614095           2.427624           2.427624   
8                  -0.614095           2.427624           2.427624   
9                  -0.614095           2.427624           2.427624   
..                       ...                ...                ...   
284                 0.363720           0.729903           0.729903   
285                 0.363720           0.729903           0.729903   
286                 0.363720           0.729903           0.729903   
287                 0.852627           0.729903           0.729903   
288                 1.341534           0.729903           0.729903   

     opinions_AMC_lag5  opinions_AMC_count_lag1  opinions_AMC_count_lag3  \
5             0.729903                -0.058926                -0.058926   
6             2.427624                -0.058926                -0.058926   
7             2.427624                -0.058926                -0.058926   
8             2.427624                -0.058926                -0.058926   
9             2.427624                -0.058926                -0.058926   
..                 ...                      ...                      ...   
284          -0.967819                -0.058926                -0.058926   
285          -0.967819                -0.058926                -0.058926   
286           0.729903                -0.058926                -0.058926   
287           0.729903                -0.058926                -0.058926   
288           0.729903                -0.058926                -0.058926   

     opinions_AMC_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                  -8.573670               -0.019115            -1.839028   
6                  -0.058926               -0.446869            -1.785457   
7                  -0.058926                0.070342            -1.772770   
8                  -0.058926                0.770026            -1.820702   
9                  -0.058926                0.164879            -1.850307   
..                       ...                     ...                  ...   
284                -0.058926               -0.866740             1.135571   
285                -0.058926               -0.250307  

## amd

In [227]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(amd_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
amd_scaled_lag = pd.DataFrame(amd_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in amd_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      amd_scaled_lag['%s_lag%d' % (col, lag)] = amd_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(amd_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in amd_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    amd_scaled_lag['%s_lag%d' % (col, lags)] = amd_scaled_df[col].shift(lags)

# Drop rows with missing data
amd_scaled_lag = amd_scaled_lag.dropna()
amd_scaled_lag

Date  comments_AMD_lag1  comments_AMD_lag3  comments_AMD_lag5  \
5   2022-01-07          -0.141409          -0.141409          -0.141409   
6   2022-01-10           0.680961           0.715227          -0.141409   
7   2022-01-11           0.680961          -0.141409          -0.141409   
8   2022-01-12           1.857407           0.680961           0.715227   
9   2022-01-13           1.857407           0.680961          -0.141409   
..         ...                ...                ...                ...   
284 2023-02-17           0.715227          -0.141409          -1.283590   
285 2023-02-21          -0.775954          -0.141409           1.000772   
286 2023-02-22          -0.775954           0.715227          -0.141409   
287 2023-02-23          -0.141409          -0.775954          -0.141409   
288 2023-02-24          -0.141409          -0.775954           0.715227   

     comments_AMD_count_lag1  comments_AMD_count_lag3  \
5                  -0.288831                -0.605013   
6                   0.659715                -0.288831   
7                   0.659715                -0.288831   
8                   0.343533                 0.659715   
9                   0.343533                 0.659715   
..                       ...                      ...   
284                -0.288831                -0.605013   
285                 0.027351                -0.288831   
286                 0.027351                -0.288831   
287                 0.343533                 0.027351   
288                 0.343533                 0.027351   

     comments_AMD_count_lag5  opinions_AMD_lag1  opinions_AMD_lag3  \
5                  -0.921195          -0.113841          -0.113841   
6                  -0.921195           1.712537          -0.113841   
7                  -0.605013           1.103744          -0.113841   
8                  -0.288831          -0.113841           1.712537   
9                  -0.288831           1.103744           1.103744   
..                       ...                ...                ...   
284                -0.605013           0.799348           0.799348   
285                -0.605013           0.799348           0.799348   
286                -0.605013           0.799348           0.799348   
287                -0.288831           0.799348           0.799348   
288                -0.288831           0.799348           0.799348   

     opinions_AMD_lag5  opinions_AMD_count_lag1  opinions_AMD_count_lag3  \
5            -0.113841                -0.313498                 1.896282   
6            -0.113841                 1.896282                 1.896282   
7            -0.113841                -0.313498                -0.313498   
8            -0.113841                -0.313498                 1.896282   
9            -0.113841                -0.313498                -0.313498   
..                 ...                      ...                      ...   
284           0.799348                 1.896282                 1.896282   
285           0.799348                 1.896282                 1.896282   
286           0.799348                 1.896282                 1.896282   
287           0.799348                 1.896282                 1.896282   
288           0.799348                 1.896282                 1.896282   

     opinions_AMD_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                  -2.523278               -0.019115            -1.839028   
6                  -0.313498               -0.446869            -1.785457   
7                   1.896282                0.070342            -1.772770   
8                   1.896282                0.770026            -1.820702   
9                  -0.313498                0.164879            -1.850307   
..                       ...                     ...                  ...   
284                 1.896282               -0.866740             1.135571   
285                 1.896282               -0.250307  

## baba

In [228]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(baba_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
baba_scaled_lag = pd.DataFrame(baba_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in baba_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      baba_scaled_lag['%s_lag%d' % (col, lag)] = baba_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(baba_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in baba_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    baba_scaled_lag['%s_lag%d' % (col, lags)] = baba_scaled_df[col].shift(lags)

# Drop rows with missing data
baba_scaled_lag = baba_scaled_lag.dropna()
baba_scaled_lag

Date  comments_BABA_lag1  comments_BABA_lag3  comments_BABA_lag5  \
5   2022-01-07            0.429950            1.331331            0.051218   
6   2022-01-10           -0.731073            0.588549            0.051218   
7   2022-01-11           -0.731073            0.429950            1.331331   
8   2022-01-12            0.972899           -0.731073            0.588549   
9   2022-01-13            0.051218           -0.731073            0.429950   
..         ...                 ...                 ...                 ...   
284 2023-02-17           -0.908866            1.331331            1.331331   
285 2023-02-21           -0.908866            1.331331            1.331331   
286 2023-02-22           -0.908866           -0.908866            1.331331   
287 2023-02-23           -0.908866           -0.908866            1.331331   
288 2023-02-24           -0.908866           -0.908866           -0.908866   

     comments_BABA_count_lag1  comments_BABA_count_lag3  \
5                    2.712950                 -0.591760   
6                    0.785202                  1.611380   
7                    0.785202                  2.712950   
8                    0.509810                  0.785202   
9                   -0.591760                  0.785202   
..                        ...                       ...   
284                 -0.316368                 -0.591760   
285                 -0.316368                 -0.591760   
286                 -0.316368                 -0.316368   
287                 -0.316368                 -0.316368   
288                 -0.316368                 -0.316368   

     comments_BABA_count_lag5  opinions_BABA_lag1  opinions_BABA_lag3  \
5                   -0.867153            1.164653            1.164653   
6                   -0.867153            1.164653            1.164653   
7                   -0.591760           -1.224591            1.164653   
8                    1.611380           -1.224591            1.164653   
9                    2.712950           -1.224591           -1.224591   
..                        ...                 ...                 ...   
284                 -0.591760           -1.224591           -1.224591   
285                 -0.591760           -1.224591           -1.224591   
286                 -0.591760           -1.224591           -1.224591   
287                 -0.591760           -1.224591           -1.224591   
288                 -0.316368           -1.224591           -1.224591   

     opinions_BABA_lag5  opinions_BABA_count_lag1  opinions_BABA_count_lag3  \
5             -0.029969                 -0.288134                 -0.288134   
6              1.164653                 -0.288134                 -0.288134   
7              1.164653                 -0.288134                 -0.288134   
8              1.164653                 -0.288134                 -0.288134   
9              1.164653                 -0.288134                 -0.288134   
..                  ...                       ...                       ...   
284           -1.224591                 -0.288134                 -0.288134   
285           -1.224591                 -0.288134                 -0.288134   
286           -1.224591                 -0.288134                 -0.288134   
287           -1.224591                 -0.288134                 -0.288134   
288           -1.224591                 -0.288134                 -0.288134   

     opinions_BABA_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -3.757744               -0.019115            -1.839028   
6                   -0.288134               -0.446869            -1.785457   
7                   -0.288134                0.070342            -1.772770   
8                   -0.288134                0.770026            -1.820702   
9                   -0.288134                0.164879            -1.850307   
..                        ...                     ...                 

## lcid

In [229]:
lags = [1, 3, 5]#  # Lags to use
num_features = len(lcid_scaled_df.iloc[:,1:5].columns)  # Number of features in the input data (i.e., number of lagged values for each column)
num_unroll = 1  # Number of time steps to predict into the future
batch_size = 16

# Create a new DataFrame to store the lagged values
lcid_scaled_lag = pd.DataFrame(lcid_scaled_df['Date'])

# Add lagged values as new columns in the DataFrame
for col in lcid_scaled_df.iloc[:,1:5].columns:
    for lag in lags:
      lcid_scaled_lag['%s_lag%d' % (col, lag)] = lcid_scaled_df[col].shift(lag)

#lags = [1, 3, 5]#  # Lags to use
lags = 1
num_features = len(lcid_scaled_df.iloc[:,5:].columns)  # Number of features in the input data (i.e., number of lagged values for each column)

# Add lagged values as new columns in the DataFrame
for col in lcid_scaled_df.iloc[:,5:].columns:
    #for lag in lags:
    lcid_scaled_lag['%s_lag%d' % (col, lags)] = lcid_scaled_df[col].shift(lags)

# Drop rows with missing data
lcid_scaled_lag = lcid_scaled_lag.dropna()
lcid_scaled_lag

Date  comments_LCID_lag1  comments_LCID_lag3  comments_LCID_lag5  \
5   2022-01-07            0.932626            0.023804            0.023804   
6   2022-01-10            0.970493            0.023804            0.023804   
7   2022-01-11            0.970493            0.932626            0.023804   
8   2022-01-12            0.023804            0.970493            0.023804   
9   2022-01-13            0.970493            0.970493            0.932626   
..         ...                 ...                 ...                 ...   
284 2023-02-17            1.917182            1.917182            1.917182   
285 2023-02-21            1.917182            1.917182            1.917182   
286 2023-02-22            1.917182            1.917182            1.917182   
287 2023-02-23            0.023804            1.917182            1.917182   
288 2023-02-24            0.023804            1.917182            1.917182   

     comments_LCID_count_lag1  comments_LCID_count_lag3  \
5                    3.296224                  4.240336   
6                    0.463889                 -0.480223   
7                    0.463889                  3.296224   
8                   -0.480223                  0.463889   
9                    0.463889                  0.463889   
..                        ...                       ...   
284                  0.463889                  0.463889   
285                  0.463889                  0.463889   
286                  0.463889                  0.463889   
287                  0.463889                  0.463889   
288                  0.463889                  0.463889   

     comments_LCID_count_lag5  opinions_LCID_lag1  opinions_LCID_lag3  \
5                   -1.424335           -0.313914           -0.313914   
6                   -1.424335           -0.313914           -0.313914   
7                    4.240336           -0.313914           -0.313914   
8                   -0.480223            1.103604           -0.313914   
9                    3.296224            1.103604           -0.313914   
..                        ...                 ...                 ...   
284                  0.463889           -0.313914           -0.313914   
285                  0.463889           -0.313914           -0.313914   
286                  0.463889           -0.313914           -0.313914   
287                  0.463889           -0.313914           -0.313914   
288                  0.463889           -0.313914           -0.313914   

     opinions_LCID_lag5  opinions_LCID_count_lag1  opinions_LCID_count_lag3  \
5             -0.313914                 -6.347103                 -6.347103   
6             -0.313914                 -6.347103                 -6.347103   
7             -0.313914                 -6.347103                 -6.347103   
8             -0.313914                  0.157552                 -6.347103   
9             -0.313914                  0.157552                 -6.347103   
..                  ...                       ...                       ...   
284           -0.313914                  0.157552                  0.157552   
285           -0.313914                  0.157552                  0.157552   
286           -0.313914                  0.157552                  0.157552   
287           -0.313914                  0.157552                  0.157552   
288           -0.313914                  0.157552                  0.157552   

     opinions_LCID_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -6.347103               -0.019115            -1.839028   
6                   -6.347103               -0.446869            -1.785457   
7                   -6.347103                0.070342            -1.772770   
8                   -6.347103                0.770026            -1.820702   
9                   -6.347103                0.164879            -1.850307   
..                        ...                     ...                 

#---Following are LSTM model---

# Tesla

## Train Test Split

In [99]:
# Split data into training and validation sets
train_size = int(len(tsla_scaled_lag) * 0.8)
train_data = tsla_scaled_lag.iloc[:train_size, 1:]
val_data = tsla_scaled_lag.iloc[train_size:, 1:]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [100]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [101]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = tsla_scaled_lag.filter(like='comment').columns.tolist() + tsla_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_tsla_scaled_lag = tsla_scaled_lag.drop(columns=columns_to_drop)
stock_tsla_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  tsla_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028         -0.218953   
6   2022-01-10               -0.446869            -1.785457         -0.350774   
7   2022-01-11                0.070342            -1.772770         -0.187376   
8   2022-01-12                0.770026            -1.820702         -0.737390   
9   2022-01-13                0.164879            -1.850307         -0.359866   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571          2.755196   
285 2023-02-21               -0.250307             1.114425          2.416709   
286 2023-02-22               -1.237982             1.293465          1.696508   
287 2023-02-23                0.112495             1.248352          1.948748   
288 2023-02-24                0.418239             1.186323          0.977615   

     tsla_percentage_lag1  
5               -0.474434  
6               -0.797642  
7                0.729547  
8                0.162974  
9                0.937258  
..                    ...  
284             -1.296710  
285              0.745600  
286             -1.193929  
287              0.435677  
288              0.165043  

[284 rows x 5 columns]

In [102]:
# Split data into training and validation sets
stock_train_size = int(len(stock_tsla_scaled_lag) * 0.8)
stock_train_data = stock_tsla_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_tsla_scaled_lag.iloc[stock_train_size:, 1:]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [103]:
len(stock_train_target)

227

In [104]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [105]:
def create_sequences(data, target, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        seq_x = data[i : (i + time_steps)]
        seq_y = target[i + time_steps]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


In [106]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)


In [107]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model.add(Dropout(0.05))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.05))

model.add(BatchNormalization())
model.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 14s 514ms/step - loss: 1.2666 - val_loss: 1.3662
Epoch 2/200
7/7 [==============================] - 3s 376ms/step - loss: 1.0803 - val_loss: 1.4046
Epoch 3/200
7/7 [==============================] - 2s 337ms/step - loss: 0.9865 - val_loss: 1.4227
Epoch 4/200
7/7 [==============================] - 2s 234ms/step - loss: 0.9546 - val_loss: 1.3927
Epoch 5/200
7/7 [==============================] - 2s 243ms/step - loss: 0.9451 - val_loss: 1.3950
Epoch 6/200
7/7 [==============================] - 2s 242ms/step - loss: 0.9364 - val_loss: 1.4042
Epoch 7/200
7/7 [==============================] - 2s 234ms/step - loss: 0.9378 - val_loss: 1.3917
Epoch 8/200
7/7 [==============================] - 2s 228ms/step - loss: 0.9310 - val_loss: 1.3966
Epoch 9/200
7/7 [==============================] - 2s 285ms/step - loss: 0.9312 - val_loss: 1.3993
Epoch 10/200
7/7 [==============================] - 3s 373ms/step - loss: 0.9291 - val_loss: 1.4024
Epoch 11

In [108]:
# Calculate evaluation metrics
tsla_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
tsla_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
tsla_stock_rmse = np.sqrt(tsla_stock_mse)
tsla_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", tsla_stock_mae)
print("MSE:", tsla_stock_mse)
print("RMSE:", tsla_stock_rmse)
print("MAPE:", tsla_stock_mape)

MAE: 0.923506021134384
MSE: 1.3699817776163947
RMSE: 1.1704622068295902
MAPE: 104.68539818928477


## Only Sentiment

In [109]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = tsla_scaled_lag.filter(like='comment').columns.tolist() + tsla_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_tsla_scaled_lag = tsla_scaled_lag[columns_to_drop]
sentiment_tsla_scaled_lag 

comments_TSLA_lag1  comments_TSLA_lag3  comments_TSLA_lag5  \
5              0.654619            0.922648            0.021044   
6              0.061160            0.687965            1.017168   
7             -0.442031            0.654619            0.922648   
8              1.202565            0.061160            0.687965   
9              1.482026           -0.442031            0.654619   
..                  ...                 ...                 ...   
284            1.088213            0.850387            1.088213   
285            1.088213            1.088213            2.216364   
286            1.088213            1.088213            0.850387   
287            1.088213            1.088213            1.088213   
288            1.088213            1.088213            1.088213   

     comments_TSLA_count_lag1  comments_TSLA_count_lag3  \
5                    1.907473                  5.704254   
6                    1.438735                  1.907473   
7                   -0.670588                  1.907473   
8                    2.657455                  1.438735   
9                    0.524695                 -0.670588   
..                        ...                       ...   
284                 -0.436219                 -0.647151   
285                 -0.436219                 -0.436219   
286                 -0.436219                 -0.436219   
287                 -0.436219                 -0.436219   
288                 -0.436219                 -0.436219   

     comments_TSLA_count_lag5  opinions_TSLA_lag1  opinions_TSLA_lag3  \
5                   -0.858083            0.820823            0.591026   
6                    0.524695            1.767250            0.820823   
7                    5.704254            1.008895            0.820823   
8                    1.907473            0.478047            1.767250   
9                    1.907473            1.767250            1.008895   
..                        ...                 ...                 ...   
284                 -0.436219           -0.052801           -0.052801   
285                 -0.787772           -0.052801            1.160566   
286                 -0.647151           -0.052801           -0.052801   
287                 -0.436219           -0.052801           -0.052801   
288                 -0.436219           -0.052801           -0.052801   

     opinions_TSLA_lag5  opinions_TSLA_count_lag1  opinions_TSLA_count_lag3  \
5             -0.052801                  2.549197                  4.219759   
6              0.965828                  0.043354                  2.549197   
7              0.591026                  1.713916                  2.549197   
8              0.820823                  1.713916                  0.043354   
9              0.820823                  0.043354                  1.713916   
..                  ...                       ...                       ...   
284           -0.052801                  0.043354                  0.043354   
285           -0.052801                  0.043354                 -0.791927   
286           -0.052801                  0.043354                  0.043354   
287            1.160566                  0.043354                  0.043354   
288           -0.052801                  0.043354                  0.043354   

     opinions_TSLA_count_lag5  
5                   -1.627208  
6                    5.890321  
7                    4.219759  
8                    2.549197  
9                    2.549197  
..                        ...  
284                  0.043354  
285                  0.043354  
286                  0.043354  
287                 -0.791927  
288                  0.043354  

[284 rows x 12 columns]

In [110]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_tsla_scaled_lag) * 0.8)
sentiment_train_data = sentiment_tsla_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_tsla_scaled_lag.iloc[sentiment_train_size:, 1:]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [111]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [112]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)


In [113]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model1 = Sequential()
model1.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model1.add(Dropout(0.05))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(Dropout(0.05))

model1.add(BatchNormalization())
model1.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model1.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model1.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model1.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 571ms/step - loss: 1.2118 - val_loss: 1.3279
Epoch 2/200
7/7 [==============================] - 2s 228ms/step - loss: 1.0777 - val_loss: 1.3307
Epoch 3/200
7/7 [==============================] - 2s 226ms/step - loss: 1.0252 - val_loss: 1.3291
Epoch 4/200
7/7 [==============================] - 2s 220ms/step - loss: 0.9996 - val_loss: 1.3283
Epoch 5/200
7/7 [==============================] - 2s 224ms/step - loss: 0.9904 - val_loss: 1.3343
Epoch 6/200
7/7 [==============================] - 2s 250ms/step - loss: 0.9689 - val_loss: 1.3320
Epoch 7/200
7/7 [==============================] - 3s 367ms/step - loss: 0.9792 - val_loss: 1.3328
Epoch 8/200
7/7 [==============================] - 2s 292ms/step - loss: 0.9618 - val_loss: 1.3301
Epoch 9/200
7/7 [==============================] - 2s 231ms/step - loss: 0.9583 - val_loss: 1.3322
Epoch 10/200
7/7 [==============================] - 2s 239ms/step - loss: 0.9649 - val_loss: 1.3320
Epoch 11

In [114]:
# Calculate evaluation metrics
tsla_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
tsla_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
tsla_sentiment_rmse = np.sqrt(tsla_sentiment_mse)
tsla_sentiment_mape = np.mean(np.abs((sentiment_y_val - stock_y_pred) / sentiment_y_val)) * 100

print("MAE:", tsla_sentiment_mae)
print("MSE:", tsla_sentiment_mse)
print("RMSE:", tsla_sentiment_rmse)
print("MAPE:", tsla_sentiment_mape)

MAE: 0.906417918331924
MSE: 1.335870499477016
RMSE: 1.155798641406459
MAPE: 104.68539818928477


## Stock & Sentiment

In [115]:
# Split data into training and validation sets
all_train_size = int(len(tsla_scaled_lag) * 0.8)
all_train_data = tsla_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = tsla_scaled_lag.iloc[all_train_size:, 1:]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [116]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [117]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [118]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model2 = Sequential()
model2.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model2.add(Dropout(0.05))
model2.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model2.add(BatchNormalization())
model2.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model2.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model2.predict(all_X_val)


Epoch 1/200
7/7 [==============================] - 14s 730ms/step - loss: 1.5556 - val_loss: 1.3129
Epoch 2/200
7/7 [==============================] - 2s 225ms/step - loss: 1.1877 - val_loss: 1.3269
Epoch 3/200
7/7 [==============================] - 2s 226ms/step - loss: 1.0686 - val_loss: 1.3251
Epoch 4/200
7/7 [==============================] - 2s 231ms/step - loss: 1.0461 - val_loss: 1.3305
Epoch 5/200
7/7 [==============================] - 2s 233ms/step - loss: 1.0322 - val_loss: 1.3294
Epoch 6/200
7/7 [==============================] - 2s 233ms/step - loss: 0.9880 - val_loss: 1.3342
Epoch 7/200
7/7 [==============================] - 2s 231ms/step - loss: 0.9830 - val_loss: 1.3273
Epoch 8/200
7/7 [==============================] - 2s 366ms/step - loss: 0.9792 - val_loss: 1.3351
Epoch 9/200
7/7 [==============================] - 2s 348ms/step - loss: 0.9793 - val_loss: 1.3350
Epoch 10/200
7/7 [==============================] - 2s 232ms/step - loss: 0.9757 - val_loss: 1.3330
Epoch 11

In [119]:
print(all_y_pred)

[[[-0.0087327 ]
  [ 0.02244568]
  [ 0.02750383]
  [ 0.03061535]
  [ 0.02857158]
  [ 0.03699337]
  [ 0.05360714]
  [ 0.07921334]
  [ 0.08493688]
  [ 0.06592924]
  [ 0.04937482]
  [ 0.05079914]
  [ 0.06547429]
  [ 0.08646847]
  [ 0.09600513]
  [ 0.09984257]
  [ 0.10066026]
  [ 0.09505448]
  [ 0.07448643]
  [ 0.04850248]]

 [[ 0.01142223]
  [ 0.01942177]
  [ 0.0243203 ]
  [ 0.02339012]
  [ 0.03268756]
  [ 0.04996752]
  [ 0.07600612]
  [ 0.08166332]
  [ 0.06235456]
  [ 0.04527282]
  [ 0.04583662]
  [ 0.06008159]
  [ 0.08026374]
  [ 0.08928256]
  [ 0.09194943]
  [ 0.09195624]
  [ 0.08583649]
  [ 0.06701851]
  [ 0.0435091 ]
  [ 0.02670963]]

 [[-0.02019427]
  [-0.01155293]
  [-0.00751561]
  [ 0.00659725]
  [ 0.02718963]
  [ 0.0559007 ]
  [ 0.06349386]
  [ 0.04577986]
  [ 0.02959776]
  [ 0.03036868]
  [ 0.04514449]
  [ 0.06563348]
  [ 0.0753435 ]
  [ 0.07993919]
  [ 0.08242632]
  [ 0.08066126]
  [ 0.0655107 ]
  [ 0.04681096]
  [ 0.03601078]
  [ 0.05427085]]

 [[-0.01264449]
  [-0.00241697]
  

In [120]:
# Calculate evaluation metrics
tsla_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
tsla_all_mse = np.mean(np.square(all_y_val - all_y_pred))
tsla_all_rmse = np.sqrt(tsla_all_mse)
tsla_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", tsla_all_mae)
print("MSE:", tsla_all_mse)
print("RMSE:", tsla_all_rmse)
print("MAPE:", tsla_all_mape)

MAE: 0.8983471961659774
MSE: 1.3201225910966048
RMSE: 1.148965878995806
MAPE: 96.93206954980616


## Stock & Opinion

In [ ]:
tsla_scaled_lag.iloc[:,7:]

opinions_TSLA_lag1  opinions_TSLA_lag3  opinions_TSLA_lag5  \
5              0.820823            0.591026           -0.052801   
6              1.767250            0.820823            0.965828   
7              1.008895            0.820823            0.591026   
8              0.478047            1.767250            0.820823   
9              1.767250            1.008895            0.820823   
..                  ...                 ...                 ...   
284           -0.052801           -0.052801           -0.052801   
285           -0.052801            1.160566           -0.052801   
286           -0.052801           -0.052801           -0.052801   
287           -0.052801           -0.052801            1.160566   
288           -0.052801           -0.052801           -0.052801   

     opinions_TSLA_count_lag1  opinions_TSLA_count_lag3  \
5                    2.549197                  4.219759   
6                    0.043354                  2.549197   
7                    1.713916                  2.549197   
8                    1.713916                  0.043354   
9                    0.043354                  1.713916   
..                        ...                       ...   
284                  0.043354                  0.043354   
285                  0.043354                 -0.791927   
286                  0.043354                  0.043354   
287                  0.043354                  0.043354   
288                  0.043354                  0.043354   

     opinions_TSLA_count_lag5  nasdaq_percentage_lag1  YrBond10_Close_lag1  \
5                   -1.627208               -0.019115            -1.839028   
6                    5.890321               -0.446869            -1.785457   
7                    4.219759                0.070342            -1.772770   
8                    2.549197                0.770026            -1.820702   
9                    2.549197                0.164879            -1.850307   
..                        ...                     ...                  ...   
284                  0.043354               -0.866740             1.135571   
285                  0.043354               -0.250307             1.114425   
286                  0.043354               -1.237982             1.293465   
287                 -0.791927                0.112495             1.248352   
288                  0.043354                0.418239             1.186323   

     tsla_Volume_lag1  tsla_percentage_lag1  
5           -0.218953             -0.474434  
6           -0.350774             -0.797642  
7           -0.187376              0.729547  
8           -0.737390              0.162974  
9           -0.359866              0.937258  
..                ...                   ...  
284          2.755196             -1.296710  
285          2.416709              0.745600  
286          1.696508             -1.193929  
287          1.948748              0.435677  
288          0.977615              0.165043  

[284 rows x 10 columns]

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(tsla_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = tsla_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = tsla_scaled_lag.iloc[all_train_size:, 7 :]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model2 = Sequential()
model2.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model2.add(Dropout(0.05))
model2.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model2.add(BatchNormalization())
model2.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model2.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model2.predict(all_X_val)


Epoch 1/200
7/7 [==============================] - 26s 1s/step - loss: 1.2654 - val_loss: 1.3530
Epoch 2/200
7/7 [==============================] - 3s 518ms/step - loss: 1.0781 - val_loss: 1.3596
Epoch 3/200
7/7 [==============================] - 4s 549ms/step - loss: 1.0232 - val_loss: 1.3541
Epoch 4/200
7/7 [==============================] - 4s 631ms/step - loss: 0.9866 - val_loss: 1.3747
Epoch 5/200
7/7 [==============================] - 3s 408ms/step - loss: 0.9802 - val_loss: 1.3574
Epoch 6/200
7/7 [==============================] - 3s 458ms/step - loss: 0.9684 - val_loss: 1.3578
Epoch 7/200
7/7 [==============================] - 3s 476ms/step - loss: 0.9629 - val_loss: 1.3546
Epoch 8/200
7/7 [==============================] - 3s 394ms/step - loss: 0.9731 - val_loss: 1.3581
Epoch 9/200
7/7 [==============================] - 3s 411ms/step - loss: 0.9597 - val_loss: 1.3734
Epoch 10/200
7/7 [==============================] - 2s 266ms/step - loss: 0.9555 - val_loss: 1.3628
Epoch 11/20

In [ ]:
print(all_y_pred)

[[[-8.62102211e-02]
  [-9.61919650e-02]
  [-1.10510521e-01]
  [-1.12921119e-01]
  [-1.07448384e-01]
  [-9.86336395e-02]
  [-8.70392844e-02]
  [-7.71358237e-02]
  [-6.25339523e-02]
  [-5.08241542e-02]
  [-3.90450247e-02]
  [-3.01542357e-02]
  [-1.46042230e-02]
  [ 4.45853584e-06]
  [ 8.29743873e-03]
  [-1.21687248e-04]
  [-1.15085337e-02]
  [-2.25271974e-02]
  [-8.93258397e-03]
  [-1.57636758e-02]]

 [[-8.95608589e-02]
  [-1.05582885e-01]
  [-1.09996237e-01]
  [-1.06247865e-01]
  [-9.84518304e-02]
  [-8.72030109e-02]
  [-7.70938993e-02]
  [-6.20941631e-02]
  [-4.98680212e-02]
  [-3.72847915e-02]
  [-2.73630321e-02]
  [-1.06573673e-02]
  [ 5.25880558e-03]
  [ 1.49419000e-02]
  [ 8.94682016e-03]
  [ 9.21663188e-04]
  [-5.17118024e-03]
  [ 1.58697441e-02]
  [ 1.91567615e-02]
  [ 7.73647567e-03]]

 [[-9.77584943e-02]
  [-1.05067074e-01]
  [-1.03948586e-01]
  [-9.79240462e-02]
  [-8.77413452e-02]
  [-7.80586600e-02]
  [-6.33375719e-02]
  [-5.14446497e-02]
  [-3.92304882e-02]
  [-2.99104042e-

In [ ]:
# Calculate evaluation metrics
tsla_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
tsla_all_mse = np.mean(np.square(all_y_val - all_y_pred))
tsla_all_rmse = np.sqrt(tsla_all_mse)
tsla_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", tsla_all_mae)
print("MSE:", tsla_all_mse)
print("RMSE:", tsla_all_rmse)
print("MAPE:", tsla_all_mape)

MAE: 0.9142946348759775
MSE: 1.3514315816837452
RMSE: 1.1625108952967904
MAPE: 101.34797673969098


## Stock & Comment

In [ ]:
comments=tsla_scaled_lag.iloc[:, 1:7]
market=tsla_scaled_lag.iloc[:, 13:]
stock_commnet= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets

all_train_size = int(len(stock_commnet) * 0.8)
all_train_data = stock_commnet.iloc[:all_train_size, :]
all_val_data = stock_commnet.iloc[all_train_size:, :]
target=tsla_scaled_df['tsla_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model2 = Sequential()
model2.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model2.add(Dropout(0.05))
model2.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model2.add(BatchNormalization())
model2.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model2.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model2.predict(all_X_val)

Epoch 1/200
7/7 [==============================] - 20s 816ms/step - loss: 1.3232 - val_loss: 1.3216
Epoch 2/200
7/7 [==============================] - 2s 342ms/step - loss: 1.1177 - val_loss: 1.3244
Epoch 3/200
7/7 [==============================] - 2s 308ms/step - loss: 1.1215 - val_loss: 1.3372
Epoch 4/200
7/7 [==============================] - 2s 329ms/step - loss: 1.0564 - val_loss: 1.3332
Epoch 5/200
7/7 [==============================] - 4s 525ms/step - loss: 0.9853 - val_loss: 1.3402
Epoch 6/200
7/7 [==============================] - 4s 536ms/step - loss: 0.9722 - val_loss: 1.3316
Epoch 7/200
7/7 [==============================] - 3s 401ms/step - loss: 0.9606 - val_loss: 1.3399
Epoch 8/200
7/7 [==============================] - 2s 310ms/step - loss: 0.9653 - val_loss: 1.3451
Epoch 9/200
7/7 [==============================] - 2s 289ms/step - loss: 0.9590 - val_loss: 1.3354
Epoch 10/200
7/7 [==============================] - 3s 488ms/step - loss: 0.9596 - val_loss: 1.3377
Epoch 11

In [ ]:
# Calculate evaluation metrics
tsla_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
tsla_all_mse = np.mean(np.square(all_y_val - all_y_pred))
tsla_all_rmse = np.sqrt(tsla_all_mse)
tsla_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", tsla_all_mae)
print("MSE:", tsla_all_mse)
print("RMSE:", tsla_all_rmse)
print("MAPE:", tsla_all_mape)

MAE: 0.9001189233770709
MSE: 1.3230135807235797
RMSE: 1.1502232742922478
MAPE: 96.57193109882121


# MSFT

## Train Test Split

In [121]:
# Split data into training and validation sets
train_size = int(len(msft_scaled_lag) * 0.8)
train_data = msft_scaled_lag.iloc[:train_size, 1:]
val_data = msft_scaled_lag.iloc[train_size:, 1:]
target=msft_scaled_df['msft_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [122]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [123]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = msft_scaled_lag.filter(like='comment').columns.tolist() + msft_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_msft_scaled_lag = msft_scaled_lag.drop(columns=columns_to_drop)
stock_msft_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  msft_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028          0.724038   
6   2022-01-10               -0.446869            -1.785457          0.111418   
7   2022-01-11                0.070342            -1.772770          1.134752   
8   2022-01-12                0.770026            -1.820702         -0.183407   
9   2022-01-13                0.164879            -1.850307          0.257557   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.164231   
285 2023-02-21               -0.250307             1.114425         -0.129160   
286 2023-02-22               -1.237982             1.293465         -0.270920   
287 2023-02-23                0.112495             1.248352         -0.793338   
288 2023-02-24                0.418239             1.186323         -0.198240   

     msft_percentage_lag1  
5               -0.321837  
6                0.059175  
7                0.069256  
8                0.138420  
9                0.509200  
..                    ...  
284             -1.169807  
285             -0.670607  
286             -0.909989  
287             -0.171867  
288              0.623196  

[284 rows x 5 columns]

In [124]:
# Split data into training and validation sets
stock_train_size = int(len(stock_msft_scaled_lag) * 0.8)
stock_train_data = stock_msft_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_msft_scaled_lag.iloc[stock_train_size:, 1:]
target=msft_scaled_df['msft_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [125]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [126]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [127]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model3 = Sequential()
model3.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model3.add(Dropout(0.05))
model3.add(Bidirectional(LSTM(128, return_sequences=True)))
model3.add(Dropout(0.05))

model3.add(BatchNormalization())
model3.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model3.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model3.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model3.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 14s 518ms/step - loss: 1.4034 - val_loss: 0.9053
Epoch 2/200
7/7 [==============================] - 2s 350ms/step - loss: 1.2607 - val_loss: 0.9035
Epoch 3/200
7/7 [==============================] - 3s 366ms/step - loss: 1.2096 - val_loss: 0.9049
Epoch 4/200
7/7 [==============================] - 2s 238ms/step - loss: 1.2163 - val_loss: 0.9065
Epoch 5/200
7/7 [==============================] - 2s 228ms/step - loss: 1.1849 - val_loss: 0.9041
Epoch 6/200
7/7 [==============================] - 2s 230ms/step - loss: 1.1741 - val_loss: 0.9065
Epoch 7/200
7/7 [==============================] - 2s 228ms/step - loss: 1.1791 - val_loss: 0.9052
Epoch 8/200
7/7 [==============================] - 2s 227ms/step - loss: 1.1688 - val_loss: 0.9057
Epoch 9/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1669 - val_loss: 0.9065
Epoch 10/200
7/7 [==============================] - 3s 389ms/step - loss: 1.1650 - val_loss: 0.9063
Epoch 11

In [128]:
# Calculate evaluation metrics
msft_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
msft_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
msft_stock_rmse = np.sqrt(msft_stock_mse)
msft_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", msft_stock_mae)
print("MSE:", msft_stock_mse)
print("RMSE:", msft_stock_rmse)
print("MAPE:", msft_stock_mape)

MAE: 0.7581534557989812
MSE: 0.9047334208701538
RMSE: 0.9511747583226512
MAPE: 116.171078118648


## Only Sentiment

In [129]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = msft_scaled_lag.filter(like='comment').columns.tolist() + msft_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_msft_scaled_lag = msft_scaled_lag[columns_to_drop]
sentiment_msft_scaled_lag 

comments_MSFT_lag1  comments_MSFT_lag3  comments_MSFT_lag5  \
5              0.456069           -0.447242           -0.447242   
6              0.456069            0.456069           -0.447242   
7              0.456069            0.456069           -0.447242   
8             -1.651656            0.456069            0.456069   
9             -1.651656            0.456069            0.456069   
..                  ...                 ...                 ...   
284           -1.651656            0.456069            0.456069   
285           -1.651656           -1.651656            0.456069   
286           -1.651656           -1.651656            0.456069   
287           -1.651656           -1.651656           -1.651656   
288           -1.651656           -1.651656           -1.651656   

     comments_MSFT_count_lag1  comments_MSFT_count_lag3  \
5                    0.182224                 -0.229203   
6                    0.182224                  0.182224   
7                    0.182224                  0.182224   
8                   -0.229203                  0.182224   
9                   -0.229203                  0.182224   
..                        ...                       ...   
284                 -0.229203                  0.182224   
285                 -0.229203                 -0.229203   
286                 -0.229203                 -0.229203   
287                 -0.229203                 -0.229203   
288                 -0.229203                 -0.229203   

     comments_MSFT_count_lag5  opinions_MSFT_lag1  opinions_MSFT_lag3  \
5                   -0.640631            1.026275           -0.313664   
6                   -0.640631            1.026275            1.026275   
7                   -0.229203            1.026275            1.026275   
8                    0.182224           -0.313664            1.026275   
9                    0.182224            0.691290            1.026275   
..                        ...                 ...                 ...   
284                  0.182224            0.691290           -0.313664   
285                  0.182224            0.691290           -0.313664   
286                  0.182224            1.026275            0.691290   
287                 -0.229203            1.026275            0.691290   
288                 -0.229203            1.026275            1.026275   

     opinions_MSFT_lag5  opinions_MSFT_count_lag1  opinions_MSFT_count_lag3  \
5             -0.313664                 -0.526192                  0.282688   
6              1.463806                 -0.526192                 -0.526192   
7             -0.313664                 -0.526192                 -0.526192   
8              1.026275                  1.900448                 -0.526192   
9              1.026275                  0.282688                 -0.526192   
..                  ...                       ...                       ...   
284            0.691290                  0.282688                  0.282688   
285           -0.313664                  0.282688                 -0.526192   
286           -0.313664                 -0.526192                  0.282688   
287           -0.313664                 -0.526192                  0.282688   
288            0.691290                 -0.526192                 -0.526192   

     opinions_MSFT_count_lag5  
5                   -1.335072  
6                    4.327088  
7                    0.282688  
8                   -0.526192  
9                   -0.526192  
..                        ...  
284                  0.282688  
285                 -0.526192  
286                  0.282688  
287                 -0.526192  
288                  0.282688  

[284 rows x 12 columns]

In [130]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_msft_scaled_lag) * 0.8)
sentiment_train_data = sentiment_msft_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_msft_scaled_lag.iloc[sentiment_train_size:, 1:]
target=msft_scaled_df['msft_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [131]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [132]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)

In [133]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model4 = Sequential()
model4.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model4.add(Dropout(0.05))
model4.add(Bidirectional(LSTM(128, return_sequences=True)))
model4.add(Dropout(0.05))

model4.add(BatchNormalization())
model4.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model4.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model4.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model4.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 511ms/step - loss: 1.6440 - val_loss: 0.9064
Epoch 2/200
7/7 [==============================] - 2s 227ms/step - loss: 1.3535 - val_loss: 0.9092
Epoch 3/200
7/7 [==============================] - 2s 227ms/step - loss: 1.2903 - val_loss: 0.9096
Epoch 4/200
7/7 [==============================] - 2s 225ms/step - loss: 1.2403 - val_loss: 0.9108
Epoch 5/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2135 - val_loss: 0.9066
Epoch 6/200
7/7 [==============================] - 2s 379ms/step - loss: 1.2060 - val_loss: 0.9109
Epoch 7/200
7/7 [==============================] - 2s 338ms/step - loss: 1.1946 - val_loss: 0.9094
Epoch 8/200
7/7 [==============================] - 2s 234ms/step - loss: 1.1909 - val_loss: 0.9086
Epoch 9/200
7/7 [==============================] - 2s 327ms/step - loss: 1.1821 - val_loss: 0.9099
Epoch 10/200
7/7 [==============================] - 2s 243ms/step - loss: 1.1883 - val_loss: 0.9080
Epoch 11

2/2 [==============================] - 2s 26ms/step
[[[ 0.0186276 ]
  [ 0.00383172]
  [-0.00920196]
  [-0.0157217 ]
  [-0.01953651]
  [-0.02409485]
  [-0.02540779]
  [-0.02177723]
  [-0.02222244]
  [-0.02699726]
  [-0.03120669]
  [-0.03132523]
  [-0.0256909 ]
  [-0.01396241]
  [-0.0051218 ]
  [ 0.00551252]
  [ 0.02720031]
  [ 0.05068629]
  [ 0.07006547]
  [ 0.0813115 ]]

 [[-0.0033737 ]
  [-0.01453011]
  [-0.02078923]
  [-0.0250652 ]
  [-0.03059921]
  [-0.03297361]
  [-0.03023651]
  [-0.03132076]
  [-0.03643585]
  [-0.04076651]
  [-0.04081488]
  [-0.03486653]
  [-0.02257513]
  [-0.01300937]
  [-0.00137908]
  [ 0.02182336]
  [ 0.0473621 ]
  [ 0.06978524]
  [ 0.08506286]
  [ 0.09210182]]

 [[-0.0192356 ]
  [-0.0258927 ]
  [-0.03046058]
  [-0.03643435]
  [-0.03919096]
  [-0.03682757]
  [-0.03821791]
  [-0.04345864]
  [-0.04784906]
  [-0.04790307]
  [-0.04185736]
  [-0.02938225]
  [-0.01961298]
  [-0.00760591]
  [ 0.01632147]
  [ 0.04293745]
  [ 0.06725158]
  [ 0.08542713]
  [ 0.09632792]


In [134]:
# Calculate evaluation metrics
msft_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
msft_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
msft_sentiment_rmse = np.sqrt(msft_sentiment_mse)
msft_sentiment_mape = np.mean(np.abs((sentiment_y_val -sentiment_y_pred) / sentiment_y_val)) * 100

print("MAE:", msft_sentiment_mae)
print("MSE:", msft_sentiment_mse)
print("RMSE:", msft_sentiment_rmse)
print("MAPE:", msft_sentiment_mape)

MAE: 0.7579837996137958
MSE: 0.9084873740771391
RMSE: 0.9531460402672505
MAPE: 111.73841941569285


## Stock & Sentiment

In [135]:
# Split data into training and validation sets
all_train_size = int(len(msft_scaled_lag) * 0.8)
all_train_data = msft_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = msft_scaled_lag.iloc[all_train_size:, 1:]
target=msft_scaled_df['msft_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [136]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [137]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [138]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model5 = Sequential()
model5.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model5.add(Dropout(0.05))
model5.add(Bidirectional(LSTM(128, return_sequences=True)))
model5.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model5.add(BatchNormalization())
model5.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model5.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model5.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model5.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 14s 506ms/step - loss: 1.7998 - val_loss: 0.9102
Epoch 2/200
7/7 [==============================] - 2s 324ms/step - loss: 1.3547 - val_loss: 0.9149
Epoch 3/200
7/7 [==============================] - 3s 394ms/step - loss: 1.2975 - val_loss: 0.9093
Epoch 4/200
7/7 [==============================] - 2s 231ms/step - loss: 1.2633 - val_loss: 0.9099
Epoch 5/200
7/7 [==============================] - 2s 235ms/step - loss: 1.2430 - val_loss: 0.9089
Epoch 6/200
7/7 [==============================] - 2s 233ms/step - loss: 1.2299 - val_loss: 0.9139
Epoch 7/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2091 - val_loss: 0.9116
Epoch 8/200
7/7 [==============================] - 2s 237ms/step - loss: 1.1975 - val_loss: 0.9117
Epoch 9/200
7/7 [==============================] - 2s 234ms/step - loss: 1.1854 - val_loss: 0.9126
Epoch 10/200
7/7 [==============================] - 3s 376ms/step - loss: 1.1890 - val_loss: 0.9118
Epoch 11

2/2 [==============================] - 2s 34ms/step
[[[-3.22508439e-02]
  [-2.95031872e-02]
  [-2.51753647e-02]
  [-2.55928691e-02]
  [-2.62258109e-02]
  [-2.01574434e-02]
  [-7.69531960e-03]
  [ 1.03162765e-03]
  [ 1.28236730e-02]
  [ 2.71573607e-02]
  [ 3.90381254e-02]
  [ 3.90286930e-02]
  [ 2.05530766e-02]
  [ 7.50485202e-03]
  [ 2.38102837e-03]
  [ 4.94951149e-03]
  [ 7.17215380e-03]
  [ 1.68499742e-02]
  [ 3.09769381e-02]
  [ 4.59910817e-02]]

 [[-1.98836233e-02]
  [-1.36452392e-02]
  [-1.40124690e-02]
  [-1.57582816e-02]
  [-1.15504432e-02]
  [-1.26596005e-03]
  [ 5.17668342e-03]
  [ 1.48168262e-02]
  [ 2.72288118e-02]
  [ 3.73232104e-02]
  [ 3.55660692e-02]
  [ 1.51399355e-02]
  [ 3.17247293e-04]
  [-6.45544240e-03]
  [-5.98506490e-03]
  [-5.88406483e-03]
  [ 1.84521242e-03]
  [ 1.43848900e-02]
  [ 2.86349524e-02]
  [ 4.10505235e-02]]

 [[-1.98433199e-03]
  [-9.20282968e-04]
  [-2.47984962e-03]
  [ 9.42183367e-04]
  [ 9.58680920e-03]
  [ 1.38633307e-02]
  [ 2.12602261e-02]
  [ 

In [139]:
# Calculate evaluation metrics
msft_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
msft_all_mse = np.mean(np.square(all_y_val - all_y_pred))
msft_all_rmse = np.sqrt(msft_all_mse)
msft_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", msft_all_mae)
print("MSE:", msft_all_mse)
print("RMSE:", msft_all_rmse)
print("MAPE:", msft_all_mape)

MAE: 0.7571917659568891
MSE: 0.9108792096099362
RMSE: 0.9543999212122433
MAPE: 103.3845228962714


## Stock & Opinion

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(msft_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = msft_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = msft_scaled_lag.iloc[all_train_size:, 7:]
target=msft_scaled_df['msft_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model5 = Sequential()
model5.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model5.add(Dropout(0.05))
model5.add(Bidirectional(LSTM(128, return_sequences=True)))
model5.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model5.add(BatchNormalization())
model5.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model5.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model5.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model5.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 21s 1s/step - loss: 1.3878 - val_loss: 0.9176
Epoch 2/200
7/7 [==============================] - 3s 463ms/step - loss: 1.2623 - val_loss: 0.9157
Epoch 3/200
7/7 [==============================] - 3s 478ms/step - loss: 1.2223 - val_loss: 0.9140
Epoch 4/200
7/7 [==============================] - 5s 736ms/step - loss: 1.2086 - val_loss: 0.9191
Epoch 5/200
7/7 [==============================] - 4s 581ms/step - loss: 1.2032 - val_loss: 0.9194
Epoch 6/200
7/7 [==============================] - 3s 431ms/step - loss: 1.2055 - val_loss: 0.9144
Epoch 7/200
7/7 [==============================] - 5s 756ms/step - loss: 1.2108 - val_loss: 0.9210
Epoch 8/200
7/7 [==============================] - 5s 723ms/step - loss: 1.2125 - val_loss: 0.9151
Epoch 9/200
7/7 [==============================] - 4s 594ms/step - loss: 1.1853 - val_loss: 0.9148
Epoch 10/200
7/7 [==============================] - 3s 478ms/step - loss: 1.1991 - val_loss: 0.9181
Epoch 11/20

In [ ]:
# Calculate evaluation metrics
msft_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
msft_all_mse = np.mean(np.square(all_y_val - all_y_pred))
msft_all_rmse = np.sqrt(msft_all_mse)
msft_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", msft_all_mae)
print("MSE:", msft_all_mse)
print("RMSE:", msft_all_rmse)
print("MAPE:", msft_all_mape)

MAE: 0.7570179594106852
MSE: 0.911708693237671
RMSE: 0.9548343800040251
MAPE: 101.16542992280995


## Stock & Comment

In [ ]:
comments=msft_scaled_lag.iloc[:, 1:7]

In [ ]:
market=msft_scaled_lag.iloc[:, 13:]

In [ ]:
stock_commnet= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets

all_train_size = int(len(stock_commnet) * 0.8)
all_train_data = stock_commnet.iloc[:all_train_size, :]
all_val_data = stock_commnet.iloc[all_train_size:, :]
target=msft_scaled_df['msft_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model2 = Sequential()
model2.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model2.add(Dropout(0.05))
model2.add(Bidirectional(LSTM(128, return_sequences=True)))
model2.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model2.add(BatchNormalization())
model2.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model2.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model2.predict(all_X_val)

Epoch 1/200
7/7 [==============================] - 19s 925ms/step - loss: 1.5009 - val_loss: 0.9149
Epoch 2/200
7/7 [==============================] - 2s 303ms/step - loss: 1.3212 - val_loss: 0.9055
Epoch 3/200
7/7 [==============================] - 2s 300ms/step - loss: 1.2651 - val_loss: 0.9102
Epoch 4/200
7/7 [==============================] - 2s 309ms/step - loss: 1.2333 - val_loss: 0.9125
Epoch 5/200
7/7 [==============================] - 3s 409ms/step - loss: 1.2176 - val_loss: 0.9100
Epoch 6/200
7/7 [==============================] - 3s 390ms/step - loss: 1.1955 - val_loss: 0.9122
Epoch 7/200
7/7 [==============================] - 3s 471ms/step - loss: 1.1981 - val_loss: 0.9091
Epoch 8/200
7/7 [==============================] - 2s 287ms/step - loss: 1.1928 - val_loss: 0.9103
Epoch 9/200
7/7 [==============================] - 2s 309ms/step - loss: 1.1905 - val_loss: 0.9095
Epoch 10/200
7/7 [==============================] - 2s 323ms/step - loss: 1.1860 - val_loss: 0.9081
Epoch 11

In [ ]:
# Calculate evaluation metrics
msft_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
msft_all_mse = np.mean(np.square(all_y_val - all_y_pred))
msft_all_rmse = np.sqrt(msft_all_mse)
msft_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", msft_all_mae)
print("MSE:", msft_all_mse)
print("RMSE:", msft_all_rmse)
print("MAPE:", msft_all_mape)

MAE: 0.7572916915946665
MSE: 0.9090891169023084
RMSE: 0.9534616494134981
MAPE: 105.13054362175693


# NIO

## Train Test Split

In [140]:
# Split data into training and validation sets
train_size = int(len(nio_scaled_lag) * 0.8)
train_data = nio_scaled_lag.iloc[:train_size, 1:]
val_data = nio_scaled_lag.iloc[train_size:, 1:]
target=nio_scaled_df['nio_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [141]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [142]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = nio_scaled_lag.filter(like='comment').columns.tolist() + nio_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_nio_scaled_lag = nio_scaled_lag.drop(columns=columns_to_drop)
stock_nio_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  nio_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028        -0.404393   
6   2022-01-10               -0.446869            -1.785457        -0.731131   
7   2022-01-11                0.070342            -1.772770        -0.564419   
8   2022-01-12                0.770026            -1.820702        -0.702570   
9   2022-01-13                0.164879            -1.850307        -0.176116   
..         ...                     ...                  ...              ...   
284 2023-02-17               -0.866740             1.135571        -0.818949   
285 2023-02-21               -0.250307             1.114425        -0.867497   
286 2023-02-22               -1.237982             1.293465        -0.967073   
287 2023-02-23                0.112495             1.248352        -1.006047   
288 2023-02-24                0.418239             1.186323        -0.341100   

     nio_percentage_lag1  
5              -0.287887  
6               0.040533  
7              -0.186815  
8               0.719565  
9               1.015841  
..                   ...  
284            -0.471028  
285             0.046514  
286            -0.228731  
287             0.308672  
288            -0.625054  

[284 rows x 5 columns]

In [143]:
# Split data into training and validation sets
stock_train_size = int(len(stock_nio_scaled_lag) * 0.8)
stock_train_data = stock_nio_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_nio_scaled_lag.iloc[stock_train_size:, 1:]
target=nio_scaled_df['nio_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [144]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [145]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [146]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model6 = Sequential()
model6.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model6.add(Dropout(0.05))
model6.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.05))

model6.add(BatchNormalization())
model6.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model6.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model6.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 528ms/step - loss: 1.4946 - val_loss: 0.4110
Epoch 2/200
7/7 [==============================] - 2s 242ms/step - loss: 1.2776 - val_loss: 0.4122
Epoch 3/200
7/7 [==============================] - 3s 395ms/step - loss: 1.2652 - val_loss: 0.4131
Epoch 4/200
7/7 [==============================] - 2s 316ms/step - loss: 1.2289 - val_loss: 0.4120
Epoch 5/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1990 - val_loss: 0.4126
Epoch 6/200
7/7 [==============================] - 2s 239ms/step - loss: 1.1947 - val_loss: 0.4121
Epoch 7/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1850 - val_loss: 0.4113
Epoch 8/200
7/7 [==============================] - 2s 244ms/step - loss: 1.1794 - val_loss: 0.4112
Epoch 9/200
7/7 [==============================] - 2s 240ms/step - loss: 1.1785 - val_loss: 0.4111
Epoch 10/200
7/7 [==============================] - 2s 308ms/step - loss: 1.1786 - val_loss: 0.4109
Epoch 11

In [147]:
# Calculate evaluation metrics
nio_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
nio_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
nio_stock_rmse = np.sqrt(nio_stock_mse)
nio_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", nio_stock_mae)
print("MSE:", nio_stock_mse)
print("RMSE:", nio_stock_rmse)
print("MAPE:", nio_stock_mape)

MAE: 0.48943605778827887
MSE: 0.41035511777784345
RMSE: 0.6405896641203661
MAPE: 95.70046145252877


## Only Sentiment

In [148]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = nio_scaled_lag.filter(like='comment').columns.tolist() + nio_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_nio_scaled_lag = nio_scaled_lag[columns_to_drop]
sentiment_nio_scaled_lag 

comments_NIO_lag1  comments_NIO_lag3  comments_NIO_lag5  \
5             0.612377           1.523414          -1.110573   
6             0.305042           0.568701           1.633749   
7             1.084771           0.612377           1.523414   
8            -0.457936           0.305042           0.568701   
9             0.500650           1.084771           0.612377   
..                 ...                ...                ...   
284           0.625543           0.404229           0.067773   
285           0.404229          -1.091133           0.067773   
286           1.002374           0.625543           0.404229   
287          -0.343452           0.404229          -1.091133   
288           0.221463           1.002374           0.625543   

     comments_NIO_count_lag1  comments_NIO_count_lag3  \
5                   3.843096                 1.318881   
6                   2.798593                 1.754090   
7                  -0.160832                 3.843096   
8                   1.231839                 2.798593   
9                   1.405923                -0.160832   
..                       ...                      ...   
284                -0.334916                -0.508999   
285                -0.508999                -0.508999   
286                -0.683083                -0.334916   
287                -0.683083                -0.508999   
288                 0.013252                -0.683083   

     comments_NIO_count_lag5  opinions_NIO_lag1  opinions_NIO_lag3  \
5                   0.100294          -0.156886          -0.156886   
6                   0.013252          -0.156886          -0.156886   
7                   1.318881          -0.156886          -0.156886   
8                   1.754090          -1.350041          -0.156886   
9                   3.843096          -1.350041          -0.156886   
..                       ...                ...                ...   
284                -0.247874          -1.350041          -1.350041   
285                -0.247874          -1.350041          -1.350041   
286                -0.508999          -1.350041          -1.350041   
287                -0.508999          -1.350041          -1.350041   
288                -0.334916          -1.350041          -1.350041   

     opinions_NIO_lag5  opinions_NIO_count_lag1  opinions_NIO_count_lag3  \
5            -0.156886                -6.347103                -6.347103   
6            -0.156886                -6.347103                -6.347103   
7            -0.156886                -6.347103                -6.347103   
8            -0.156886                 0.157552                -6.347103   
9            -0.156886                 0.157552                -6.347103   
..                 ...                      ...                      ...   
284          -1.350041                 0.157552                 0.157552   
285          -1.350041                 0.157552                 0.157552   
286          -1.350041                 0.157552                 0.157552   
287          -1.350041                 0.157552                 0.157552   
288          -1.350041                 0.157552                 0.157552   

     opinions_NIO_count_lag5  
5                  -6.347103  
6                  -6.347103  
7                  -6.347103  
8                  -6.347103  
9                  -6.347103  
..                       ...  
284                 0.157552  
285                 0.157552  
286                 0.157552  
287                 0.157552  
288                 0.157552  

[284 rows x 12 columns]

In [149]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_nio_scaled_lag) * 0.8)
sentiment_train_data = sentiment_nio_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_nio_scaled_lag.iloc[sentiment_train_size:, 1:]
target=nio_scaled_df['nio_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [150]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [151]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)

In [152]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model7 = Sequential()
model7.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model7.add(Dropout(0.05))
model7.add(Bidirectional(LSTM(128, return_sequences=True)))
model7.add(Dropout(0.05))

model7.add(BatchNormalization())
model7.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model7.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model7.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model7.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 625ms/step - loss: 1.6194 - val_loss: 0.4095
Epoch 2/200
7/7 [==============================] - 2s 235ms/step - loss: 1.3613 - val_loss: 0.4093
Epoch 3/200
7/7 [==============================] - 2s 233ms/step - loss: 1.3309 - val_loss: 0.4100
Epoch 4/200
7/7 [==============================] - 2s 234ms/step - loss: 1.3101 - val_loss: 0.4103
Epoch 5/200
7/7 [==============================] - 2s 243ms/step - loss: 1.2884 - val_loss: 0.4098
Epoch 6/200
7/7 [==============================] - 2s 264ms/step - loss: 1.2852 - val_loss: 0.4100
Epoch 7/200
7/7 [==============================] - 3s 383ms/step - loss: 1.2784 - val_loss: 0.4100
Epoch 8/200
7/7 [==============================] - 2s 281ms/step - loss: 1.2773 - val_loss: 0.4097
Epoch 9/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2627 - val_loss: 0.4107
Epoch 10/200
7/7 [==============================] - 2s 228ms/step - loss: 1.2645 - val_loss: 0.4104
Epoch 11

In [153]:
# Calculate evaluation metrics
nio_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
nio_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
nio_sentiment_rmse = np.sqrt(nio_sentiment_mse)
nio_sentiment_mape = np.mean(np.abs((sentiment_y_val - sentiment_y_pred) / sentiment_y_val)) * 100

print("MAE:", nio_sentiment_mae)
print("MSE:", nio_sentiment_mse)
print("RMSE:", nio_sentiment_rmse)
print("MAPE:", nio_sentiment_mape)

MAE: 0.4903408914780837
MSE: 0.4106355770839432
RMSE: 0.6408085338725938
MAPE: 97.09742919289756


## Stock & Sentiment

In [154]:
# Split data into training and validation sets
all_train_size = int(len(nio_scaled_lag) * 0.8)
all_train_data = nio_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = nio_scaled_lag.iloc[all_train_size:, 1:]
target=nio_scaled_df['nio_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [155]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [156]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [157]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model8 = Sequential()
model8.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model8.add(Dropout(0.05))
model8.add(Bidirectional(LSTM(128, return_sequences=True)))
model8.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model8.add(BatchNormalization())
model8.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model8.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model8.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model8.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 500ms/step - loss: 1.5358 - val_loss: 0.4117
Epoch 2/200
7/7 [==============================] - 2s 229ms/step - loss: 1.3397 - val_loss: 0.4115
Epoch 3/200
7/7 [==============================] - 3s 380ms/step - loss: 1.3227 - val_loss: 0.4113
Epoch 4/200
7/7 [==============================] - 2s 305ms/step - loss: 1.3015 - val_loss: 0.4120
Epoch 5/200
7/7 [==============================] - 2s 237ms/step - loss: 1.2932 - val_loss: 0.4109
Epoch 6/200
7/7 [==============================] - 2s 232ms/step - loss: 1.2797 - val_loss: 0.4115
Epoch 7/200
7/7 [==============================] - 2s 243ms/step - loss: 1.2793 - val_loss: 0.4113
Epoch 8/200
7/7 [==============================] - 2s 238ms/step - loss: 1.2828 - val_loss: 0.4112
Epoch 9/200
7/7 [==============================] - 2s 229ms/step - loss: 1.2967 - val_loss: 0.4131
Epoch 10/200
7/7 [==============================] - 2s 288ms/step - loss: 1.2822 - val_loss: 0.4117
Epoch 11

In [158]:
# Calculate evaluation metrics
nio_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
nio_all_mse = np.mean(np.square(all_y_val - all_y_pred))
nio_all_rmse = np.sqrt(nio_all_mse)
nio_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", nio_all_mae)
print("MSE:", nio_all_mse)
print("RMSE:", nio_all_rmse)
print("MAPE:", nio_all_mape)

MAE: 0.4903791011778557
MSE: 0.4106375483045899
RMSE: 0.640810071943778
MAPE: 97.25716293710585


## Stock & Opinion

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(nio_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = nio_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = nio_scaled_lag.iloc[all_train_size:, 7:]
target=nio_scaled_df['nio_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model8 = Sequential()
model8.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model8.add(Dropout(0.05))
model8.add(Bidirectional(LSTM(128, return_sequences=True)))
model8.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model8.add(BatchNormalization())
model8.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model8.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model8.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model8.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 26s 2s/step - loss: 1.8114 - val_loss: 0.4202
Epoch 2/200
7/7 [==============================] - 4s 609ms/step - loss: 1.5184 - val_loss: 0.4118
Epoch 3/200
7/7 [==============================] - 6s 874ms/step - loss: 1.4185 - val_loss: 0.4135
Epoch 4/200
7/7 [==============================] - 4s 623ms/step - loss: 1.3295 - val_loss: 0.4130
Epoch 5/200
7/7 [==============================] - 4s 587ms/step - loss: 1.3161 - val_loss: 0.4137
Epoch 6/200
7/7 [==============================] - 5s 690ms/step - loss: 1.3001 - val_loss: 0.4132
Epoch 7/200
7/7 [==============================] - 5s 604ms/step - loss: 1.2940 - val_loss: 0.4125
Epoch 8/200
7/7 [==============================] - 5s 671ms/step - loss: 1.2991 - val_loss: 0.4137
Epoch 9/200
7/7 [==============================] - 3s 461ms/step - loss: 1.2956 - val_loss: 0.4126
Epoch 10/200
7/7 [==============================] - 5s 688ms/step - loss: 1.2972 - val_loss: 0.4147
Epoch 11/20

2/2 [==============================] - 3s 90ms/step
[[[-9.32438765e-03]
  [-1.25157209e-02]
  [-1.23203499e-02]
  [-9.24470462e-03]
  [-7.12434622e-03]
  [-6.89294981e-03]
  [-6.79975795e-03]
  [-5.37215592e-03]
  [-4.90928581e-03]
  [-7.66693288e-03]
  [-1.23056248e-02]
  [-1.63040720e-02]
  [-1.62968840e-02]
  [-1.55139035e-02]
  [-1.94221567e-02]
  [-2.51392070e-02]
  [-2.80538909e-02]
  [-2.39255615e-02]
  [-1.45198293e-02]
  [-7.70657370e-03]]

 [[-1.35637419e-02]
  [-1.57220215e-02]
  [-1.37054641e-02]
  [-1.18043283e-02]
  [-1.14117051e-02]
  [-1.10349292e-02]
  [-9.25762858e-03]
  [-8.45268089e-03]
  [-1.10062445e-02]
  [-1.55292200e-02]
  [-1.94867570e-02]
  [-1.95695627e-02]
  [-1.89726260e-02]
  [-2.32005678e-02]
  [-2.93453895e-02]
  [-3.27415317e-02]
  [-2.89157461e-02]
  [-1.95332859e-02]
  [-1.19425328e-02]
  [-8.02765880e-03]]

 [[-1.20992074e-02]
  [-1.19538205e-02]
  [-1.04961302e-02]
  [-9.86111071e-03]
  [-9.30497143e-03]
  [-7.21797859e-03]
  [-6.34158915e-03]
  [-

In [ ]:
# Calculate evaluation metrics
nio_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
nio_all_mse = np.mean(np.square(all_y_val - all_y_pred))
nio_all_rmse = np.sqrt(nio_all_mse)
nio_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", nio_all_mae)
print("MSE:", nio_all_mse)
print("RMSE:", nio_all_rmse)
print("MAPE:", nio_all_mape)

MAE: 0.49002020103576954
MSE: 0.41198320104348113
RMSE: 0.641859175398686
MAPE: 101.3945171498805


## Stock & Comment

In [ ]:
comments=nio_scaled_lag.iloc[:, 1:7]

In [ ]:
market=nio_scaled_lag.iloc[:, 13:]

In [ ]:
stock_comment= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(stock_comment) * 0.8)
all_train_data = stock_comment.iloc[:all_train_size, :]
all_val_data = stock_comment.iloc[all_train_size:, :]
target=nio_scaled_df['nio_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model8 = Sequential()
model8.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model8.add(Dropout(0.05))
model8.add(Bidirectional(LSTM(128, return_sequences=True)))
model8.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model8.add(BatchNormalization())
model8.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model8.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model8.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model8.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 18s 651ms/step - loss: 2.0385 - val_loss: 0.4156
Epoch 2/200
7/7 [==============================] - 2s 332ms/step - loss: 1.5167 - val_loss: 0.4123
Epoch 3/200
7/7 [==============================] - 2s 326ms/step - loss: 1.3978 - val_loss: 0.4149
Epoch 4/200
7/7 [==============================] - 3s 496ms/step - loss: 1.3255 - val_loss: 0.4118
Epoch 5/200
7/7 [==============================] - 2s 322ms/step - loss: 1.3000 - val_loss: 0.4150
Epoch 6/200
7/7 [==============================] - 2s 311ms/step - loss: 1.2977 - val_loss: 0.4136
Epoch 7/200
7/7 [==============================] - 2s 309ms/step - loss: 1.2814 - val_loss: 0.4121
Epoch 8/200
7/7 [==============================] - 2s 285ms/step - loss: 1.2691 - val_loss: 0.4139
Epoch 9/200
7/7 [==============================] - 2s 350ms/step - loss: 1.2757 - val_loss: 0.4124
Epoch 10/200
7/7 [==============================] - 4s 509ms/step - loss: 1.2595 - val_loss: 0.4134
Epoch 11

2/2 [==============================] - 2s 48ms/step
[[[-1.35008907e-02]
  [-2.31160782e-03]
  [ 7.65814632e-03]
  [-1.60361722e-03]
  [-6.10038359e-03]
  [-1.53222578e-02]
  [-1.17675858e-02]
  [-1.52654189e-03]
  [ 6.76635373e-03]
  [-3.08518429e-05]
  [-1.42064681e-02]
  [-2.74247304e-02]
  [-2.42769755e-02]
  [-2.08996218e-02]
  [-1.95653774e-02]
  [-1.89318825e-02]
  [-1.51536465e-02]
  [-6.60105329e-03]
  [-2.03844439e-03]
  [ 3.15956492e-03]]

 [[-2.80122552e-03]
  [ 8.96168221e-03]
  [ 1.71126158e-03]
  [-1.34836964e-03]
  [-9.45188757e-03]
  [-5.08070271e-03]
  [ 5.38569968e-03]
  [ 1.34396600e-02]
  [ 6.14823308e-03]
  [-8.70706979e-03]
  [-2.28950158e-02]
  [-2.09862869e-02]
  [-1.89440046e-02]
  [-1.90832447e-02]
  [-1.99456699e-02]
  [-1.78958382e-02]
  [-1.06027769e-02]
  [-6.05914649e-03]
  [ 1.00698613e-03]
  [ 1.05860224e-02]]

 [[ 1.02201635e-02]
  [ 4.06424422e-03]
  [ 8.76751787e-04]
  [-6.91091362e-03]
  [-2.36028060e-03]
  [ 8.56984966e-03]
  [ 1.71510447e-02]
  [ 

In [ ]:
# Calculate evaluation metrics
nio_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
nio_all_mse = np.mean(np.square(all_y_val - all_y_pred))
nio_all_rmse = np.sqrt(nio_all_mse)
nio_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", nio_all_mae)
print("MSE:", nio_all_mse)
print("RMSE:", nio_all_rmse)
print("MAPE:", nio_all_mape)

MAE: 0.4901750015692269
MSE: 0.4123932065570166
RMSE: 0.6421784849689505
MAPE: 102.8308295316549


# META

## Train Test Split

In [159]:
# Split data into training and validation sets
train_size = int(len(meta_scaled_lag) * 0.8)
train_data = meta_scaled_lag.iloc[:train_size, 1:]
val_data = meta_scaled_lag.iloc[train_size:, 1:]
target=meta_scaled_df['meta_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [160]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [161]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = meta_scaled_lag.filter(like='comment').columns.tolist() + meta_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_meta_scaled_lag = meta_scaled_lag.drop(columns=columns_to_drop)
stock_meta_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  meta_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028         -0.309505   
6   2022-01-10               -0.446869            -1.785457         -0.870154   
7   2022-01-11                0.070342            -1.772770         -0.437396   
8   2022-01-12                0.770026            -1.820702         -0.806437   
9   2022-01-13                0.164879            -1.850307         -0.896284   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.399919   
285 2023-02-21               -0.250307             1.114425         -0.470047   
286 2023-02-22               -1.237982             1.293465         -0.028782   
287 2023-02-23                0.112495             1.248352         -0.543642   
288 2023-02-24                0.418239             1.186323         -0.645917   

     meta_percentage_lag1  
5                0.660912  
6               -0.012815  
7               -0.237406  
8                0.505357  
9               -0.044670  
..                    ...  
284             -0.614237  
285              0.098712  
286             -0.076609  
287             -0.099842  
288              0.167693  

[284 rows x 5 columns]

In [162]:
# Split data into training and validation sets
stock_train_size = int(len(stock_meta_scaled_lag) * 0.8)
stock_train_data = stock_meta_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_meta_scaled_lag.iloc[stock_train_size:, 1:]
target=meta_scaled_df['meta_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [163]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [164]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [165]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model9 = Sequential()
model9.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model9.add(Dropout(0.05))
model9.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.05))

model9.add(BatchNormalization())
model9.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model9.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model9.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model9.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 14s 502ms/step - loss: 1.2643 - val_loss: 1.1598
Epoch 2/200
7/7 [==============================] - 2s 227ms/step - loss: 1.0249 - val_loss: 1.1633
Epoch 3/200
7/7 [==============================] - 2s 342ms/step - loss: 0.9556 - val_loss: 1.1541
Epoch 4/200
7/7 [==============================] - 3s 360ms/step - loss: 0.9551 - val_loss: 1.1624
Epoch 5/200
7/7 [==============================] - 2s 228ms/step - loss: 0.9372 - val_loss: 1.1601
Epoch 6/200
7/7 [==============================] - 2s 230ms/step - loss: 0.9415 - val_loss: 1.1586
Epoch 7/200
7/7 [==============================] - 2s 233ms/step - loss: 0.9318 - val_loss: 1.1655
Epoch 8/200
7/7 [==============================] - 2s 229ms/step - loss: 0.9335 - val_loss: 1.1631
Epoch 9/200
7/7 [==============================] - 2s 236ms/step - loss: 0.9314 - val_loss: 1.1580
Epoch 10/200
7/7 [==============================] - 2s 229ms/step - loss: 0.9272 - val_loss: 1.1647
Epoch 11

In [166]:
# Calculate evaluation metrics
meta_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
meta_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
meta_stock_rmse = np.sqrt(meta_stock_mse)
meta_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", meta_stock_mae)
print("MSE:", meta_stock_mse)
print("RMSE:", meta_stock_rmse)
print("MAPE:", meta_stock_mape)

MAE: 0.5789445788475661
MSE: 1.153456581411588
RMSE: 1.0739909596507728
MAPE: 104.48904680543521


## Only Sentiment

In [167]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = meta_scaled_lag.filter(like='comment').columns.tolist() + meta_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_meta_scaled_lag = meta_scaled_lag[columns_to_drop]
sentiment_meta_scaled_lag 

comments_META_lag1  comments_META_lag3  comments_META_lag5  \
5             -0.789824            0.061247            0.061247   
6              0.061247           -0.789824            0.061247   
7              0.061247           -0.789824            0.061247   
8              1.763389            0.061247           -0.789824   
9              1.763389            0.061247           -0.789824   
..                  ...                 ...                 ...   
284            1.196008            0.061247            0.061247   
285            1.196008            1.196008            0.061247   
286           -1.073514            1.196008            0.061247   
287           -1.073514            1.196008            1.196008   
288           -1.073514           -1.073514            1.196008   

     comments_META_count_lag1  comments_META_count_lag3  \
5                   -0.220699                 -0.295560   
6                   -0.295560                 -0.220699   
7                   -0.295560                 -0.220699   
8                   -0.220699                 -0.295560   
9                   -0.220699                 -0.295560   
..                        ...                       ...   
284                 -0.295560                 -0.295560   
285                 -0.295560                 -0.295560   
286                 -0.295560                 -0.295560   
287                 -0.295560                 -0.295560   
288                 -0.295560                 -0.295560   

     comments_META_count_lag5  opinions_META_lag1  opinions_META_lag3  \
5                   -0.370421            1.982918            1.982918   
6                   -0.370421            1.982918            1.982918   
7                   -0.295560            0.137701            1.982918   
8                   -0.220699            0.137701            1.982918   
9                   -0.220699            0.137701            0.137701   
..                        ...                 ...                 ...   
284                 -0.295560            1.367846            1.060309   
285                 -0.295560            1.367846            1.367846   
286                 -0.295560            1.367846            1.367846   
287                 -0.295560            1.367846            1.367846   
288                 -0.295560            1.367846            1.367846   

     opinions_META_lag5  opinions_META_count_lag1  opinions_META_count_lag3  \
5              0.137701                  0.305313                  0.305313   
6              0.137701                  0.305313                  0.305313   
7              1.982918                 -0.417929                  0.305313   
8              1.982918                 -0.417929                  0.305313   
9              1.982918                 -0.417929                 -0.417929   
..                  ...                       ...                       ...   
284            1.982918                 -0.417929                  0.305313   
285            1.982918                 -0.417929                 -0.417929   
286            1.060309                 -0.417929                 -0.417929   
287            1.367846                 -0.417929                 -0.417929   
288            1.367846                 -0.417929                 -0.417929   

     opinions_META_count_lag5  
5                   -1.141170  
6                   -1.141170  
7                    0.305313  
8                    0.305313  
9                    0.305313  
..                        ...  
284                  0.305313  
285                  0.305313  
286                  0.305313  
287                 -0.417929  
288                 -0.417929  

[284 rows x 12 columns]

In [168]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_meta_scaled_lag) * 0.8)
sentiment_train_data = sentiment_meta_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_meta_scaled_lag.iloc[sentiment_train_size:, 1:]
target=meta_scaled_df['meta_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [169]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [170]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)

In [171]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model10 = Sequential()
model10.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model10.add(Dropout(0.05))
model10.add(Bidirectional(LSTM(128, return_sequences=True)))
model10.add(Dropout(0.05))

model10.add(BatchNormalization())
model10.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model10.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model10.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model10.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 510ms/step - loss: 1.5169 - val_loss: 1.1749
Epoch 2/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1285 - val_loss: 1.1651
Epoch 3/200
7/7 [==============================] - 2s 241ms/step - loss: 1.0923 - val_loss: 1.1704
Epoch 4/200
7/7 [==============================] - 2s 249ms/step - loss: 1.0522 - val_loss: 1.1623
Epoch 5/200
7/7 [==============================] - 3s 387ms/step - loss: 1.0368 - val_loss: 1.1609
Epoch 6/200
7/7 [==============================] - 2s 347ms/step - loss: 1.0320 - val_loss: 1.1619
Epoch 7/200
7/7 [==============================] - 2s 238ms/step - loss: 1.0225 - val_loss: 1.1607
Epoch 8/200
7/7 [==============================] - 2s 246ms/step - loss: 1.0306 - val_loss: 1.1581
Epoch 9/200
7/7 [==============================] - 2s 243ms/step - loss: 1.0214 - val_loss: 1.1651
Epoch 10/200
7/7 [==============================] - 2s 226ms/step - loss: 1.0207 - val_loss: 1.1624
Epoch 11

In [172]:
# Calculate evaluation metrics
meta_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
meta_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
meta_sentiment_rmse = np.sqrt(meta_sentiment_mse)
meta_sentiment_mape = np.mean(np.abs((sentiment_y_val - sentiment_y_pred) / sentiment_y_val)) * 100

print("MAE:", meta_sentiment_mae)
print("MSE:", meta_sentiment_mse)
print("RMSE:", meta_sentiment_rmse)
print("MAPE:", meta_sentiment_mape)

MAE: 0.5902648638282262
MSE: 1.1506647257224967
RMSE: 1.0726904146688814
MAPE: 155.9832902478312


## Stock & Sentiment

In [173]:
# Split data into training and validation sets
all_train_size = int(len(meta_scaled_lag) * 0.8)
all_train_data = meta_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = meta_scaled_lag.iloc[all_train_size:, 1:]
target=meta_scaled_df['meta_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target = target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [174]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [175]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [176]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model11 = Sequential()
model11.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model11.add(Dropout(0.05))
model11.add(Bidirectional(LSTM(128, return_sequences=True)))
model11.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model11.add(BatchNormalization())
model11.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model11.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model11.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model11.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 14s 583ms/step - loss: 1.9407 - val_loss: 1.1811
Epoch 2/200
7/7 [==============================] - 3s 374ms/step - loss: 1.3041 - val_loss: 1.1664
Epoch 3/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1734 - val_loss: 1.1778
Epoch 4/200
7/7 [==============================] - 2s 225ms/step - loss: 1.0879 - val_loss: 1.1716
Epoch 5/200
7/7 [==============================] - 2s 227ms/step - loss: 1.0679 - val_loss: 1.1764
Epoch 6/200
7/7 [==============================] - 2s 231ms/step - loss: 1.0396 - val_loss: 1.1735
Epoch 7/200
7/7 [==============================] - 2s 235ms/step - loss: 1.0273 - val_loss: 1.1751
Epoch 8/200
7/7 [==============================] - 2s 231ms/step - loss: 1.0304 - val_loss: 1.1747
Epoch 9/200
7/7 [==============================] - 2s 375ms/step - loss: 1.0333 - val_loss: 1.1780
Epoch 10/200
7/7 [==============================] - 2s 347ms/step - loss: 1.0263 - val_loss: 1.1758
Epoch 11

In [177]:
# Calculate evaluation metrics
meta_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
meta_all_mse = np.mean(np.square(all_y_val - all_y_pred))
meta_all_rmse = np.sqrt(meta_all_mse)
meta_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", meta_all_mae)
print("MSE:", meta_all_mse)
print("RMSE:", meta_all_rmse)
print("MAPE:", meta_all_mape)

MAE: 0.5853637502621034
MSE: 1.1741011453064467
RMSE: 1.0835594793579384
MAPE: 115.46329768168792


## Stock & Opinion

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(meta_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = meta_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = meta_scaled_lag.iloc[all_train_size:, 7:]
target=meta_scaled_df['meta_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target = target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model11 = Sequential()
model11.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model11.add(Dropout(0.05))
model11.add(Bidirectional(LSTM(128, return_sequences=True)))
model11.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model11.add(BatchNormalization())
model11.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model11.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model11.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model11.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 16s 881ms/step - loss: 1.3590 - val_loss: 1.1582
Epoch 2/200
7/7 [==============================] - 2s 364ms/step - loss: 1.1915 - val_loss: 1.1701
Epoch 3/200
7/7 [==============================] - 2s 325ms/step - loss: 1.0883 - val_loss: 1.1653
Epoch 4/200
7/7 [==============================] - 2s 281ms/step - loss: 1.0658 - val_loss: 1.1645
Epoch 5/200
7/7 [==============================] - 3s 379ms/step - loss: 1.0497 - val_loss: 1.1698
Epoch 6/200
7/7 [==============================] - 5s 700ms/step - loss: 1.0258 - val_loss: 1.1645
Epoch 7/200
7/7 [==============================] - 4s 608ms/step - loss: 1.0162 - val_loss: 1.1690
Epoch 8/200
7/7 [==============================] - 4s 541ms/step - loss: 1.0137 - val_loss: 1.1662
Epoch 9/200
7/7 [==============================] - 6s 954ms/step - loss: 1.0074 - val_loss: 1.1684
Epoch 10/200
7/7 [==============================] - 5s 636ms/step - loss: 1.0103 - val_loss: 1.1642
Epoch 11

In [ ]:
# Calculate evaluation metrics
meta_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
meta_all_mse = np.mean(np.square(all_y_val - all_y_pred))
meta_all_rmse = np.sqrt(meta_all_mse)
meta_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", meta_all_mae)
print("MSE:", meta_all_mse)
print("RMSE:", meta_all_rmse)
print("MAPE:", meta_all_mape)

MAE: 0.5805840698991293
MSE: 1.1619908644636194
RMSE: 1.0779568008336973
MAPE: 104.64458313007452


## Stock & Comment

In [ ]:
comments=meta_scaled_lag.iloc[:, 1:7]

In [ ]:
market=meta_scaled_lag.iloc[:, 13:]

In [ ]:
stock_comment= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(stock_comment) * 0.8)
all_train_data = stock_comment.iloc[:all_train_size, :]
all_val_data = stock_comment.iloc[all_train_size:, :]
target=meta_scaled_df['meta_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target = target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model8 = Sequential()
model8.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model8.add(Dropout(0.05))
model8.add(Bidirectional(LSTM(128, return_sequences=True)))
model8.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model8.add(BatchNormalization())
model8.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model8.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model8.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model8.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 15s 668ms/step - loss: 1.1396 - val_loss: 1.1700
Epoch 2/200
7/7 [==============================] - 5s 610ms/step - loss: 1.0475 - val_loss: 1.1655
Epoch 3/200
7/7 [==============================] - 3s 335ms/step - loss: 1.0438 - val_loss: 1.1769
Epoch 4/200
7/7 [==============================] - 2s 316ms/step - loss: 1.0119 - val_loss: 1.1803
Epoch 5/200
7/7 [==============================] - 2s 293ms/step - loss: 1.0018 - val_loss: 1.1783
Epoch 6/200
7/7 [==============================] - 2s 304ms/step - loss: 0.9944 - val_loss: 1.1775
Epoch 7/200
7/7 [==============================] - 3s 369ms/step - loss: 0.9879 - val_loss: 1.1752
Epoch 8/200
7/7 [==============================] - 4s 572ms/step - loss: 0.9968 - val_loss: 1.1745
Epoch 9/200
7/7 [==============================] - 2s 332ms/step - loss: 0.9879 - val_loss: 1.1792
Epoch 10/200
7/7 [==============================] - 2s 318ms/step - loss: 0.9922 - val_loss: 1.1759
Epoch 11

In [ ]:
# Calculate evaluation metrics
meta_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
meta_all_mse = np.mean(np.square(all_y_val - all_y_pred))
meta_all_rmse = np.sqrt(meta_all_mse)
meta_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", meta_all_mae)
print("MSE:", meta_all_mse)
print("RMSE:", meta_all_rmse)
print("MAPE:", meta_all_mape)

MAE: 0.5811998335109376
MSE: 1.167334782668506
RMSE: 1.0804326830804898
MAPE: 101.78867468103745


# AMZN

## Train Test Split

In [178]:
# Split data into training and validation sets
train_size = int(len(amzn_scaled_lag) * 0.8)
train_data = amzn_scaled_lag.iloc[:train_size, 1:]
val_data = amzn_scaled_lag.iloc[train_size:, 1:]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [179]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [180]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = amzn_scaled_lag.filter(like='comment').columns.tolist() + amzn_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_amzn_scaled_lag = amzn_scaled_lag.drop(columns=columns_to_drop)
stock_amzn_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  amzn_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028         -0.730973   
6   2022-01-10               -0.446869            -1.785457         -0.894246   
7   2022-01-11                0.070342            -1.772770          0.362395   
8   2022-01-12                0.770026            -1.820702         -0.400034   
9   2022-01-13                0.164879            -1.850307         -0.789790   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.597316   
285 2023-02-21               -0.250307             1.114425         -0.484739   
286 2023-02-22               -1.237982             1.293465         -0.589958   
287 2023-02-23                0.112495             1.248352         -0.499849   
288 2023-02-24                0.418239             1.186323         -0.837473   

     amzn_percentage_lag1  
5               -0.166781  
6               -0.088896  
7               -0.162255  
8                0.820375  
9                0.018791  
..                    ...  
284             -0.907439  
285             -0.262180  
286             -0.817438  
287              0.460113  
288              0.058984  

[284 rows x 5 columns]

In [181]:
# Split data into training and validation sets
stock_train_size = int(len(stock_amzn_scaled_lag) * 0.8)
stock_train_data = stock_amzn_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_amzn_scaled_lag.iloc[stock_train_size:, 1:]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [182]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [183]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [184]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model6 = Sequential()
model6.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model6.add(Dropout(0.05))
model6.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.05))

model6.add(BatchNormalization())
model6.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model6.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model6.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 508ms/step - loss: 1.3825 - val_loss: 0.8383
Epoch 2/200
7/7 [==============================] - 2s 227ms/step - loss: 1.1526 - val_loss: 0.8311
Epoch 3/200
7/7 [==============================] - 2s 224ms/step - loss: 1.0932 - val_loss: 0.8345
Epoch 4/200
7/7 [==============================] - 2s 235ms/step - loss: 1.0750 - val_loss: 0.8357
Epoch 5/200
7/7 [==============================] - 2s 350ms/step - loss: 1.0565 - val_loss: 0.8358
Epoch 6/200
7/7 [==============================] - 3s 357ms/step - loss: 1.0433 - val_loss: 0.8344
Epoch 7/200
7/7 [==============================] - 2s 231ms/step - loss: 1.0392 - val_loss: 0.8353
Epoch 8/200
7/7 [==============================] - 2s 229ms/step - loss: 1.0389 - val_loss: 0.8345
Epoch 9/200
7/7 [==============================] - 2s 229ms/step - loss: 1.0381 - val_loss: 0.8344
Epoch 10/200
7/7 [==============================] - 2s 229ms/step - loss: 1.0364 - val_loss: 0.8364
Epoch 11

In [185]:
# Calculate evaluation metrics
amzn_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
amzn_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
amzn_stock_rmse = np.sqrt(amzn_stock_mse)
amzn_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", amzn_stock_mae)
print("MSE:", amzn_stock_mse)
print("RMSE:", amzn_stock_rmse)
print("MAPE:", amzn_stock_mape)

MAE: 0.6980036430744797
MSE: 0.831720287584124
RMSE: 0.9119869996793397
MAPE: 98.69676652973683


## Only Sentiment

In [186]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = amzn_scaled_lag.filter(like='comment').columns.tolist() + amzn_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_amzn_scaled_lag = amzn_scaled_lag[columns_to_drop]
sentiment_amzn_scaled_lag 

comments_AMZN_lag1  comments_AMZN_lag3  comments_AMZN_lag5  \
5             -0.855572            1.102543           -0.156245   
6              0.787846           -0.855572           -0.156245   
7              0.787846           -0.855572            1.102543   
8              0.543081            0.787846           -0.855572   
9              0.945194            0.787846           -0.855572   
..                  ...                 ...                 ...   
284            1.731937            1.731937           -1.362584   
285            1.731937            1.731937           -1.362584   
286            1.731937            1.731937            1.731937   
287            1.731937            1.731937            1.731937   
288            1.731937            1.731937            1.731937   

     comments_AMZN_count_lag1  comments_AMZN_count_lag3  \
5                   -0.197330                  -0.31289   
6                   -0.255110                  -0.19733   
7                   -0.255110                  -0.19733   
8                   -0.197330                  -0.25511   
9                   -0.139551                  -0.25511   
..                        ...                       ...   
284                 -0.255110                  -0.25511   
285                 -0.255110                  -0.25511   
286                 -0.255110                  -0.25511   
287                 -0.255110                  -0.25511   
288                 -0.255110                  -0.25511   

     comments_AMZN_count_lag5  opinions_AMZN_lag1  opinions_AMZN_lag3  \
5                   -0.370669            2.045383            0.647493   
6                   -0.370669            2.045383            1.206649   
7                   -0.312890            1.206649            2.045383   
8                   -0.197330            0.401464            2.045383   
9                   -0.197330            0.854381            1.206649   
..                        ...                 ...                 ...   
284                  0.322686           -0.051453           -0.051453   
285                  0.322686           -0.051453           -0.051453   
286                 -0.255110           -0.995029           -0.051453   
287                 -0.255110           -0.995029           -0.051453   
288                 -0.255110           -0.995029           -0.995029   

     opinions_AMZN_lag5  opinions_AMZN_count_lag1  opinions_AMZN_count_lag3  \
5             -0.051453                  0.653014                  0.653014   
6              1.101807                  0.653014                 -0.680704   
7              0.647493                 -0.680704                  0.653014   
8              1.206649                  1.986731                  0.653014   
9              2.045383                  1.986731                 -0.680704   
..                  ...                       ...                       ...   
284            0.892124                 -0.680704                 -0.680704   
285           -0.051453                 -0.680704                 -0.680704   
286           -0.051453                 -0.013845                 -0.680704   
287           -0.051453                 -0.013845                 -0.680704   
288           -0.051453                 -0.013845                 -0.013845   

     opinions_AMZN_count_lag5  
5                   -1.347562  
6                    2.653590  
7                    0.653014  
8                   -0.680704  
9                    0.653014  
..                        ...  
284                 -0.013845  
285                 -0.680704  
286                 -0.680704  
287                 -0.680704  
288                 -0.680704  

[284 rows x 12 columns]

In [187]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_amzn_scaled_lag) * 0.8)
sentiment_train_data = sentiment_amzn_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_amzn_scaled_lag.iloc[sentiment_train_size:, 1:]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [188]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [189]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)

In [191]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model7 = Sequential()
model7.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model7.add(Dropout(0.05))
model7.add(Bidirectional(LSTM(128, return_sequences=True)))
model7.add(Dropout(0.05))

model7.add(BatchNormalization())
model7.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model7.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model7.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model7.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 18s 675ms/step - loss: 1.5643 - val_loss: 0.8279
Epoch 2/200
7/7 [==============================] - 2s 289ms/step - loss: 1.2362 - val_loss: 0.8335
Epoch 3/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1809 - val_loss: 0.8374
Epoch 4/200
7/7 [==============================] - 2s 237ms/step - loss: 1.1334 - val_loss: 0.8311
Epoch 5/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1279 - val_loss: 0.8349
Epoch 6/200
7/7 [==============================] - 2s 241ms/step - loss: 1.1226 - val_loss: 0.8372
Epoch 7/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1202 - val_loss: 0.8308
Epoch 8/200
7/7 [==============================] - 2s 316ms/step - loss: 1.1128 - val_loss: 0.8348
Epoch 9/200
7/7 [==============================] - 3s 379ms/step - loss: 1.1193 - val_loss: 0.8364
Epoch 10/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1101 - val_loss: 0.8360
Epoch 11

In [192]:
# Calculate evaluation metrics
amzn_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
amzn_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
amzn_sentiment_rmse = np.sqrt(amzn_sentiment_mse)
amzn_sentiment_mape = np.mean(np.abs((sentiment_y_val - sentiment_y_pred) / sentiment_y_val)) * 100

print("MAE:", amzn_sentiment_mae)
print("MSE:", amzn_sentiment_mse)
print("RMSE:", amzn_sentiment_rmse)
print("MAPE:", amzn_sentiment_mape)

MAE: 0.6985445381543979
MSE: 0.8331257802170621
RMSE: 0.9127572405722466
MAPE: 100.39551087844141


## Stock & Sentiment

In [193]:
# Split data into training and validation sets
all_train_size = int(len(amzn_scaled_lag) * 0.8)
all_train_data = amzn_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = amzn_scaled_lag.iloc[all_train_size:, 1:]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [194]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [195]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [196]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model12 = Sequential()
model12.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model12.add(Dropout(0.05))
model12.add(Bidirectional(LSTM(128, return_sequences=True)))
model12.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model12.add(BatchNormalization())
model12.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model12.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model12.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model12.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 502ms/step - loss: 1.5989 - val_loss: 0.8339
Epoch 2/200
7/7 [==============================] - 2s 241ms/step - loss: 1.2821 - val_loss: 0.8317
Epoch 3/200
7/7 [==============================] - 2s 231ms/step - loss: 1.2145 - val_loss: 0.8399
Epoch 4/200
7/7 [==============================] - 2s 240ms/step - loss: 1.1674 - val_loss: 0.8374
Epoch 5/200
7/7 [==============================] - 2s 328ms/step - loss: 1.1560 - val_loss: 0.8356
Epoch 6/200
7/7 [==============================] - 3s 386ms/step - loss: 1.1541 - val_loss: 0.8352
Epoch 7/200
7/7 [==============================] - 2s 234ms/step - loss: 1.1397 - val_loss: 0.8364
Epoch 8/200
7/7 [==============================] - 2s 229ms/step - loss: 1.1331 - val_loss: 0.8343
Epoch 9/200
7/7 [==============================] - 2s 229ms/step - loss: 1.1319 - val_loss: 0.8345
Epoch 10/200
7/7 [==============================] - 2s 233ms/step - loss: 1.1293 - val_loss: 0.8341
Epoch 11

In [197]:
# Calculate evaluation metrics
amzn_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
amzn_all_mse = np.mean(np.square(all_y_val - all_y_pred))
amzn_all_rmse = np.sqrt(amzn_all_mse)
amzn_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", amzn_all_mae)
print("MSE:", amzn_all_mse)
print("RMSE:", amzn_all_rmse)
print("MAPE:", amzn_all_mape)

MAE: 0.6994788139847403
MSE: 0.8353516166339731
RMSE: 0.9139757199367897
MAPE: 100.98270270641432


## Stock & Opinion

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(amzn_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = amzn_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = amzn_scaled_lag.iloc[all_train_size:, 7:]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model12 = Sequential()
model12.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model12.add(Dropout(0.05))
model12.add(Bidirectional(LSTM(128, return_sequences=True)))
model12.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model12.add(BatchNormalization())
model12.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model12.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model12.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model12.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 23s 925ms/step - loss: 1.3844 - val_loss: 0.8243
Epoch 2/200
7/7 [==============================] - 4s 647ms/step - loss: 1.2354 - val_loss: 0.8365
Epoch 3/200
7/7 [==============================] - 5s 677ms/step - loss: 1.1995 - val_loss: 0.8324
Epoch 4/200
7/7 [==============================] - 3s 465ms/step - loss: 1.1585 - val_loss: 0.8307
Epoch 5/200
7/7 [==============================] - 3s 480ms/step - loss: 1.1410 - val_loss: 0.8327
Epoch 6/200
7/7 [==============================] - 4s 648ms/step - loss: 1.1335 - val_loss: 0.8298
Epoch 7/200
7/7 [==============================] - 4s 622ms/step - loss: 1.1292 - val_loss: 0.8306
Epoch 8/200
7/7 [==============================] - 4s 585ms/step - loss: 1.1278 - val_loss: 0.8305
Epoch 9/200
7/7 [==============================] - 5s 674ms/step - loss: 1.1175 - val_loss: 0.8305
Epoch 10/200
7/7 [==============================] - 5s 688ms/step - loss: 1.1228 - val_loss: 0.8298
Epoch 11

In [ ]:
# Calculate evaluation metrics
amzn_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
amzn_all_mse = np.mean(np.square(all_y_val - all_y_pred))
amzn_all_rmse = np.sqrt(amzn_all_mse)
amzn_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", amzn_all_mae)
print("MSE:", amzn_all_mse)
print("RMSE:", amzn_all_rmse)
print("MAPE:", amzn_all_mape)

MAE: 0.6964176509927229
MSE: 0.8279600321005876
RMSE: 0.9099230913107919
MAPE: 98.5489114490517


## Stock & Comment

In [ ]:
comments=amzn_scaled_lag.iloc[:, 1:7]

In [ ]:
market=amzn_scaled_lag.iloc[:, 13:]

In [ ]:
stock_comment= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(stock_comment) * 0.8)
all_train_data = stock_comment.iloc[:all_train_size, :]
all_val_data = stock_comment.iloc[all_train_size:, :]
target=amzn_scaled_df['amzn_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model12 = Sequential()
model12.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model12.add(Dropout(0.05))
model12.add(Bidirectional(LSTM(128, return_sequences=True)))
model12.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model12.add(BatchNormalization())
model12.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model12.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model12.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model12.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 16s 697ms/step - loss: 1.4674 - val_loss: 0.8296
Epoch 2/200
7/7 [==============================] - 3s 463ms/step - loss: 1.2426 - val_loss: 0.8359
Epoch 3/200
7/7 [==============================] - 4s 611ms/step - loss: 1.1911 - val_loss: 0.8344
Epoch 4/200
7/7 [==============================] - 4s 596ms/step - loss: 1.1458 - val_loss: 0.8339
Epoch 5/200
7/7 [==============================] - 5s 743ms/step - loss: 1.1410 - val_loss: 0.8364
Epoch 6/200
7/7 [==============================] - 6s 788ms/step - loss: 1.1271 - val_loss: 0.8361
Epoch 7/200
7/7 [==============================] - 5s 670ms/step - loss: 1.1191 - val_loss: 0.8339
Epoch 8/200
7/7 [==============================] - 6s 958ms/step - loss: 1.1250 - val_loss: 0.8332
Epoch 9/200
7/7 [==============================] - 7s 981ms/step - loss: 1.1248 - val_loss: 0.8354
Epoch 10/200
7/7 [==============================] - 5s 649ms/step - loss: 1.1213 - val_loss: 0.8370
Epoch 11

In [ ]:
# Calculate evaluation metrics
amzn_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
amzn_all_mse = np.mean(np.square(all_y_val - all_y_pred))
amzn_all_rmse = np.sqrt(amzn_all_mse)
amzn_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", amzn_all_mae)
print("MSE:", amzn_all_mse)
print("RMSE:", amzn_all_rmse)
print("MAPE:", amzn_all_mape)

MAE: 0.6974295579281525
MSE: 0.8305366899138074
RMSE: 0.9113378571714265
MAPE: 98.14190527318584


# AAPL

## Train Test Split

In [230]:
# Split data into training and validation sets
train_size = int(len(aapl_scaled_lag) * 0.8)
train_data = aapl_scaled_lag.iloc[:train_size, 1:]
val_data = aapl_scaled_lag.iloc[train_size:, 1:]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [231]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [232]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = aapl_scaled_lag.filter(like='comment').columns.tolist() + aapl_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_aapl_scaled_lag = aapl_scaled_lag.drop(columns=columns_to_drop)
stock_aapl_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  aapl_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028          0.468194   
6   2022-01-10               -0.446869            -1.785457          0.039844   
7   2022-01-11                0.070342            -1.772770          0.882541   
8   2022-01-12                0.770026            -1.820702         -0.404301   
9   2022-01-13                0.164879            -1.850307         -0.460313   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.739187   
285 2023-02-21               -0.250307             1.114425         -1.118332   
286 2023-02-22               -1.237982             1.293465         -1.129966   
287 2023-02-23                0.112495             1.248352         -1.460041   
288 2023-02-24                0.418239             1.186323         -1.570002   

     aapl_percentage_lag1  
5               -0.745686  
6                0.064797  
7                0.024819  
8                0.788820  
9                0.137307  
..                    ...  
284             -0.458562  
285             -0.326428  
286             -1.203441  
287              0.152242  
288              0.170322  

[284 rows x 5 columns]

In [233]:
# Split data into training and validation sets
stock_train_size = int(len(stock_aapl_scaled_lag) * 0.8)
stock_train_data = stock_aapl_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_aapl_scaled_lag.iloc[stock_train_size:, 1:]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target = target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [234]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [235]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [236]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model16 = Sequential()
model16.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model16.add(Dropout(0.05))
model16.add(Bidirectional(LSTM(128, return_sequences=True)))
model16.add(Dropout(0.05))

model16.add(BatchNormalization())
model16.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model16.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model16.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 15s 506ms/step - loss: 1.9416 - val_loss: 0.5981
Epoch 2/200
7/7 [==============================] - 2s 239ms/step - loss: 1.4015 - val_loss: 0.6027
Epoch 3/200
7/7 [==============================] - 2s 235ms/step - loss: 1.2440 - val_loss: 0.6026
Epoch 4/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2459 - val_loss: 0.6048
Epoch 5/200
7/7 [==============================] - 2s 341ms/step - loss: 1.2063 - val_loss: 0.6070
Epoch 6/200
7/7 [==============================] - 3s 386ms/step - loss: 1.2003 - val_loss: 0.6063
Epoch 7/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1940 - val_loss: 0.6071
Epoch 8/200
7/7 [==============================] - 2s 244ms/step - loss: 1.1929 - val_loss: 0.6049
Epoch 9/200
7/7 [==============================] - 2s 230ms/step - loss: 1.1904 - val_loss: 0.6057
Epoch 10/200
7/7 [==============================] - 2s 232ms/step - loss: 1.1864 - val_loss: 0.6050
Epoch 11

In [237]:
# Calculate evaluation metrics
aapl_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
aapl_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
aapl_stock_rmse = np.sqrt(aapl_stock_mse)
aapl_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", aapl_stock_mae)
print("MSE:", aapl_stock_mse)
print("RMSE:", aapl_stock_rmse)
print("MAPE:", aapl_stock_mape)

MAE: 0.624218077386365
MSE: 0.5960523827468943
RMSE: 0.772044288073485
MAPE: 102.78518436542092


## Only Sentiment

In [238]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = aapl_scaled_lag.filter(like='comment').columns.tolist() + aapl_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
sentiment_aapl_scaled_lag = aapl_scaled_lag[columns_to_drop]
sentiment_aapl_scaled_lag 

comments_AAPL_lag1  comments_AAPL_lag3  comments_AAPL_lag5  \
5             -0.947273           -0.372743            0.022247   
6             -0.372743            0.365207            0.022247   
7              1.314942           -0.947273           -0.372743   
8              0.708167           -0.372743            0.365207   
9              0.708167            1.314942           -0.947273   
..                  ...                 ...                 ...   
284            0.022247            0.022247           -1.270447   
285            0.022247            0.022247           -1.270447   
286            0.022247            0.022247            0.022247   
287            0.022247            0.022247            0.022247   
288            0.022247            0.022247            0.022247   

     comments_AAPL_count_lag1  comments_AAPL_count_lag3  \
5                   -0.213530                  0.827994   
6                    0.827994                  1.088375   
7                   -0.473912                 -0.213530   
8                    1.088375                  0.827994   
9                    1.088375                 -0.473912   
..                        ...                       ...   
284                 -0.473912                 -0.473912   
285                 -0.473912                 -0.473912   
286                 -0.473912                 -0.473912   
287                 -0.473912                 -0.473912   
288                 -0.473912                 -0.473912   

     comments_AAPL_count_lag5  opinions_AAPL_lag1  opinions_AAPL_lag3  \
5                   -0.734293            0.957228            0.913676   
6                   -0.734293            0.957228            0.957228   
7                    0.827994           -0.131558            0.957228   
8                    1.088375           -0.131558            0.957228   
9                   -0.213530            0.957228           -0.131558   
..                        ...                 ...                 ...   
284                 -0.473912            1.320156           -0.131558   
285                 -0.473912            1.320156           -0.131558   
286                 -0.473912           -0.131558            1.320156   
287                 -0.473912           -0.131558            1.320156   
288                 -0.473912           -0.131558           -0.131558   

     opinions_AAPL_lag5  opinions_AAPL_count_lag1  opinions_AAPL_count_lag3  \
5             -0.131558                  0.075709                  2.419980   
6              1.391846                  0.075709                  0.075709   
7              0.913676                  1.638556                  0.075709   
8              0.957228                  1.638556                  0.075709   
9              0.957228                  0.075709                  1.638556   
..                  ...                       ...                       ...   
284           -0.131558                 -0.705715                 -0.705715   
285            0.957228                 -0.705715                 -0.705715   
286           -0.131558                 -0.705715                 -0.705715   
287           -0.131558                 -0.705715                 -0.705715   
288            1.320156                 -0.705715                 -0.705715   

     opinions_AAPL_count_lag5  
5                   -1.487138  
6                    5.545674  
7                    2.419980  
8                    0.075709  
9                    0.075709  
..                        ...  
284                 -0.705715  
285                  0.075709  
286                 -0.705715  
287                 -0.705715  
288                 -0.705715  

[284 rows x 12 columns]

In [239]:
# Split data into training and validation sets
sentiment_train_size = int(len(sentiment_aapl_scaled_lag) * 0.8)
sentiment_train_data = sentiment_aapl_scaled_lag.iloc[:sentiment_train_size, 1:]
sentiment_val_data = sentiment_aapl_scaled_lag.iloc[sentiment_train_size:, 1:]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
sentiment_train_features = sentiment_train_data
sentiment_train_target =target.iloc[:sentiment_train_size]
sentiment_val_features = sentiment_val_data
sentiment_val_target = target.iloc[sentiment_train_size:]

In [240]:
# Convert the input features and target variable to numpy arrays
sentiment_train_features = sentiment_train_features.values
sentiment_train_target = sentiment_train_target.values
sentiment_val_features = sentiment_val_features.values
sentiment_val_target = sentiment_val_target.values

In [241]:
time_steps = 20
sentiment_X_train, sentiment_y_train = create_sequences(sentiment_train_features, sentiment_train_target, time_steps)
sentiment_X_val, sentiment_y_val = create_sequences(sentiment_val_features, sentiment_val_target, time_steps)

In [242]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model17 = Sequential()
model17.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model17.add(Dropout(0.05))
model17.add(Bidirectional(LSTM(128, return_sequences=True)))
model17.add(Dropout(0.05))

model17.add(BatchNormalization())
model17.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model17.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model17.fit(sentiment_X_train, sentiment_y_train, validation_data=(sentiment_X_val, sentiment_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

sentiment_y_pred = model17.predict(sentiment_X_val)
print(sentiment_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 516ms/step - loss: 1.7376 - val_loss: 0.6177
Epoch 2/200
7/7 [==============================] - 2s 228ms/step - loss: 1.4008 - val_loss: 0.6105
Epoch 3/200
7/7 [==============================] - 2s 312ms/step - loss: 1.2900 - val_loss: 0.6069
Epoch 4/200
7/7 [==============================] - 3s 390ms/step - loss: 1.2602 - val_loss: 0.6148
Epoch 5/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2610 - val_loss: 0.6102
Epoch 6/200
7/7 [==============================] - 2s 233ms/step - loss: 1.2400 - val_loss: 0.6121
Epoch 7/200
7/7 [==============================] - 2s 231ms/step - loss: 1.2287 - val_loss: 0.6133
Epoch 8/200
7/7 [==============================] - 2s 240ms/step - loss: 1.2178 - val_loss: 0.6136
Epoch 9/200
7/7 [==============================] - 2s 232ms/step - loss: 1.2130 - val_loss: 0.6131
Epoch 10/200
7/7 [==============================] - 2s 230ms/step - loss: 1.2123 - val_loss: 0.6094
Epoch 11

In [243]:
# Calculate evaluation metrics
aapl_sentiment_mae = np.mean(np.abs(sentiment_y_val - sentiment_y_pred))
aapl_sentiment_mse = np.mean(np.square(sentiment_y_val - sentiment_y_pred))
aapl_sentiment_rmse = np.sqrt(aapl_sentiment_mse)
aapl_sentiment_mape = np.mean(np.abs((sentiment_y_val - sentiment_y_pred) / sentiment_y_val)) * 100

print("MAE:", aapl_sentiment_mae)
print("MSE:", aapl_sentiment_mse)
print("RMSE:", aapl_sentiment_rmse)
print("MAPE:", aapl_sentiment_mape)

MAE: 0.6279293103917428
MSE: 0.6011423625410053
RMSE: 0.7753337104376445
MAPE: 104.98584172698087


## Stock & Sentiment

In [244]:
# Split data into training and validation sets
all_train_size = int(len(aapl_scaled_lag) * 0.8)
all_train_data = aapl_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = aapl_scaled_lag.iloc[all_train_size:, 1:]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [245]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [246]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [247]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 507ms/step - loss: 1.9465 - val_loss: 0.6048
Epoch 2/200
7/7 [==============================] - 2s 228ms/step - loss: 1.4861 - val_loss: 0.6000
Epoch 3/200
7/7 [==============================] - 2s 222ms/step - loss: 1.2954 - val_loss: 0.6046
Epoch 4/200
7/7 [==============================] - 2s 232ms/step - loss: 1.2750 - val_loss: 0.5968
Epoch 5/200
7/7 [==============================] - 2s 271ms/step - loss: 1.2441 - val_loss: 0.5987
Epoch 6/200
7/7 [==============================] - 3s 401ms/step - loss: 1.2350 - val_loss: 0.5980
Epoch 7/200
7/7 [==============================] - 2s 273ms/step - loss: 1.2273 - val_loss: 0.5996
Epoch 8/200
7/7 [==============================] - 2s 237ms/step - loss: 1.2214 - val_loss: 0.5994
Epoch 9/200
7/7 [==============================] - 2s 244ms/step - loss: 1.2237 - val_loss: 0.5978
Epoch 10/200
7/7 [==============================] - 2s 230ms/step - loss: 1.2209 - val_loss: 0.5961
Epoch 11

In [248]:
# Calculate evaluation metrics
aapl_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
aapl_all_mse = np.mean(np.square(all_y_val - all_y_pred))
aapl_all_rmse = np.sqrt(aapl_all_mse)
aapl_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", aapl_all_mae)
print("MSE:", aapl_all_mse)
print("RMSE:", aapl_all_rmse)
print("MAPE:", aapl_all_mape)

MAE: 0.6209825388215632
MSE: 0.5918481846407837
RMSE: 0.7693166998322496
MAPE: 107.71738977779077


## Stock & Opinion

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(nio_scaled_lag.iloc[:,7:]) * 0.8)
all_train_data = nio_scaled_lag.iloc[:all_train_size, 7:]
all_val_data = nio_scaled_lag.iloc[all_train_size:, 7:]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 18s 691ms/step - loss: 1.7954 - val_loss: 0.5935
Epoch 2/200
7/7 [==============================] - 4s 558ms/step - loss: 1.4064 - val_loss: 0.6092
Epoch 3/200
7/7 [==============================] - 3s 352ms/step - loss: 1.3262 - val_loss: 0.5908
Epoch 4/200
7/7 [==============================] - 2s 343ms/step - loss: 1.2716 - val_loss: 0.6027
Epoch 5/200
7/7 [==============================] - 3s 356ms/step - loss: 1.2547 - val_loss: 0.5935
Epoch 6/200
7/7 [==============================] - 2s 285ms/step - loss: 1.2463 - val_loss: 0.5988
Epoch 7/200
7/7 [==============================] - 2s 348ms/step - loss: 1.2393 - val_loss: 0.5983
Epoch 8/200
7/7 [==============================] - 3s 500ms/step - loss: 1.2323 - val_loss: 0.5962
Epoch 9/200
7/7 [==============================] - 2s 322ms/step - loss: 1.2356 - val_loss: 0.5979
Epoch 10/200
7/7 [==============================] - 2s 314ms/step - loss: 1.2295 - val_loss: 0.5958
Epoch 11

In [ ]:
# Calculate evaluation metrics
aapl_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
aapl_all_mse = np.mean(np.square(all_y_val - all_y_pred))
aapl_all_rmse = np.sqrt(aapl_all_mse)
aapl_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", aapl_all_mae)
print("MSE:", aapl_all_mse)
print("RMSE:", aapl_all_rmse)
print("MAPE:", aapl_all_mape)

MAE: 0.6191608500908224
MSE: 0.5895360822363681
RMSE: 0.767812530658603
MAPE: 107.61538276250582


## Stock & Comment

In [ ]:
comments=aapl_scaled_lag.iloc[:, 1:7]

In [ ]:
market=aapl_scaled_lag.iloc[:, 13:]

In [ ]:
stock_comment= pd.concat([comments, market], axis=1)

In [ ]:
# Split data into training and validation sets
all_train_size = int(len(stock_comment) * 0.8)
all_train_data = stock_comment.iloc[:all_train_size, :]
all_val_data = stock_comment.iloc[all_train_size:, :]
target=aapl_scaled_df['aapl_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [ ]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [ ]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [ ]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 24s 645ms/step - loss: 2.1468 - val_loss: 0.6141
Epoch 2/200
7/7 [==============================] - 2s 279ms/step - loss: 1.4902 - val_loss: 0.5943
Epoch 3/200
7/7 [==============================] - 3s 444ms/step - loss: 1.3795 - val_loss: 0.6005
Epoch 4/200
7/7 [==============================] - 3s 423ms/step - loss: 1.2767 - val_loss: 0.6040
Epoch 5/200
7/7 [==============================] - 2s 321ms/step - loss: 1.2454 - val_loss: 0.5973
Epoch 6/200
7/7 [==============================] - 2s 329ms/step - loss: 1.2411 - val_loss: 0.5999
Epoch 7/200
7/7 [==============================] - 2s 318ms/step - loss: 1.2223 - val_loss: 0.6015
Epoch 8/200
7/7 [==============================] - 2s 301ms/step - loss: 1.2172 - val_loss: 0.6017
Epoch 9/200
7/7 [==============================] - 3s 509ms/step - loss: 1.2156 - val_loss: 0.6022
Epoch 10/200
7/7 [==============================] - 3s 365ms/step - loss: 1.2073 - val_loss: 0.6021
Epoch 11

In [ ]:
# Calculate evaluation metrics
aapl_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
aapl_all_mse = np.mean(np.square(all_y_val - all_y_pred))
aapl_all_rmse = np.sqrt(aapl_all_mse)
aapl_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", aapl_all_mae)
print("MSE:", aapl_all_mse)
print("RMSE:", aapl_all_rmse)
print("MAPE:", aapl_all_mape)

MAE: 0.6283269218729258
MSE: 0.601604153411465
RMSE: 0.7756314546300098
MAPE: 100.81643118462398


# AMC

## Train Test Split

In [249]:
# Split data into training and validation sets
train_size = int(len(amc_scaled_lag) * 0.8)
train_data = amc_scaled_lag.iloc[:train_size, 1:]
val_data = amc_scaled_lag.iloc[train_size:, 1:]
target=amc_scaled_df['amc_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [250]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [251]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = amc_scaled_lag.filter(like='comment').columns.tolist() + amc_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_amc_scaled_lag = amc_scaled_lag.drop(columns=columns_to_drop)
stock_amc_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  amc_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028         0.674833   
6   2022-01-10               -0.446869            -1.785457         0.318804   
7   2022-01-11                0.070342            -1.772770        -0.113567   
8   2022-01-12                0.770026            -1.820702        -0.177153   
9   2022-01-13                0.164879            -1.850307        -0.494740   
..         ...                     ...                  ...              ...   
284 2023-02-17               -0.866740             1.135571        -0.324209   
285 2023-02-21               -0.250307             1.114425        -0.490293   
286 2023-02-22               -1.237982             1.293465         1.207130   
287 2023-02-23                0.112495             1.248352         1.028009   
288 2023-02-24                0.418239             1.186323         0.047608   

     amc_percentage_lag1  
5              -0.139631  
6               0.348840  
7              -0.091074  
8               0.037591  
9              -0.009591  
..                   ...  
284             0.239659  
285             0.006091  
286             2.237479  
287             0.384214  
288            -0.032717  

[284 rows x 5 columns]

In [252]:
# Split data into training and validation sets
stock_train_size = int(len(stock_amc_scaled_lag) * 0.8)
stock_train_data = stock_amc_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_amc_scaled_lag.iloc[stock_train_size:, 1:]
target=amc_scaled_df['amc_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [253]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [256]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [257]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model17 = Sequential()
model17.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model17.add(Dropout(0.05))
model17.add(Bidirectional(LSTM(128, return_sequences=True)))
model17.add(Dropout(0.05))

model17.add(BatchNormalization())
model17.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model17.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model17.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model17.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 18s 536ms/step - loss: 1.4466 - val_loss: 1.0514
Epoch 2/200
7/7 [==============================] - 2s 295ms/step - loss: 1.2572 - val_loss: 1.0654
Epoch 3/200
7/7 [==============================] - 3s 383ms/step - loss: 1.2068 - val_loss: 1.0562
Epoch 4/200
7/7 [==============================] - 2s 244ms/step - loss: 1.1928 - val_loss: 1.0607
Epoch 5/200
7/7 [==============================] - 2s 241ms/step - loss: 1.1824 - val_loss: 1.0556
Epoch 6/200
7/7 [==============================] - 2s 253ms/step - loss: 1.1726 - val_loss: 1.0517
Epoch 7/200
7/7 [==============================] - 2s 246ms/step - loss: 1.1625 - val_loss: 1.0560
Epoch 8/200
7/7 [==============================] - 2s 237ms/step - loss: 1.1610 - val_loss: 1.0549
Epoch 9/200
7/7 [==============================] - 2s 239ms/step - loss: 1.1594 - val_loss: 1.0548
Epoch 10/200
7/7 [==============================] - 2s 369ms/step - loss: 1.1563 - val_loss: 1.0517
Epoch 11

In [258]:
# Calculate evaluation metrics
amc_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
amc_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
amc_stock_rmse = np.sqrt(amc_stock_mse)
amc_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", amc_stock_mae)
print("MSE:", amc_stock_mse)
print("RMSE:", amc_stock_rmse)
print("MAPE:", amc_stock_mape)

MAE: 0.7384807713406295
MSE: 1.0508210457254403
RMSE: 1.0250956276003913
MAPE: 112.45630514329852


## Stock & Sentiment

In [260]:
# Split data into training and validation sets
all_train_size = int(len(amc_scaled_lag) * 0.8)
all_train_data = amc_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = amc_scaled_lag.iloc[all_train_size:, 1:]
target=amc_scaled_df['amc_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [261]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [262]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [263]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 20s 577ms/step - loss: 1.6859 - val_loss: 1.0553
Epoch 2/200
7/7 [==============================] - 2s 236ms/step - loss: 1.3870 - val_loss: 1.0512
Epoch 3/200
7/7 [==============================] - 2s 236ms/step - loss: 1.3188 - val_loss: 1.0442
Epoch 4/200
7/7 [==============================] - 2s 237ms/step - loss: 1.2744 - val_loss: 1.0542
Epoch 5/200
7/7 [==============================] - 2s 296ms/step - loss: 1.2459 - val_loss: 1.0516
Epoch 6/200
7/7 [==============================] - 3s 374ms/step - loss: 1.2357 - val_loss: 1.0575
Epoch 7/200
7/7 [==============================] - 2s 253ms/step - loss: 1.2112 - val_loss: 1.0511
Epoch 8/200
7/7 [==============================] - 2s 241ms/step - loss: 1.2168 - val_loss: 1.0470
Epoch 9/200
7/7 [==============================] - 2s 236ms/step - loss: 1.2143 - val_loss: 1.0552
Epoch 10/200
7/7 [==============================] - 2s 239ms/step - loss: 1.2114 - val_loss: 1.0508
Epoch 11

In [264]:
# Calculate evaluation metrics
amc_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
amc_all_mse = np.mean(np.square(all_y_val - all_y_pred))
amc_all_rmse = np.sqrt(amc_all_mse)
amc_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", amc_all_mae)
print("MSE:", amc_all_mse)
print("RMSE:", amc_all_rmse)
print("MAPE:", amc_all_mape)

MAE: 0.7401364540220375
MSE: 1.0437297338566824
RMSE: 1.0216309186084191
MAPE: 129.77115541318287


# AMD

## Train Test Split

In [265]:
# Split data into training and validation sets
train_size = int(len(amd_scaled_lag) * 0.8)
train_data = amd_scaled_lag.iloc[:train_size, 1:]
val_data = amd_scaled_lag.iloc[train_size:, 1:]
target=amd_scaled_df['amd_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [266]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [267]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = amd_scaled_lag.filter(like='comment').columns.tolist() + amd_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_amd_scaled_lag = amd_scaled_lag.drop(columns=columns_to_drop)
stock_amd_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  amd_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028        -0.754976   
6   2022-01-10               -0.446869            -1.785457        -0.971716   
7   2022-01-11                0.070342            -1.772770        -0.078080   
8   2022-01-12                0.770026            -1.820702        -0.334190   
9   2022-01-13                0.164879            -1.850307        -0.589173   
..         ...                     ...                  ...              ...   
284 2023-02-17               -0.866740             1.135571        -0.995084   
285 2023-02-21               -0.250307             1.114425        -1.333341   
286 2023-02-22               -1.237982             1.293465        -1.462155   
287 2023-02-23                0.112495             1.248352        -1.249524   
288 2023-02-24                0.418239             1.186323        -0.360171   

     amd_percentage_lag1  
5               0.051663  
6              -0.775348  
7               0.036303  
8               1.087836  
9               0.066763  
..                   ...  
284            -1.528770  
285            -0.479443  
286            -0.539773  
287            -0.018175  
288             1.107690  

[284 rows x 5 columns]

In [268]:
# Split data into training and validation sets
stock_train_size = int(len(stock_amd_scaled_lag) * 0.8)
stock_train_data = stock_amd_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_amd_scaled_lag.iloc[stock_train_size:, 1:]
target=amd_scaled_df['amd_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [269]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [270]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [271]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model16 = Sequential()
model16.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model16.add(Dropout(0.05))
model16.add(Bidirectional(LSTM(128, return_sequences=True)))
model16.add(Dropout(0.05))

model16.add(BatchNormalization())
model16.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model16.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model16.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 695ms/step - loss: 1.9252 - val_loss: 0.9424
Epoch 2/200
7/7 [==============================] - 2s 256ms/step - loss: 1.3644 - val_loss: 0.9443
Epoch 3/200
7/7 [==============================] - 2s 230ms/step - loss: 1.2322 - val_loss: 0.9379
Epoch 4/200
7/7 [==============================] - 2s 234ms/step - loss: 1.1789 - val_loss: 0.9436
Epoch 5/200
7/7 [==============================] - 2s 236ms/step - loss: 1.1507 - val_loss: 0.9335
Epoch 6/200
7/7 [==============================] - 2s 237ms/step - loss: 1.1484 - val_loss: 0.9425
Epoch 7/200
7/7 [==============================] - 2s 229ms/step - loss: 1.1362 - val_loss: 0.9397
Epoch 8/200
7/7 [==============================] - 2s 327ms/step - loss: 1.1399 - val_loss: 0.9429
Epoch 9/200
7/7 [==============================] - 3s 366ms/step - loss: 1.1341 - val_loss: 0.9409
Epoch 10/200
7/7 [==============================] - 2s 231ms/step - loss: 1.1341 - val_loss: 0.9447
Epoch 11

In [272]:
# Calculate evaluation metrics
amd_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
amd_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
amd_stock_rmse = np.sqrt(amd_stock_mse)
amd_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", amd_stock_mae)
print("MSE:", amd_stock_mse)
print("RMSE:", amd_stock_rmse)
print("MAPE:", amd_stock_mape)

MAE: 0.7215537991124638
MSE: 0.9278282130785311
RMSE: 0.9632383988808436
MAPE: 102.13662799044793


## Stock & Sentiment

In [274]:
# Split data into training and validation sets
all_train_size = int(len(amd_scaled_lag) * 0.8)
all_train_data = amd_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = amd_scaled_lag.iloc[all_train_size:, 1:]
target=amd_scaled_df['amd_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [275]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [276]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [277]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 17s 507ms/step - loss: 1.8479 - val_loss: 0.9315
Epoch 2/200
7/7 [==============================] - 2s 226ms/step - loss: 1.4474 - val_loss: 0.9290
Epoch 3/200
7/7 [==============================] - 2s 240ms/step - loss: 1.3364 - val_loss: 0.9297
Epoch 4/200
7/7 [==============================] - 2s 245ms/step - loss: 1.2379 - val_loss: 0.9258
Epoch 5/200
7/7 [==============================] - 2s 352ms/step - loss: 1.2289 - val_loss: 0.9253
Epoch 6/200
7/7 [==============================] - 2s 355ms/step - loss: 1.2107 - val_loss: 0.9253
Epoch 7/200
7/7 [==============================] - 2s 232ms/step - loss: 1.2112 - val_loss: 0.9303
Epoch 8/200
7/7 [==============================] - 2s 235ms/step - loss: 1.1989 - val_loss: 0.9233
Epoch 9/200
7/7 [==============================] - 2s 232ms/step - loss: 1.1970 - val_loss: 0.9225
Epoch 10/200
7/7 [==============================] - 2s 238ms/step - loss: 1.1985 - val_loss: 0.9257
Epoch 11

In [278]:
# Calculate evaluation metrics
amd_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
amd_all_mse = np.mean(np.square(all_y_val - all_y_pred))
amd_all_rmse = np.sqrt(aapl_all_mse)
amd_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", amd_all_mae)
print("MSE:", amd_all_mse)
print("RMSE:", amd_all_rmse)
print("MAPE:", amd_all_mape)

MAE: 0.717879132425538
MSE: 0.9158324324092119
RMSE: 0.7693166998322496
MAPE: 106.82070105668782


# BABA

## Train Test Split

In [279]:
# Split data into training and validation sets
train_size = int(len(baba_scaled_lag) * 0.8)
train_data = baba_scaled_lag.iloc[:train_size, 1:]
val_data = baba_scaled_lag.iloc[train_size:, 1:]
target=baba_scaled_df['baba_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [280]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [281]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = baba_scaled_lag.filter(like='comment').columns.tolist() + baba_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_baba_scaled_lag = baba_scaled_lag.drop(columns=columns_to_drop)
stock_baba_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  baba_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028          0.350982   
6   2022-01-10               -0.446869            -1.785457          0.711162   
7   2022-01-11                0.070342            -1.772770         -0.163926   
8   2022-01-12                0.770026            -1.820702         -0.149429   
9   2022-01-13                0.164879            -1.850307         -0.010236   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.260400   
285 2023-02-21               -0.250307             1.114425         -0.475627   
286 2023-02-22               -1.237982             1.293465          0.061691   
287 2023-02-23                0.112495             1.248352         -0.097924   
288 2023-02-24                0.418239             1.186323          0.851851   

     baba_percentage_lag1  
5                0.996954  
6                0.554811  
7               -0.256121  
8                0.669726  
9                0.872081  
..                    ...  
284              0.007019  
285             -0.662916  
286             -1.082898  
287             -0.073664  
288             -0.143768  

[284 rows x 5 columns]

In [282]:
# Split data into training and validation sets
stock_train_size = int(len(stock_baba_scaled_lag) * 0.8)
stock_train_data = stock_baba_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_baba_scaled_lag.iloc[stock_train_size:, 1:]
target=baba_scaled_df['baba_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [283]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [284]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [285]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model16 = Sequential()
model16.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model16.add(Dropout(0.05))
model16.add(Bidirectional(LSTM(128, return_sequences=True)))
model16.add(Dropout(0.05))

model16.add(BatchNormalization())
model16.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model16.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model16.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 17s 535ms/step - loss: 1.4539 - val_loss: 0.5487
Epoch 2/200
7/7 [==============================] - 2s 236ms/step - loss: 1.3421 - val_loss: 0.5473
Epoch 3/200
7/7 [==============================] - 2s 345ms/step - loss: 1.3051 - val_loss: 0.5494
Epoch 4/200
7/7 [==============================] - 3s 372ms/step - loss: 1.2923 - val_loss: 0.5495
Epoch 5/200
7/7 [==============================] - 2s 227ms/step - loss: 1.2873 - val_loss: 0.5485
Epoch 6/200
7/7 [==============================] - 2s 234ms/step - loss: 1.2859 - val_loss: 0.5487
Epoch 7/200
7/7 [==============================] - 2s 246ms/step - loss: 1.2876 - val_loss: 0.5492
Epoch 8/200
7/7 [==============================] - 2s 238ms/step - loss: 1.2776 - val_loss: 0.5499
Epoch 9/200
7/7 [==============================] - 2s 237ms/step - loss: 1.2831 - val_loss: 0.5488
Epoch 10/200
7/7 [==============================] - 2s 241ms/step - loss: 1.2866 - val_loss: 0.5495
Epoch 11

In [286]:
# Calculate evaluation metrics
baba_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
baba_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
baba_stock_rmse = np.sqrt(baba_stock_mse)
baba_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", baba_stock_mae)
print("MSE:", baba_stock_mse)
print("RMSE:", baba_stock_rmse)
print("MAPE:", baba_stock_mape)

MAE: 0.5149614447890496
MSE: 0.5486076298317295
RMSE: 0.740680518058717
MAPE: 101.15889503619793


## Stock & Sentiment

In [287]:
# Split data into training and validation sets
all_train_size = int(len(baba_scaled_lag) * 0.8)
all_train_data = baba_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = baba_scaled_lag.iloc[all_train_size:, 1:]
target=baba_scaled_df['baba_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [288]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [289]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [290]:
# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 759ms/step - loss: 2.1651 - val_loss: 0.5445
Epoch 2/200
7/7 [==============================] - 2s 233ms/step - loss: 1.6043 - val_loss: 0.5472
Epoch 3/200
7/7 [==============================] - 2s 236ms/step - loss: 1.4564 - val_loss: 0.5420
Epoch 4/200
7/7 [==============================] - 2s 238ms/step - loss: 1.3940 - val_loss: 0.5448
Epoch 5/200
7/7 [==============================] - 2s 236ms/step - loss: 1.3544 - val_loss: 0.5455
Epoch 6/200
7/7 [==============================] - 2s 229ms/step - loss: 1.3363 - val_loss: 0.5435
Epoch 7/200
7/7 [==============================] - 2s 244ms/step - loss: 1.3317 - val_loss: 0.5441
Epoch 8/200
7/7 [==============================] - 3s 382ms/step - loss: 1.3318 - val_loss: 0.5444
Epoch 9/200
7/7 [==============================] - 2s 304ms/step - loss: 1.3200 - val_loss: 0.5446
Epoch 10/200
7/7 [==============================] - 2s 249ms/step - loss: 1.3221 - val_loss: 0.5440
Epoch 11

In [291]:
# Calculate evaluation metrics
baba_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
baba_all_mse = np.mean(np.square(all_y_val - all_y_pred))
baba_all_rmse = np.sqrt(baba_all_mse)
baba_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", baba_all_mae)
print("MSE:", baba_all_mse)
print("RMSE:", baba_all_rmse)
print("MAPE:", baba_all_mape)

MAE: 0.5209622612258787
MSE: 0.550335252645687
RMSE: 0.7418458415639242
MAPE: 129.6650564135867


# LCID

## Train Test Split

In [292]:
# Split data into training and validation sets
train_size = int(len(lcid_scaled_lag) * 0.8)
train_data = lcid_scaled_lag.iloc[:train_size, 1:]
val_data = lcid_scaled_lag.iloc[train_size:, 1:]
target=lcid_scaled_df['lcid_percentage'].iloc[5:]


# Extract the input features and target variable from the training and validation data
train_features = train_data
train_target =target.iloc[:train_size]
val_features = val_data
val_target = target.iloc[train_size:]

In [293]:
train_features = train_features.values
train_target = train_target.values
val_features = val_features.values
val_target = val_target.values

## Only stocks

In [294]:
# Filter the column names based on the substrings "comment" or "opinion"
columns_to_drop = lcid_scaled_lag.filter(like='comment').columns.tolist() + lcid_scaled_lag.filter(like='opinion').columns.tolist()
# Drop the filtered columns
stock_lcid_scaled_lag = lcid_scaled_lag.drop(columns=columns_to_drop)
stock_lcid_scaled_lag 

Date  nasdaq_percentage_lag1  YrBond10_Close_lag1  lcid_Volume_lag1  \
5   2022-01-07               -0.019115            -1.839028          1.912232   
6   2022-01-10               -0.446869            -1.785457          3.089970   
7   2022-01-11                0.070342            -1.772770          1.088325   
8   2022-01-12                0.770026            -1.820702          2.040247   
9   2022-01-13                0.164879            -1.850307          1.396178   
..         ...                     ...                  ...               ...   
284 2023-02-17               -0.866740             1.135571         -0.215219   
285 2023-02-21               -0.250307             1.114425         -0.589485   
286 2023-02-22               -1.237982             1.293465         -0.139222   
287 2023-02-23                0.112495             1.248352          0.067604   
288 2023-02-24                0.418239             1.186323          2.382708   

     lcid_percentage_lag1  
5                0.797477  
6                1.783854  
7               -0.045212  
8                1.635305  
9                0.047730  
..                    ...  
284             -0.540573  
285              0.556752  
286             -1.585176  
287              0.204459  
288             -2.022497  

[284 rows x 5 columns]

In [295]:
# Split data into training and validation sets
stock_train_size = int(len(stock_lcid_scaled_lag) * 0.8)
stock_train_data = stock_lcid_scaled_lag.iloc[:stock_train_size, 1:]
stock_val_data = stock_lcid_scaled_lag.iloc[stock_train_size:, 1:]
target=lcid_scaled_df['lcid_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
stock_train_features = stock_train_data
stock_train_target =target.iloc[:stock_train_size]
stock_val_features = stock_val_data
stock_val_target = target.iloc[stock_train_size:]

In [296]:
# Convert the input features and target variable to numpy arrays
stock_train_features = stock_train_features.values
stock_train_target = stock_train_target.values
stock_val_features = stock_val_features.values
stock_val_target = stock_val_target.values

In [297]:
time_steps = 20
stock_X_train, stock_y_train = create_sequences(stock_train_features, stock_train_target, time_steps)
stock_X_val, stock_y_val = create_sequences(stock_val_features, stock_val_target, time_steps)

In [298]:
from keras.layers import Bidirectional, LSTM, Dropout, Dense, BatchNormalization, AdditiveAttention, Lambda
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Define the LSTM model
model16 = Sequential()
model16.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model16.add(Dropout(0.05))
model16.add(Bidirectional(LSTM(128, return_sequences=True)))
model16.add(Dropout(0.05))

model16.add(BatchNormalization())
model16.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model16.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model16.fit(stock_X_train, stock_y_train, validation_data=(stock_X_val, stock_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

stock_y_pred = model6.predict(stock_X_val)
print(stock_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 505ms/step - loss: 1.0752 - val_loss: 2.5270
Epoch 2/200
7/7 [==============================] - 2s 232ms/step - loss: 0.9011 - val_loss: 2.5179
Epoch 3/200
7/7 [==============================] - 3s 368ms/step - loss: 0.8707 - val_loss: 2.5261
Epoch 4/200
7/7 [==============================] - 2s 309ms/step - loss: 0.8527 - val_loss: 2.5303
Epoch 5/200
7/7 [==============================] - 2s 236ms/step - loss: 0.8461 - val_loss: 2.5326
Epoch 6/200
7/7 [==============================] - 2s 234ms/step - loss: 0.8430 - val_loss: 2.5294
Epoch 7/200
7/7 [==============================] - 2s 238ms/step - loss: 0.8372 - val_loss: 2.5280
Epoch 8/200
7/7 [==============================] - 2s 237ms/step - loss: 0.8363 - val_loss: 2.5273
Epoch 9/200
7/7 [==============================] - 2s 245ms/step - loss: 0.8341 - val_loss: 2.5251
Epoch 10/200
7/7 [==============================] - 2s 302ms/step - loss: 0.8338 - val_loss: 2.5282
Epoch 11

In [299]:
# Calculate evaluation metrics
lcid_stock_mae = np.mean(np.abs(stock_y_val - stock_y_pred))
lcid_stock_mse = np.mean(np.square(stock_y_val - stock_y_pred))
lcid_stock_rmse = np.sqrt(lcid_stock_mse)
lcid_stock_mape = np.mean(np.abs((stock_y_val - stock_y_pred) / stock_y_val)) * 100

print("MAE:", lcid_stock_mae)
print("MSE:", lcid_stock_mse)
print("RMSE:", lcid_stock_rmse)
print("MAPE:", lcid_stock_mape)

MAE: 0.9795061676197553
MSE: 2.5277552059792554
RMSE: 1.5898915705101575
MAPE: 108.0132826208696


## Stock & Sentiment

In [300]:
# Split data into training and validation sets
all_train_size = int(len(lcid_scaled_lag) * 0.8)
all_train_data = lcid_scaled_lag.iloc[:all_train_size, 1:]
all_val_data = lcid_scaled_lag.iloc[all_train_size:, 1:]
target=lcid_scaled_df['lcid_percentage'].iloc[5:]

# Extract the input features and target variable from the training and validation data
all_train_features = all_train_data
all_train_target =target.iloc[:all_train_size]
all_val_features = all_val_data
all_val_target = target.iloc[all_train_size:]

In [301]:
# Convert the input features and target variable to numpy arrays
all_train_features = all_train_features.values
all_train_target = all_train_target.values
all_val_features = all_val_features.values
all_val_target = all_val_target.values

In [302]:
time_steps = 20
all_X_train, all_y_train = create_sequences(all_train_features, all_train_target, time_steps)
all_X_val, all_y_val = create_sequences(all_val_features, all_val_target, time_steps)

In [303]:
# Define the LSTM model
model18 = Sequential()
model18.add(Bidirectional(LSTM(256, input_shape=(time_steps, 9), return_sequences=True)))
model18.add(Dropout(0.05))
model18.add(Bidirectional(LSTM(128, return_sequences=True)))
model18.add(Dropout(0.05))
#model2.add(Lambda(lambda x: tf.stack([x, x], axis=-1)))  # Duplicate the input sequence
#model2.add(AdditiveAttention())
model18.add(BatchNormalization())
model18.add(Dense(1))
optimizer = Adam(learning_rate=0.0001)
model18.compile(optimizer=optimizer, loss='mean_squared_error')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with increased epochs and early stopping
model18.fit(all_X_train, all_y_train, validation_data=(all_X_val, all_y_val), epochs=200, verbose=1, callbacks=[early_stopping])

# Define the threshold for classification
threshold = 0.5

all_y_pred = model18.predict(all_X_val)
print(all_y_pred)

Epoch 1/200
7/7 [==============================] - 13s 515ms/step - loss: 1.3308 - val_loss: 2.5682
Epoch 2/200
7/7 [==============================] - 2s 228ms/step - loss: 1.1110 - val_loss: 2.5756
Epoch 3/200
7/7 [==============================] - 2s 225ms/step - loss: 0.9715 - val_loss: 2.5806
Epoch 4/200
7/7 [==============================] - 2s 228ms/step - loss: 0.9662 - val_loss: 2.5832
Epoch 5/200
7/7 [==============================] - 2s 245ms/step - loss: 0.9439 - val_loss: 2.5654
Epoch 6/200
7/7 [==============================] - 2s 362ms/step - loss: 0.9195 - val_loss: 2.5845
Epoch 7/200
7/7 [==============================] - 3s 359ms/step - loss: 0.9168 - val_loss: 2.5709
Epoch 8/200
7/7 [==============================] - 2s 241ms/step - loss: 0.8994 - val_loss: 2.5799
Epoch 9/200
7/7 [==============================] - 2s 243ms/step - loss: 0.9023 - val_loss: 2.5744
Epoch 10/200
7/7 [==============================] - 2s 233ms/step - loss: 0.8961 - val_loss: 2.5866
Epoch 11

In [304]:
# Calculate evaluation metrics
lcid_all_mae = np.mean(np.abs(all_y_val - all_y_pred))
lcid_all_mse = np.mean(np.square(all_y_val - all_y_pred))
lcid_all_rmse = np.sqrt(lcid_all_mse)
lcid_all_mape = np.mean(np.abs((all_y_val - all_y_pred) / all_y_val)) * 100

print("MAE:", lcid_all_mae)
print("MSE:", lcid_all_mse)
print("RMSE:", lcid_all_rmse)
print("MAPE:", lcid_all_mape)

MAE: 0.9863996150127308
MSE: 2.5494068257060207
RMSE: 1.5966862013889958
MAPE: 154.81569525988635


# LCID-grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

def build_model(layers=2, units=64, dropout_rate=0.1, learning_rate=0.001, time_steps=10):
    # Preprocess the data based on the given time_steps
    X_train, y_train = create_sequences(all_train_features, all_train_target, time_steps)
    X_val, y_val = create_sequences(all_val_features, all_val_target, time_steps)

    model = Sequential()
    for i in range(layers):
        if i == 0:
            model.add(Bidirectional(LSTM(units, input_shape=(time_steps, 9), return_sequences=True)))
        else:
            model.add(Bidirectional(LSTM(units, return_sequences=True)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, verbose=1, callbacks=[early_stopping])

    return model

# Define KerasRegressor with build_model
regressor = KerasRegressor(build_fn=build_model, epochs=200, verbose=1)

param_grid = {
    'layers': [3,5],
    'units': [64, 128],
    'dropout_rate': [0.1, 0.3],
    'learning_rate': [0.001, 0.0001],
    'time_steps': [10, 20, 30]
}

grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error')
grid_result = grid_search.fit(all_train_features, all_train_target)



Epoch 1/200


<ipython-input-307-b1db1dfcc8f2>:26: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  regressor = KerasRegressor(build_fn=build_model, epochs=200, verbose=1)


7/7 [==============================] - 22s 1s/step - loss: 0.8009 - val_loss: 2.2989
Epoch 2/200
7/7 [==============================] - 0s 55ms/step - loss: 0.8028 - val_loss: 2.1051
Epoch 3/200
7/7 [==============================] - 0s 57ms/step - loss: 0.7970 - val_loss: 2.0377
Epoch 4/200
7/7 [==============================] - 0s 55ms/step - loss: 0.7917 - val_loss: 2.0930
Epoch 5/200
7/7 [==============================] - 0s 58ms/step - loss: 0.7902 - val_loss: 2.1295
Epoch 6/200
7/7 [==============================] - 0s 54ms/step - loss: 0.7922 - val_loss: 2.1145
Epoch 7/200
7/7 [==============================] - 0s 55ms/step - loss: 0.7895 - val_loss: 2.0871
Epoch 8/200
7/7 [==============================] - 0s 54ms/step - loss: 0.7913 - val_loss: 2.0796
Epoch 9/200
7/7 [==============================] - 0s 54ms/step - loss: 0.7905 - val_loss: 2.0821
Epoch 10/200
7/7 [==============================] - 0s 56ms/step - loss: 0.7898 - val_loss: 2.0768
Epoch 11/200
7/7 [==============

Exception ignored in: <function _xla_gc_callback at 0x7f9969212310>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Epoch 1/200
Epoch 1/200
7/7 [==============================] - 26s 794ms/step - loss: 0.7829 - val_loss: 3.0753
Epoch 2/200
7/7 [==============================] - 2s 337ms/step - loss: 0.7774 - val_loss: 3.0552
Epoch 3/200
7/7 [==============================] - 4s 530ms/step - loss: 0.7795 - val_loss: 3.1030
Epoch 4/200
7/7 [==============================] - 3s 400ms/step - loss: 0.7793 - val_loss: 3.0625
Epoch 5/200
7/7 [==============================] - 3s 348ms/step - loss: 0.7757 - val_loss: 3.0521
Epoch 6/200
7/7 [==============================] - 2s 334ms/step - loss: 0.7748 - val_loss: 3.0645
Epoch 7/200
7/7 [==============================] - 2s 341ms/step - loss: 0.7757 - val_loss: 3.0841
Epoch 8/200
7/7 [==============================] - 3s 500ms/step - loss: 0.7780 - val_loss: 3.1176
Epoch 9/200
7/7 [==============================] - 3s 420ms/step - loss: 0.7824 - val_loss: 3.0629
Epoch 10/200
7/7 [==============================] - 2s 333ms/step - loss: 0.7756 - val_loss: 3.0

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f9956e5a310>
Traceback (most recent call last):
  File "/usr/lib/python3.9/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


7/7 [==============================] - 49s 1s/step - loss: 0.7696 - val_loss: 2.5468
Epoch 2/200
7/7 [==============================] - 1s 177ms/step - loss: 0.7675 - val_loss: 2.5388
Epoch 3/200
7/7 [==============================] - 1s 174ms/step - loss: 0.7673 - val_loss: 2.5404
Epoch 4/200
7/7 [==============================] - 1s 179ms/step - loss: 0.7665 - val_loss: 2.5497
Epoch 5/200
7/7 [==============================] - 1s 175ms/step - loss: 0.7654 - val_loss: 2.5536
Epoch 6/200
7/7 [==============================] - 1s 173ms/step - loss: 0.7670 - val_loss: 2.5531
Epoch 7/200
7/7 [==============================] - 1s 175ms/step - loss: 0.7644 - val_loss: 2.5536
Epoch 8/200
7/7 [==============================] - 1s 172ms/step - loss: 0.7674 - val_loss: 2.5491
Epoch 9/200
7/7 [==============================] - 2s 243ms/step - loss: 0.7666 - val_loss: 2.5539
Epoch 10/200
7/7 [==============================] - 2s 271ms/step - loss: 0.7672 - val_loss: 2.5602
Epoch 11/200
7/7 [=====